### Retrieval-Augmented Generation (RAG) Model for QA Bot on P&L Data

##### Dependencies Installation 

In [1]:
%pip install "unstructured[all-docs]" unstructured-client watermark python-dotenv pydantic langchain langchain-community langchain_core langchain_openai chromadb


Note: you may need to restart the kernel to use updated packages.


In [3]:
import watermark

%load_ext watermark

In [4]:
import warnings
warnings.filterwarnings('ignore')

In [5]:
import unstructured.partition

help(unstructured.partition)

Help on package unstructured.partition in unstructured:

NAME
    unstructured.partition

PACKAGE CONTENTS
    api
    auto
    common (package)
    csv
    doc
    docx
    email
    epub
    html (package)
    image
    json
    md
    model_init
    msg
    ndjson
    odt
    org
    pdf
    pdf_image (package)
    ppt
    pptx
    rst
    rtf
    strategies
    text
    text_type
    tsv
    utils (package)
    xlsx
    xml

FILE
    c:\users\manikpuriya\anaconda3\envs\unstructured\lib\site-packages\unstructured\partition\__init__.py




#### Loading the Document(pdf)

In [6]:
from unstructured.partition.pdf import partition_pdf

pdf_path = "Data\Sample Financial Statement.pdf"

elements = partition_pdf(filename=pdf_path)

In [7]:
print(elements)

[<unstructured.documents.elements.Text object at 0x0000026544545BD0>, <unstructured.documents.elements.Title object at 0x0000026544545F30>, <unstructured.documents.elements.Title object at 0x0000026544546200>, <unstructured.documents.elements.Text object at 0x00000265445461D0>, <unstructured.documents.elements.Text object at 0x00000265445462C0>, <unstructured.documents.elements.Text object at 0x00000265445457B0>, <unstructured.documents.elements.Text object at 0x0000026544546530>, <unstructured.documents.elements.ListItem object at 0x0000026544546500>, <unstructured.documents.elements.Text object at 0x0000026544547310>, <unstructured.documents.elements.Header object at 0x00000265443D4070>, <unstructured.documents.elements.NarrativeText object at 0x00000265443D40D0>, <unstructured.documents.elements.Title object at 0x00000265443D4970>, <unstructured.documents.elements.Text object at 0x00000265443D4C40>, <unstructured.documents.elements.Text object at 0x00000265443D4D30>, <unstructured.d

In [8]:
print(f"Length of elements: {len(elements)}")

Length of elements: 2194


In [9]:
import json

element_dict = [element.to_dict() for element in elements]
output = json.dumps(element_dict, indent=2)

In [10]:
print(output)

[
  {
    "type": "UncategorizedText",
    "element_id": "11e3d59a85507f6dff2c9f2f33b18a4f",
    "text": "INFOSYS LIMITED AND SUBSIDIARIES Condensed Consolidated Financial Statements under Indian Accounting Standards (Ind AS) for the three months and year ended March 31, 2024",
    "metadata": {
      "coordinates": {
        "points": [
          [
            142.46,
            50.78399999999999
          ],
          [
            142.46,
            139.86800000000005
          ],
          [
            591.9079999999999,
            139.86800000000005
          ],
          [
            591.9079999999999,
            50.78399999999999
          ]
        ],
        "system": "PixelSpace",
        "layout_width": 703.44,
        "layout_height": 910.32
      },
      "file_directory": "Data",
      "filename": "Sample Financial Statement.pdf",
      "languages": [
        "eng"
      ],
      "last_modified": "2025-01-17T01:30:57",
      "page_number": 1,
      "filetype": "appl

In [11]:
unique_element_type = set()

for element in element_dict:
    unique_element_type.add(element["type"])

In [12]:
print(unique_element_type)

{'NarrativeText', 'Title', 'Header', 'ListItem', 'Footer', 'UncategorizedText'}


#### Document Preprocessing

###### Extraction of Elements(Tables, Titles, Footer etc) of Document

In [13]:
import os
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

True

In [14]:
unstructured_api_key = os.getenv("UNSTRUCTURED_API_KEY")
unstructured_api_url = os.getenv("UNSTRUCTURED_API_URL")

In [15]:
from unstructured_client import UnstructuredClient

client = UnstructuredClient(
    api_key_auth=unstructured_api_key,
    server_url=unstructured_api_url
)

In [16]:
from unstructured_client.models import shared, operations
from unstructured_client.models.errors import SDKError
from unstructured.staging.base import dict_to_elements

In [17]:
with open(pdf_path, "rb") as f:
    files = shared.Files(
        content=f.read(),
        file_name="Sample Financial Statement.pdf"
    )
req = operations.PartitionRequest(
    partition_parameters=shared.PartitionParameters(
        files=files,
        strategy="hi_res",
        hi_res_model_name="yolox",
        skip_infer_table_types=[],  
        pdf_infer_table_structure=True
        )
)

try:
    resp = client.general.partition(request=req) 
    elements = dict_to_elements(resp.elements)
    print(elements) 
except SDKError as e:
    print(f"SDK Error: {e}")


INFO: HTTP Request: GET https://api.unstructured.io/general/docs "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.unstructured.io/general/v0/general "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.unstructured.io/general/v0/general "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.unstructured.io/general/v0/general "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.unstructured.io/general/v0/general "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.unstructured.io/general/v0/general "HTTP/1.1 200 OK"


[<unstructured.documents.elements.Title object at 0x00000265454411B0>, <unstructured.documents.elements.Text object at 0x0000026513F13BE0>, <unstructured.documents.elements.Text object at 0x00000265493835E0>, <unstructured.documents.elements.Table object at 0x00000265493835B0>, <unstructured.documents.elements.Title object at 0x0000026549383580>, <unstructured.documents.elements.Title object at 0x00000265493834F0>, <unstructured.documents.elements.Table object at 0x00000265493832E0>, <unstructured.documents.elements.NarrativeText object at 0x00000265493832B0>, <unstructured.documents.elements.NarrativeText object at 0x00000265493839D0>, <unstructured.documents.elements.Title object at 0x0000026549383310>, <unstructured.documents.elements.NarrativeText object at 0x0000026549383AF0>, <unstructured.documents.elements.Title object at 0x0000026549383760>, <unstructured.documents.elements.Title object at 0x0000026549383790>, <unstructured.documents.elements.Text object at 0x00000265493837F0>

In [18]:
unique_element_type = set()


for element in elements:
    unique_element_type.add(element.category)

In [19]:
print(unique_element_type)

{'Table', 'Image', 'NarrativeText', 'Title', 'ListItem', 'Footer', 'UncategorizedText'}


In [20]:
tables = [element for element in elements if element.category == "Table"]

In [21]:
print(tables)

[<unstructured.documents.elements.Table object at 0x00000265493835B0>, <unstructured.documents.elements.Table object at 0x00000265493832E0>, <unstructured.documents.elements.Table object at 0x0000026547C4E890>, <unstructured.documents.elements.Table object at 0x0000026547C4DC30>, <unstructured.documents.elements.Table object at 0x00000265493BD270>, <unstructured.documents.elements.Table object at 0x00000265493BFB80>, <unstructured.documents.elements.Table object at 0x000002654938C9A0>, <unstructured.documents.elements.Table object at 0x00000265494033A0>, <unstructured.documents.elements.Table object at 0x00000265494037C0>, <unstructured.documents.elements.Table object at 0x0000026549403040>, <unstructured.documents.elements.Table object at 0x0000026549403D30>, <unstructured.documents.elements.Table object at 0x0000026549400A90>, <unstructured.documents.elements.Table object at 0x00000265494003D0>, <unstructured.documents.elements.Table object at 0x0000026549400EB0>, <unstructured.docum

In [22]:
print(f"Length of tables: {len(tables)}")

Length of tables: 49


In [23]:
tables[1].text

'Condensed Consolidated Balance Sheets as at Note No. ASSETS Non-current assets Property, plant and equipment 2.2 Right-of-use assets 2.19 Capital work-in-progress Goodwill 2.3 Other intangible assets Financial assets Investments 2.4 Loans 2.5 Other financial assets 2.6 Deferred tax assets (net) Income tax assets (net) Other non-current assets 2.9 Total non-current assets Current assets Financial assets Investments 2.4 Trade receivables 2.7 Cash and cash equivalents 2.8 Loans 2.5 Other financial assets 2.6 Income tax assets (net) Other current assets 2.9 Total current assets Total assets EQUITY AND LIABILITIES Equity Equity share capital 2.11 Other equity Total equity attributable to equity holders of the Company Non-controlling interests Total equity Liabilities Non-current liabilities Financial Liabilities Lease liabilities 2.19 Other financial liabilities 2.12 Deferred tax liabilities (net) Other non-current liabilities 2.13 Total non-current liabilities Current liabilities Financia

In [24]:
tables[1].metadata

In [25]:
first_table_html = tables[1].metadata.text_as_html
print(first_table_html)

<table><thead><tr><th>Condensed Consolidated Balance Sheets as at</th><th>Note No.</th><th>March 31, 2024</th><th>(4~ crore March 31, 2023</th></tr></thead><tbody><tr><td colspan="4">ASSETS</td></tr><tr><td colspan="4">Non-current assets</td></tr><tr><td>Property, plant and equipment</td><td>22</td><td>12,370</td><td>13,346</td></tr><tr><td>Right-of-use assets</td><td>2.19</td><td>6,552</td><td>6,882</td></tr><tr><td>Capital work-in-progress</td><td></td><td>293</td><td>288</td></tr><tr><td>Goodwi</td><td>2.3</td><td>7,303</td><td>7,248</td></tr><tr><td>Other intangible assets</td><td></td><td>1,397</td><td>1,749</td></tr><tr><td colspan="4">Financial assets</td></tr><tr><td>Investments</td><td>2.4</td><td>11,708</td><td>12,569</td></tr><tr><td>Loans</td><td>2.5</td><td>34</td><td>39</td></tr><tr><td>Other financial assets</td><td>2.6</td><td>3,105</td><td>2,798</td></tr><tr><td>Deferred tax assets (net)</td><td></td><td>454</td><td>1,245</td></tr><tr><td>ncome tax assets (net)</td><td

In [26]:
from io import StringIO
from lxml import etree

parser = etree.XMLParser(remove_blank_text=True)
file_obj = StringIO(first_table_html)
tree = etree.parse(file_obj, parser)
print(etree.tostring(tree, pretty_print=True).decode())

<table>
  <thead>
    <tr>
      <th>Condensed Consolidated Balance Sheets as at</th>
      <th>Note No.</th>
      <th>March 31, 2024</th>
      <th>(4~ crore March 31, 2023</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td colspan="4">ASSETS</td>
    </tr>
    <tr>
      <td colspan="4">Non-current assets</td>
    </tr>
    <tr>
      <td>Property, plant and equipment</td>
      <td>22</td>
      <td>12,370</td>
      <td>13,346</td>
    </tr>
    <tr>
      <td>Right-of-use assets</td>
      <td>2.19</td>
      <td>6,552</td>
      <td>6,882</td>
    </tr>
    <tr>
      <td>Capital work-in-progress</td>
      <td/>
      <td>293</td>
      <td>288</td>
    </tr>
    <tr>
      <td>Goodwi</td>
      <td>2.3</td>
      <td>7,303</td>
      <td>7,248</td>
    </tr>
    <tr>
      <td>Other intangible assets</td>
      <td/>
      <td>1,397</td>
      <td>1,749</td>
    </tr>
    <tr>
      <td colspan="4">Financial assets</td>
    </tr>
    <tr>
      <td>Investments</td>
      <

In [27]:
from IPython.core.display import HTML

HTML(first_table_html)

In [28]:
import pandas as pd

dfs = pd.read_html(first_table_html)
df = dfs[0]
df.head()

,Condensed Consolidated Balance Sheets as at,Note No.,"March 31, 2024","(4~ crore March 31, 2023",Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7
0,ASSETS,ASSETS,ASSETS,ASSETS,NaN,NaN,NaN,NaN
1,Non-current assets,Non-current assets,Non-current assets,Non-current assets,NaN,NaN,NaN,NaN
2,"Property, plant and equipment",22,12370,13346,NaN,NaN,NaN,NaN
3,Right-of-use assets,2.19,6552,6882,NaN,NaN,NaN,NaN
4,Capital work-in-progress,NaN,293,288,NaN,NaN,NaN,NaN


In [29]:
texts = [el for el in elements if el.category != "Text"]
texts[500].text

'At the date of commencement of the lease, the Group recognizes a right-of-use asset (“ROU”) and a corresponding lease liability for all lease arrangements in which it is a lessee, except for leases with a term of twelve months or less (short-term leases) and low value leases. For these short-term and low value leases, the Group recognizes the lease payments as an operating expense on a straight-line basis over the term of the lease.'

In [30]:
extracted_text = ""

for cat in elements:
    if cat.category == "Formula":
        extracted_text += cat.text + "\n"
    if cat.category == "FigureCaption":
        extracted_text += cat.text + "\n"
    if cat.category == "NarrativeText":
        extracted_text += cat.text + "\n"
    if cat.category == "ListItem":
        extracted_text += cat.text + "\n"
    if cat.category == "Title":
        extracted_text += cat.text + "\n"
    if cat.category == "Address":
        extracted_text += cat.text + "\n"
    if cat.category == "EmailAddress":
        extracted_text += cat.text + "\n"
    if cat.category == "Table":
        extracted_text += cat.metadata.text_as_html + "\n"
    if cat.category == "Header":
        extracted_text += cat.text + "\n"
    if cat.category == "Footer":
        extracted_text += cat.text + "\n"
    if cat.category == "CodeSnippet":
        extracted_text += cat.text + "\n"
    if cat.category == "UncategorizedText":
        extracted_text += cat.text + "\n"

In [31]:
print(extracted_text)

INFOSYS LIMITED AND SUBSIDIARIES Condensed Consolidated Financial Statements under Indian Accounting Standards (Ind AS) for the three months and year ended March 31, 2024
1.
2.
<table><thead><tr><th>ndex</th><th>Page No.</th></tr></thead><tbody><tr><td>“ondensed Consolidated Balance Sheet</td><td>1</td></tr><tr><td>“ondensed Consolidated Statement of Profit and Loss</td><td></td></tr><tr><td>“ondensed Consolidated Statement of Changes in Equity</td><td>E</td></tr><tr><td>“ondensed Consolidated Statement of Cash Flows</td><td>5</td></tr><tr><td colspan="2">)verview and Notes to the Interim Condensed Consolidated Financial Statements</td></tr><tr><td colspan="2">. Overview</td></tr><tr><td>1.1 Company overview</td><td></td></tr><tr><td>1.2 Basis of preparation of financial statements</td><td>R 7</td></tr><tr><td>1.3 Basis of consolidation</td><td>S</td></tr><tr><td>1.4 Use of estimates and judgments</td><td>7</td></tr><tr><td>1.5 Critical accounting estimates and judgments....</td><td>[ 

In [32]:
from IPython.core.display import Markdown

Markdown(extracted_text)

INFOSYS LIMITED AND SUBSIDIARIES Condensed Consolidated Financial Statements under Indian Accounting Standards (Ind AS) for the three months and year ended March 31, 2024
1.
2.
<table><thead><tr><th>ndex</th><th>Page No.</th></tr></thead><tbody><tr><td>“ondensed Consolidated Balance Sheet</td><td>1</td></tr><tr><td>“ondensed Consolidated Statement of Profit and Loss</td><td></td></tr><tr><td>“ondensed Consolidated Statement of Changes in Equity</td><td>E</td></tr><tr><td>“ondensed Consolidated Statement of Cash Flows</td><td>5</td></tr><tr><td colspan="2">)verview and Notes to the Interim Condensed Consolidated Financial Statements</td></tr><tr><td colspan="2">. Overview</td></tr><tr><td>1.1 Company overview</td><td></td></tr><tr><td>1.2 Basis of preparation of financial statements</td><td>R 7</td></tr><tr><td>1.3 Basis of consolidation</td><td>S</td></tr><tr><td>1.4 Use of estimates and judgments</td><td>7</td></tr><tr><td>1.5 Critical accounting estimates and judgments....</td><td>[ R 8</td></tr><tr><td colspan="2">. Notes to the Interim Condensed Consolidated Financial Statements</td></tr><tr><td>2.1 Business Combinations</td><td>10</td></tr><tr><td>2.2 Property, plant and equipment</td><td>12</td></tr><tr><td>2.3 Goodwill and other intangible assets</td><td>14</td></tr><tr><td>2.4 Investments</td><td>15</td></tr><tr><td>2.5 Loans</td><td>16</td></tr><tr><td>2.6 Other financial assets</td><td>16</td></tr><tr><td>2.7 Trade receivables</td><td>16</td></tr><tr><td>2.8 Cash and cash equivalents ..</td><td>17</td></tr><tr><td></td><td></td></tr><tr><td>2.9 Other assets</td><td>17</td></tr><tr><td>2.10 Financial instruments</td><td>18</td></tr><tr><td>2.11 Equity</td><td>22</td></tr><tr><td>2.12 Other financial liabilities</td><td>25</td></tr><tr><td>2.13 Other liabilities</td><td>25</td></tr><tr><td>2.14 Provisions</td><td>26</td></tr><tr><td>2.15 Income taxes</td><td>27</td></tr><tr><td>2.16 Revenue from operations</td><td>28</td></tr><tr><td>2.17 Other income, net</td><td>30</td></tr><tr><td>2.18 Expenses</td><td>31</td></tr><tr><td>2.19 Leases</td><td>32</td></tr><tr><td>2.20 Basic and diluted shares used in computing earnings per equity share</td><td>34</td></tr><tr><td>2.21 Contingent liabilities and commitments</td><td>34</td></tr><tr><td>2.22 Related party transactions</td><td>36</td></tr><tr><td>2.23 Segment reporting</td><td>37</td></tr><tr><td>2.24 Function wise classification of Condensed Consolidated Statement of Profit and Loss</td><td>39</td></tr></tbody></table>
No.
INFOSYS LIMITED AND SUBSIDIARIES
<table><thead><tr><th>Condensed Consolidated Balance Sheets as at</th><th>Note No.</th><th>March 31, 2024</th><th>(4~ crore March 31, 2023</th></tr></thead><tbody><tr><td colspan="4">ASSETS</td></tr><tr><td colspan="4">Non-current assets</td></tr><tr><td>Property, plant and equipment</td><td>22</td><td>12,370</td><td>13,346</td></tr><tr><td>Right-of-use assets</td><td>2.19</td><td>6,552</td><td>6,882</td></tr><tr><td>Capital work-in-progress</td><td></td><td>293</td><td>288</td></tr><tr><td>Goodwi</td><td>2.3</td><td>7,303</td><td>7,248</td></tr><tr><td>Other intangible assets</td><td></td><td>1,397</td><td>1,749</td></tr><tr><td colspan="4">Financial assets</td></tr><tr><td>Investments</td><td>2.4</td><td>11,708</td><td>12,569</td></tr><tr><td>Loans</td><td>2.5</td><td>34</td><td>39</td></tr><tr><td>Other financial assets</td><td>2.6</td><td>3,105</td><td>2,798</td></tr><tr><td>Deferred tax assets (net)</td><td></td><td>454</td><td>1,245</td></tr><tr><td>ncome tax assets (net)</td><td></td><td>3,045</td><td>6,453</td></tr><tr><td>Other non-current assets</td><td>2.9</td><td>2,121</td><td>2,318</td></tr><tr><td>Total non-current assets</td><td></td><td>48,382</td><td>54,935</td></tr><tr><td colspan="4">Current assets</td></tr><tr><td colspan="4">Financial assets</td></tr><tr><td>Investments</td><td>2.4</td><td>12,915</td><td>6,909</td></tr><tr><td>Trade receivables</td><td>2.7</td><td>30,193</td><td>25,424</td></tr><tr><td>Cash and cash equivalents</td><td>2.8</td><td>14,786</td><td>12,173</td></tr><tr><td>Loans</td><td>2.5</td><td>248</td><td>289</td></tr><tr><td>Other financial assets</td><td>2.6</td><td>12,085</td><td>11,604</td></tr><tr><td>Income tax assets (net)</td><td></td><td>6,397</td><td>6</td></tr><tr><td>Other current assets</td><td>2.9</td><td>12,808</td><td>14,476</td></tr><tr><td>Total current assets</td><td></td><td>89,432</td><td>70,881</td></tr><tr><td>Total assets</td><td></td><td>137,814</td><td>125,816</td></tr><tr><td colspan="4" rowspan="2">EQUITY AND LIABILITIES Equity</td></tr><tr><td>Equity share capital</td><td>2.11</td><td>2,071</td><td>2,069</td></tr><tr><td>Other equity</td><td></td><td>86,045</td><td>73,338</td></tr><tr><td>Total equity attributable to equity holders of the C ompany</td><td></td><td>88,116</td><td>75,407</td></tr><tr><td>Non-controlling interests</td><td></td><td>345</td><td>388</td></tr><tr><td>Total equity</td><td></td><td>88,461</td><td>75,795</td></tr><tr><td colspan="4">Liabilities
 Non-current liabilities</td></tr><tr><td>Financial Liabilities</td><td></td><td></td><td></td></tr><tr><td>Lease liabilities</td><td>2.19</td><td>6,400</td><td>7,057</td></tr><tr><td>Other financial liabilities</td><td>2.12</td><td>2,130</td><td>2,058</td></tr><tr><td>Deferred tax liabilities (net)</td><td></td><td>1,794</td><td>1,220</td></tr><tr><td>Other non-current liabilities</td><td>2.13</td><td>235</td><td>500</td></tr><tr><td>Total non-current liabilities</td><td></td><td>10,559</td><td>10,835</td></tr><tr><td colspan="4">Current liabilities</td></tr><tr><td>Financial Liabilities</td><td></td><td></td><td></td></tr><tr><td>Lease liabilities</td><td>2.19</td><td>1,959</td><td>1,242</td></tr><tr><td>Trade payables</td><td></td><td>3,956</td><td>3,865</td></tr><tr><td>Other financial liabilities</td><td>2.12</td><td>16,959</td><td>18,558</td></tr><tr><td>Other current liabilities</td><td>2.13</td><td>10,539</td><td>10,830</td></tr><tr><td>Provisions</td><td>2.14</td><td>1,796</td><td>1,307</td></tr><tr><td>Income tax liabilities (net)</td><td></td><td>3,585</td><td>3,384</td></tr><tr><td>Total current liabilities</td><td></td><td>38,794</td><td>39,186</td></tr><tr><td>Total equity and liabilities</td><td></td><td>137,814</td><td>125,816</td></tr></tbody></table>
The accompanying notes form an integral part of the interim condensed consolidated financial statements
As per our report of even date attached
for Deloitte Haskins & Sells LLP
for and on behalf of the Board of Directors of Infosys Limited
Chartered Accountants
Firm’s Registration No :
117366W/ W-100018
(In ₹ crore )
39
6
Sanjiv V. Pilgaonkar Partner Membership No. 039826
D. Sundaram Lead Independent Director
Salil Parekh Chief Executive Officer and Managing Director
Bobby Parikh Director
Bengaluru April 18, 2024
Jayesh Sanghrajka Chief Financial Officer
1
A.G.S. Manikantha Company Secretary
INFOSYS LIMITED AND SUBSIDIARIES
(In % crore, except equity share and per equity share data)
<table><thead><tr><th rowspan="2">Condensed Consolidated Statement of Profit and L oss for the</th><th rowspan="2">Note No.</th><th colspan="4">in &lt; crore, except equily share ana per equity share data)</th></tr><tr><th>Three months ended</th><th colspan="3">March 31, Y ear ended March 31,</th></tr></thead><tbody><tr><td>Revenue from operations</td><td>2.16</td><td>2024 37,923</td><td>2023 37,44</td><td>2024 153,670</td><td>2023 146,767</td></tr><tr><td>Other income, net</td><td>2.17</td><td>2,729</td><td>67</td><td>4,711</td><td>2,701</td></tr><tr><td>Total income</td><td></td><td>40,652</td><td>38,112</td><td>158,381</td><td>149,468</td></tr><tr><td>Expenses</td><td></td><td></td><td></td><td></td><td></td></tr><tr><td>Employee benefit expenses</td><td>2.18</td><td>20,393</td><td>20,31</td><td>82,620</td><td>78,359</td></tr><tr><td>Cost of technical sub-contractors</td><td></td><td>2,967</td><td>3,116</td><td>12,232</td><td>14,062</td></tr><tr><td>Travel expenses</td><td></td><td>471</td><td>426</td><td>1,759</td><td>1,525</td></tr><tr><td>Cost of software packages and others</td><td>2.18</td><td>3,687</td><td>2,886</td><td>13,515</td><td>10,902</td></tr><tr><td>Communication expenses</td><td></td><td>147</td><td>7</td><td>677</td><td>713</td></tr><tr><td>Consultancy and professional charges</td><td></td><td>489</td><td>387</td><td>1,726</td><td>1,684</td></tr><tr><td>Depreciation and amortization expenses</td><td></td><td>1,163</td><td>1,12</td><td>4,678</td><td>4,225</td></tr><tr><td>Finance cost</td><td></td><td>110</td><td>82</td><td>470</td><td>284</td></tr><tr><td>Other expenses Total</td><td>2.18</td><td>985</td><td>1,146</td><td>4,716</td><td>4,392</td></tr><tr><td>expenses</td><td></td><td>30,412</td><td>29,646</td><td>122,393</td><td>116,146</td></tr><tr><td colspan="6">Profit before tax 10,240 8,466 35,988 33,322</td></tr><tr><td>Tax expense:</td><td></td><td></td><td></td><td></td><td></td></tr><tr><td>Current tax</td><td>2.15</td><td>1,173</td><td>2,260</td><td>8,390</td><td>9,287</td></tr><tr><td>Deferred tax</td><td>2.15</td><td>1,092</td><td>72</td><td>1,350</td><td>(73)</td></tr><tr><td>Profit for the period</td><td></td><td>7,975</td><td>6,134</td><td>26,248</td><td>24,108</td></tr><tr><td colspan="6">Other comprehensive income</td></tr><tr><td colspan="6">tems that will not be reclassified subsequently to profit or loss</td></tr><tr><td>Remeasurement of the net defined benefit liability/asset, net</td><td></td><td>26</td><td>25</td><td>120</td><td></td></tr><tr><td>Equity instruments through other comprehensive income, net</td><td></td><td>(12)</td><td>(15)</td><td>19</td><td>(N</td></tr><tr><td></td><td></td><td>14</td><td>10</td><td>139</td><td></td></tr><tr><td>that will be reclassified loss</td><td></td><td></td><td></td><td></td><td></td></tr><tr><td>tems subsequently to profit or</td><td></td><td></td><td></td><td></td><td></td></tr><tr><td>Fair value changes on derivatives designated as cash flow hedge, net</td><td></td><td>28</td><td>36</td><td>11</td><td>(7)</td></tr><tr><td>Exchange differences on translation of foreign operations</td><td></td><td>(231)</td><td>61</td><td>226</td><td>776</td></tr><tr><td>Fair value changes on investments, net</td><td></td><td>37</td><td>42</td><td>144</td><td>(256)</td></tr><tr><td></td><td></td><td>(166)</td><td>139</td><td>381</td><td>513</td></tr><tr><td>Total other comprehensive income /(loss), net of tax</td><td></td><td>(152)</td><td>149</td><td>520</td><td>514</td></tr><tr><td>Total comprehensive income for the period</td><td></td><td>7,823</td><td>6,283</td><td>26,768</td><td>24,622</td></tr><tr><td>Profit attributable to:</td><td></td><td></td><td></td><td></td><td></td></tr><tr><td>Owners of the Company</td><td></td><td>7,969</td><td>6,128</td><td>26,233</td><td>24,095</td></tr><tr><td>Non-controlling interests</td><td></td><td>6</td><td>6</td><td>15</td><td>13</td></tr><tr><td></td><td></td><td>7,975</td><td>6,134</td><td>26,248</td><td>24,108</td></tr><tr><td colspan="6">Total comprehensive income attributable to:</td></tr><tr><td>Owners of the Company</td><td></td><td>7,821</td><td>6,276</td><td>26,754</td><td>24,598</td></tr><tr><td>Non-controlling interests</td><td></td><td>2 7,823</td><td>7 6,283</td><td>14 26,768</td><td>24 24,622</td></tr><tr><td colspan="6">Earnings per equity share</td></tr><tr><td colspan="6">Equity shares of par value Z5/- each</td></tr><tr><td>Basic (in X per share)</td><td></td><td>19.25</td><td>14.79</td><td>63.39</td><td>57.63</td></tr><tr><td>Diluted (in % per share)</td><td></td><td>19.22</td><td>14.77</td><td>63.29</td><td>57.54</td></tr><tr><td colspan="6">Weighted average equity shares used in computing earnings per equity share</td></tr><tr><td>Basic (in shares)</td><td>2.20</td><td>4,139,432,133</td><td>4,144,013,195</td><td>4,138,568,090</td><td>4,180,897,857</td></tr><tr><td>Diluted (in shares)</td><td>2.20</td><td>4,145,052,370</td><td>4,149,555,426</td><td>4,144,680,425</td><td>4,187,731,070</td></tr></tbody></table>
The accompanying notes form an integral part of the interim condensed consolidated financial statements
As per our report of even date attached
for Deloitte Haskins & Sells LLP
for and on behalf of the Board of Directors of Infosys Limited
Chartered Accountants
Firm’s Registration No :
117366W/ W-100018
o

Sanjiv V. Pilgaonkar Partner Membership No. 039826
D. Sundaram Lead Independent Director
Salil Parekh Chief Executive Officer and Managing Director
Bobby Parikh Director
Bengaluru April 18, 2024
Jayesh Sanghrajka Chief Financial Officer
2
A.G.S. Manikantha Company Secretary
INFOSYS LIMITED AND SUBSIDIARIES
Condensed Consolidated Statement of Changes in Equity
<table><thead><tr><th rowspan="2">Particulars</th><th rowspan="2">Equity 
 Share capital</th><th rowspan="2">Capital 
 reserve</th><th colspan="5">OTHER EQUITY</th><th></th><th colspan="4"></th><th rowspan="2">Total equity attributable to equity holders of the Company</th><th rowspan="2">Non- controlling 
 interest</th><th rowspan="2">Total equity</th></tr><tr><th>Capital 
 redemption 
 reserve</th><th>Securities Premium</th><th>Reserves &amp; Surplus Retained earnings</th><th>General Share reserve Options 
 Outstanding 
 Account</th><th>Special 
 Economic Zone Re- investment 
 @ reserve</th><th>Other (&amp;) reserves</th><th>Equity 
 instruments 
 through other comprehensive 
 income</th><th>Other comprehensive Exchange 
 differences translating the financial statements of foreign operation</th><th>income Effective portion of Cash Flow Hedges</th><th>Other items of other comprehensive 
 income / (loss)</th></tr></thead><tbody><tr><td>Balance as at April 1, 2022</td><td>2,098</td><td>54</td><td>139</td><td>200</td><td>61,313</td><td>1,061</td><td>8,339</td><td>16</td><td>254</td><td>1,560</td><td></td><td>(292)</td><td>75,350</td><td>386</td><td>75,736</td></tr><tr><td>mpact on adoption of amendment to Ind AS 37*</td><td></td><td></td><td></td><td></td><td>(19)</td><td></td><td>—                 —                 —</td><td></td><td>—</td><td></td><td></td><td></td><td>(19)</td><td></td><td>(19)</td></tr><tr><td></td><td>2,098</td><td>54</td><td>139</td><td>200</td><td>61,294</td><td>1,061</td><td>8,339</td><td>16</td><td>254</td><td>1,560</td><td></td><td>(292)</td><td>75,331</td><td>386</td><td>75,717</td></tr><tr><td colspan="16">Changes in equity for the year ended March 31, 2023</td></tr><tr><td>Profit for the period</td><td></td><td></td><td></td><td></td><td>24,095</td><td></td><td>—                 —                 —</td><td></td><td></td><td>—                             —                      —</td><td></td><td>—</td><td>24,095</td><td>13</td><td>24,108</td></tr><tr><td>Remeasurement of the net defined benefit liability/asset, net*</td><td>—                  —                  —                 —                 —                 —</td><td></td><td></td><td></td><td></td><td></td><td>—                 —                 —</td><td></td><td></td><td>—                             —                      —                          8</td><td></td><td></td><td>8</td><td>—</td><td>8</td></tr><tr><td>Equity instruments through other comprehensive income, net*</td><td>—                  —                  —                 —                 —                 —</td><td></td><td></td><td></td><td></td><td></td><td>—                 —                 —</td><td></td><td>(7</td><td></td><td></td><td></td><td>(7)</td><td></td><td>(7)</td></tr><tr><td>Fair value changes on derivatives designated as cash flow hedge, net*</td><td></td><td></td><td></td><td></td><td></td><td></td><td>—                 —                 —</td><td></td><td></td><td></td><td>U]</td><td></td><td>(7</td><td></td><td>(7</td></tr><tr><td>Exchange differences on translation of foreign operations</td><td>—                  —                  —                 —                 —                 —</td><td></td><td></td><td></td><td></td><td></td><td>—                 —                 —</td><td></td><td>—</td><td>765</td><td></td><td>—</td><td>765</td><td>11</td><td>776</td></tr><tr><td>Fair value changes on investments, net*</td><td>—                  —                  —                 —                 —                 —</td><td></td><td></td><td></td><td></td><td></td><td>—                 —                 —</td><td></td><td></td><td></td><td></td><td>(256)</td><td>(256)</td><td></td><td>(256)</td></tr><tr><td>Total Comprehensive income for the period</td><td></td><td></td><td></td><td></td><td>24,095</td><td></td><td>—                 —                 —</td><td></td><td>(7)</td><td>765</td><td>(7)</td><td>(248)</td><td>24,598</td><td>24</td><td>24,622</td></tr><tr><td>Shares issued on exercise of employee stock options (Refer to Note 2.11)</td><td>1</td><td></td><td></td><td>34</td><td>.</td><td></td><td>—                 —                 —</td><td></td><td></td><td>—                             —                      —</td><td></td><td>—</td><td>35</td><td>—</td><td>35</td></tr><tr><td>Employee stock compensation expense (Refer to Note 2.11)</td><td>—                  —                  —                 —                 —                 —</td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td>—                             —                      —</td><td></td><td>—</td><td>514</td><td>—</td><td>514</td></tr><tr><td>Transferred on account of options not exercised</td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td>—                             —                      —</td><td></td><td>—</td><td>—</td><td>—</td><td>—</td></tr><tr><td>Buyback of equity shares (Refer to Note 2.11) **</td><td>(30)</td><td></td><td></td><td>(340)</td><td>(11,096)</td><td></td><td>—                 —                 —</td><td></td><td></td><td>—                             —                      —</td><td></td><td></td><td>(11,466)</td><td></td><td>(11,466)</td></tr><tr><td>Transaction costs relating to buyback*</td><td></td><td></td><td></td><td>(19)</td><td>(5)</td><td></td><td>—                 —                 —</td><td></td><td></td><td>—                             —                      —</td><td></td><td></td><td>(24)</td><td></td><td>(24)</td></tr><tr><td>Amount transferred to capital redemption reserve upon buyback</td><td></td><td></td><td>30</td><td></td><td>(21</td><td>9)</td><td>—                 —                 —</td><td></td><td></td><td>—                             —                      —</td><td></td><td>—</td><td>—</td><td>—</td><td>—</td></tr><tr><td>Transferred to Special Economic Zone Re-investment reserve</td><td></td><td></td><td></td><td></td><td>(3,139 i</td><td></td><td>3,139</td><td></td><td></td><td>—                             —                      —</td><td></td><td>—</td><td>—</td><td>—</td><td>—</td></tr><tr><td>Transfer to legal reserve</td><td></td><td></td><td></td><td></td><td>©)</td><td></td><td>—                 —                   3</td><td></td><td></td><td>—                             —                      —</td><td></td><td>—</td><td>—</td><td>—</td><td>—</td></tr><tr><td>Transferred on account of exercise of stock options (Refer to note 2.11)</td><td></td><td></td><td></td><td>291</td><td>.</td><td></td><td></td><td></td><td></td><td>—                             —                      —</td><td></td><td>—</td><td>—</td><td>—</td><td>—</td></tr><tr><td>ncome tax benefit arising on exercise of stock options</td><td>—                  —                  —                 —                 —                 —</td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td>—                             —                      —</td><td></td><td>—</td><td>51</td><td>—</td><td>51</td></tr><tr><td>Dividends "</td><td></td><td></td><td></td><td></td><td>(13,632)</td><td></td><td>—                 —                 —</td><td></td><td></td><td>—                             —                      —</td><td></td><td></td><td>(13,632)</td><td></td><td>(13,632)</td></tr><tr><td>Dividends paid to non controlling interest of subsidiary</td><td>—                  —                  —                 —                 —                 —</td><td></td><td></td><td></td><td></td><td></td><td>—                 —                 —</td><td></td><td></td><td>—                             —                      —</td><td></td><td>—</td><td></td><td>(22)</td><td>(22)</td></tr><tr><td>Transferred from Special Economic Zone Re-investment reserve on utilization</td><td></td><td></td><td></td><td></td><td>1,464</td><td></td><td>(1,464)</td><td></td><td></td><td>—                             —                      —</td><td></td><td>—</td><td>—</td><td>—</td><td>—</td></tr><tr><td>Balance as at March 31, 2023</td><td>2,069</td><td>54</td><td>169</td><td>166</td><td>58,957</td><td>1,054</td><td>10,014</td><td>19</td><td>247</td><td>2,325</td><td>()</td><td>(540)</td><td>75,407</td><td>388</td><td>75,795</td></tr></tbody></table>
(In ₹ crore)
3
Condensed Consolidated Statement of Changes in Equity (contd.)
Firm’s Registration No :
117366W/ W-100018
Sanjiv V. Pilgaonkar Partner Membership No. 039826
D. Sundaram
Lead Independent Director
Salil Parekh
Chief Executive Officer
and Managing Director
Bobby Parikh
Director
Bengaluru April 18, 2024
Jayesh Sanghrajka
Chief Financial Officer
A.G.S. Manikantha
Company Secretary
4
INFOSYS LIMITED AND SUBSIDIARIES
Condensed Consolidated Statement of Cash Flows
Accounting policy
Cash flows are reported using the indirect method, whereby profit for the period is adjusted for the effects of transactions of a non-cash nature, any deferrals or accruals of past or future operating cash receipts or payments and item of income or expenses associated with investing or financing cash flows. The cash flows from operating, investing and financing activities of the Group are segregated. The Group considers all highly liquid investments that are readily convertible to known amounts of cash to be cash equivalents.
<table><thead><tr><th rowspan="2">Particulars</th><th rowspan="2">Note No.</th><th colspan="2">(In X crore)</th></tr><tr><th>Y ear ended March</th><th>31,</th></tr></thead><tbody><tr><td colspan="4">2024
 2023
 Cash flow from operating activities</td></tr><tr><td colspan="4">Profit for the period 26,248 24,108</td></tr><tr><td colspan="4">Adjustments to reconcile net profit to net cash provided by operating activities:</td></tr><tr><td>ncome tax</td><td>2.15</td><td>9,740</td><td>9,214</td></tr><tr><td>expense and amortization</td><td></td><td>4,678</td><td>4,225</td></tr><tr><td>Depreciation nterest and dividend income</td><td></td><td>(2,067)</td><td>(1,817)</td></tr><tr><td>Finance cost</td><td></td><td>470</td><td>284</td></tr><tr><td>loss under credit loss model</td><td></td><td>121</td><td>283</td></tr><tr><td>mpairment recognized / (reversed) expected differences translation of and</td><td></td><td>76</td><td>161</td></tr><tr><td>Exchange on assets liabilities, net</td><td></td><td></td><td></td></tr><tr><td>Stock compensation expense</td><td></td><td>652</td><td>519</td></tr><tr><td>nterest on income tax refund</td><td></td><td>(1,934)</td><td>i</td></tr><tr><td>Provision for post sale client support</td><td></td><td>75</td><td>120</td></tr><tr><td colspan="4">Other adjustments 1,464 508</td></tr><tr><td>Changes in assets and liabilities</td><td></td><td></td><td></td></tr><tr><td>Trade receivables and unbilled revenue Loans, other financial assets and other assets</td><td></td><td>(2,667) (1,172)</td><td>(7,076) (3,108)</td></tr><tr><td>Trade payables</td><td></td><td>91</td><td>(279)</td></tr><tr><td>Other financial other liabilities and</td><td></td><td></td><td></td></tr><tr><td>liabilities, provisions</td><td></td><td>(1,334)</td><td>4,119</td></tr><tr><td>Cash generated from operations</td><td></td><td>34,441</td><td>31,261</td></tr><tr><td>Income taxes paid</td><td></td><td>(9,231)</td><td>(8,794)</td></tr><tr><td>Net cash generated by operating activities</td><td></td><td>25,210</td><td>22,467</td></tr><tr><td>investing Expenditure on property, plant and equipment and intangibles</td><td></td><td>(2,201)</td><td>(2,579)</td></tr><tr><td>Deposits placed with corporation</td><td></td><td>(847)</td><td>(996)</td></tr><tr><td>Redemption of deposits placed with Corporation</td><td></td><td>710</td><td>762</td></tr><tr><td>nterest and dividend received</td><td></td><td></td><td></td></tr><tr><td></td><td></td><td>1,768</td><td>1,525</td></tr><tr><td>towards of business, net of cash</td><td>2.1</td><td>—</td><td></td></tr><tr><td>Payment acquisition acquired</td><td></td><td></td><td>(910)</td></tr><tr><td>Payment of contingent consideration pertaining to acquisition of business</td><td></td><td>(101)</td><td>(60)</td></tr><tr><td>Escrow and other deposits pertaining to Buyback</td><td></td><td>—</td><td>(483)</td></tr><tr><td>Redemption of escrow and other deposits pertaining to Buyback</td><td></td><td>—</td><td>483</td></tr><tr><td>Other</td><td></td><td>128</td><td>7</td></tr><tr><td>receipts</td><td></td><td></td><td></td></tr><tr><td>Payments to acquire Investments mutual fund</td><td></td><td></td><td></td></tr><tr><td>Liquid units fund units</td><td></td><td>(66,191)</td><td>(70,631 (400)</td></tr><tr><td>Target maturity Certificates of</td><td></td><td></td><td></td></tr><tr><td>deposit</td><td></td><td>(8,509</td><td>(10,348</td></tr><tr><td>Commercial Papers</td><td></td><td>(10,387 g</td><td>(3,003</td></tr><tr><td>Non-convertible debentures</td><td></td><td>(1,526</td><td>(249)</td></tr><tr><td>Tax free bonds and government bonds</td><td></td><td></td><td>(@7)</td></tr><tr><td>Government securities</td><td></td><td></td><td>(1,569</td></tr><tr><td>Other Investments</td><td></td><td>(14)</td><td>(20)</td></tr><tr><td colspan="4">Proceeds on sale of Investments</td></tr><tr><td>Tax free bonds and government bonds</td><td></td><td>150</td><td>22</td></tr><tr><td>Liquid mutual funds units</td><td></td><td>64,767</td><td>71,85</td></tr><tr><td>Certificates of deposit</td><td></td><td>9,205</td><td>10,404</td></tr><tr><td>Commercial Papers</td><td></td><td>6,479</td><td>2,298</td></tr><tr><td>Non-convertible debentures</td><td></td><td>1,230</td><td>470</td></tr><tr><td>Government securities</td><td></td><td>304</td><td>1,882</td></tr><tr><td>Equity and preference securities</td><td></td><td>26</td><td>99</td></tr><tr><td>Net cash generated / (used in) from investing activities</td><td></td><td>(5,009)</td><td>(1,209)</td></tr></tbody></table>
5
<table><tbody><tr><td>Payment of lease liabilities</td><td>@ 2,024)</td><td>(1,231)</td></tr><tr><td>Payment of dividends</td><td>(14,692</td><td>(13,631)</td></tr><tr><td>Payment of dividend to non-controlling interest of subsidiary</td><td>(39)</td><td>(22)</td></tr><tr><td>Payment towards buyback of shares pertaining to non controlling interest of subsidiary</td><td>(18)</td><td>—</td></tr><tr><td>Shares issued on exercise of employee stock options</td><td></td><td>35</td></tr><tr><td>Other receipts</td><td></td><td>132</td></tr><tr><td>Other payments</td><td>(736)</td><td>(479)</td></tr><tr><td>Buyback of equity shares including transaction cost and tax on buyback</td><td></td><td>(11,499)</td></tr><tr><td>Net cash used in financing activities</td><td>(17,504;</td><td>(26,695)</td></tr><tr><td>Net increase / (decrease) in cash and cash equivalents</td><td>2,697</td><td>(5,437)</td></tr><tr><td>Effect of exchange rate changes on cash and cash equivalents</td><td>(84)</td><td>138</td></tr><tr><td>Cash and cash equivalents at the beginning of the period</td><td>12,173</td><td>17,472</td></tr><tr><td>Cash and cash equivalents at the end of the period</td><td>14,786</td><td>12,173</td></tr><tr><td colspan="3">Supplementary information:</td></tr><tr><td>Restricted cash balance</td><td>348</td><td>362</td></tr></tbody></table>
The accompanying notes form an integral part of the interim condensed consolidated financial statements
As per our report of even date attached
for Deloitte Haskins & Sells LLP
Chartered Accountants
Firm’s Registration No :
117366W/ W-100018
for and on behalf of the Board of Directors of Infosys Limited
Sanjiv V. Pilgaonkar Partner Membership No. 039826
D. Sundaram Lead Independent Director
Salil Parekh Chief Executive Officer and Managing Director
Bobby Parikh Director
Bengaluru April 18, 2024
Jayesh Sanghrajka Chief Financial Officer
A.G.S. Manikantha Company Secretary
6
INFOSYS LIMITED AND SUBSIDIARIES
Overview and notes to the Interim Condensed Consolidated Financial Statements
1. Overview
1.1 Company overview
Infosys Limited ('the Company' or Infosys) provides consulting, technology, outsourcing and next-generation digital services, to enable clients to execute strategies for their digital transformation. Infosys strategic objective is to build a sustainable organization that remains relevant to the agenda of clients, while creating growth opportunities for employees and generating profitable returns for investors. Infosys strategy is to be a navigator for our clients as they ideate, plan and execute on their journey to a digital future.
Infosys together with its subsidiaries and controlled trusts is hereinafter referred to as "the Group".
The Company is a public limited company incorporated and domiciled in India and has its registered office at Electronics city, Hosur Road, Bengaluru 560100, Karnataka, India. The Company has its primary listings on the BSE Ltd. and National Stock Exchange of India Limited. The Company’s American Depositary Shares (ADS) representing equity shares are listed on the New York Stock Exchange (NYSE).
The Group's interim condensed consolidated financial statements are approved for issue by the Company's Board of Directors on April 18, 2024.
1.2 Basis of preparation of financial statements
These interim condensed consolidated financial statements are prepared in compliance with Indian Accounting Standard (Ind AS) 34 Interim Financial Reporting , under the historical cost convention on accrual basis except for certain financial instruments which are measured at fair values, the provisions of the Companies Act, 2013 ('the Act') and guidelines issued by the Securities and Exchange Board of India (SEBI). Accordingly, these interim condensed consolidated financial statements do not include all the information required for a complete set of financial statements. These interim condensed consolidated financial statements should be read in conjunction with the consolidated financial statements and related notes included in the Company’s Annual Report for the year ended March 31, 2023. The Ind AS are prescribed under Section 133 of the Act read with Rule 3 of the Companies (Indian Accounting Standards) Rules, 2015 and relevant amendment rules issued thereafter.
Accounting policies have been consistently applied except where a newly issued accounting standard is initially adopted or a revision to an existing accounting standard requires a change in the accounting policy hitherto in use. The material accounting policy information used in preparation of the audited condensed consolidated interim financial statements have been discussed in the respective notes.
As the quarter and year-to-date figures are taken from the source and rounded to the nearest digits, the quarter figures in this statement added up to the figures reported for the previous quarters might not always add up to the year-to-date figures reported in this statement.
1.3 Basis of consolidation
Infosys consolidates entities which it owns or controls. The interim condensed consolidated financial statements comprise the financial statements of the Company, its controlled trusts and its subsidiaries. Control exists when the parent has power over the entity, is exposed, or has rights to variable returns from its involvement with the entity and has the ability to affect those returns by using its power over the entity. Power is demonstrated through existing rights that give the ability to direct relevant activities, those which significantly affect the entity's returns. Subsidiaries are consolidated from the date control commences until the date control ceases.
The financial statements of the Group companies are consolidated on a line-by-line basis and intra-group balances and transactions including unrealized gain / loss from such transactions are eliminated upon consolidation. These financial statements are prepared by applying uniform accounting policies in use at the Group. Non-controlling interests which represent part of the net profit or loss and net assets of subsidiaries that are not, directly or indirectly, owned or controlled by the Company, are excluded.
1.4 Use of estimates and judgments
The preparation of the interim condensed consolidated financial statements in conformity with Ind AS requires the Management to make estimates, judgments and assumptions. These estimates, judgments and assumptions affect the application of accounting policies and the reported amounts of assets and liabilities, the disclosures of contingent assets and liabilities at the date of the interim condensed consolidated financial statements and reported amounts of revenues and expenses during the period. The application of accounting policies that require critical accounting estimates involving complex and subjective judgments and the use of assumptions in these financial statements have been disclosed in Note no. 1.5 . Accounting estimates could change from period to period. Actual results could differ from those estimates. Appropriate changes in estimates are made as Management becomes aware of changes in circumstances surrounding the estimates. Changes in estimates and judgements are reflected in the interim condensed consolidated financial statements in the period in which changes are made and, if material, their effects are disclosed in the notes to the interim condensed consolidated financial statements.
7
1.5 Critical accounting estimates and judgments
a. Revenue recognition
The Group’s contracts with customers include promises to transfer multiple products and services to a customer. Revenues from customer contracts are considered for recognition and measurement when the contract has been approved, in writing, by the parties to the contract, the parties to contract are committed to perform their respective obligations under the contract, and the contract is legally enforceable. The Group assesses the services promised in a contract and identifies distinct performance obligations in the contract. Identification of distinct performance obligations to determine the deliverables and the ability of the customer to benefit independently from such deliverables, and allocation of transaction price to these distinct performance obligations involves significant judgment.
Fixed price maintenance revenue is recognized ratably on a straight-line basis when services are performed through an indefinite number of repetitive acts over a specified period. Revenue from fixed price maintenance contract is recognized ratably using a percentage of completion method when the pattern of benefits from the services rendered to the customer and the Group’s costs to fulfil the contract is not even through the period of the contract because the services are generally discrete in nature and not repetitive. The use of method to recognize the maintenance revenues requires judgment and is based on the promises in the contract and nature of the deliverables.
The Group uses the percentage-of-completion method in accounting for other fixed-price contracts. Use of the percentage-of-completion method requires the Group to determine the actual efforts or costs expended to date as a proportion of the estimated total efforts or costs to be incurred. Efforts or costs expended have been used to measure progress towards completion as there is a direct relationship between input and productivity. The estimation of total efforts or costs involves significant judgment and is assessed throughout the period of the contract to reflect any changes based on the latest available information.
Contracts with customers includes subcontractor services or third-party vendor equipment or software in certain integrated services arrangements. In these types of arrangements, revenue from sales of third-party vendor products or services is recorded net of costs when the Group is acting as an agent between the customer and the vendor, and gross when the Group is the principal for the transaction. In doing so, the Group first evaluates whether it obtains control of the specified goods or services before they are transferred to the customer. The Group considers whether it is primarily responsible for fulfilling the promise to provide the specified goods or services, inventory risk, pricing discretion and other factors to determine whether it controls the specified goods or services and therefore, is acting as a principal or an agent.
Provisions for estimated losses, if any, on incomplete contracts are recorded in the period in which such losses become probable based on the estimated efforts or costs to complete the contract.
b. Income taxes
The Group's two major tax jurisdictions are India and the United States, though the Company also files tax returns in other overseas jurisdictions.
Significant judgments are involved in determining the provision for income taxes, including amount expected to be paid / recovered for uncertain tax positions.
In assessing the realizability of deferred income tax assets, the Management considers whether some portion or all of the deferred income tax assets will not be realized. The ultimate realization of deferred income tax assets is dependent upon the generation of future taxable income during the periods in which the temporary differences become deductible. Management considers the scheduled reversals of deferred income tax liabilities, projected future taxable income and tax planning strategies in making this assessment. Based on the level of historical taxable income and projections for future taxable income over the periods in which the deferred income tax assets are deductible, the Management believes that the Group will realize the benefits of those deductible differences. The amount of the deferred income tax assets considered realizable, however, could be reduced in the near term if estimates of future taxable income during the carry forward period are reduced (Refer to Notes 2.15).
c. Business combinations and intangible assets
Business combinations are accounted for using Ind AS 103, Business Combinations. Ind AS 103 requires the identifiable intangible assets and contingent consideration to be fair valued in order to ascertain the fair value of identifiable assets, liabilities and contingent liabilities of the acquiree. These valuations are conducted by external valuation experts. Estimates are required to be made in determining the value of contingent consideration, value of option arrangements and intangible assets. These measurements are based on information available at the acquisition date and are based on expectations and assumptions that have been deemed reasonable by the Management (Refer to Note 2.1).
d. Property, plant and equipment
Property, plant and equipment represent a significant proportion of the asset base of the Group. The charge in respect of periodic depreciation is derived after determining an estimate of an asset’s expected useful life and the expected residual value at the end of its life. The useful lives and residual values of Group's assets are determined by the Management at the time the asset is acquired and reviewed periodically, including at each financial year end. The lives are based on historical experience with similar assets as well as anticipation of future events, which may impact their life, such as changes in technology (Refer to Note 2.2).
8
e. Impairment of Goodwill
Goodwill is tested for impairment on an annual basis and whenever there is an indication that the recoverable amount of a cash generating unit (CGUs) is less than its carrying amount. For the impairment test, goodwill is allocated to the CGU or groups of CGUs which benefit from the synergies of the acquisition and which represent the lowest level at which goodwill is monitored for internal management purposes.
The recoverable amount of CGUs is determined based on higher of value-in-use and fair value less cost to sell. Key assumptions in the cash flow projections are prepared based on current economic conditions and comprises estimated long term growth rates, weighted average cost of capital and estimated operating margins (Refer to Note 2.3.1).
9
2. Notes to the Interim Condensed Consolidated Financial Statements
2.1 BUSINESS COMBINATIONS
Accounting policy
Business combinations have been accounted for using the acquisition method under the provisions of Ind AS 103, Business Combinations.
The purchase price in an acquisition is measured at the fair value of the assets transferred, equity instruments issued and liabilities incurred or assumed at the date of acquisition, which is the date on which control is transferred to the Group. The purchase price also includes the fair value of any contingent consideration. Identifiable assets acquired and liabilities and contingent liabilities assumed in a business combination are measured initially at their fair value on the date of acquisition. Contingent consideration is remeasured at fair value at each reporting date and changes in the fair value of the contingent consideration are recognized in the Consolidated Statement of Profit and Loss.
The interest of non-controlling shareholders is initially measured either at fair value or at the non-controlling interests’ proportionate share of the acquiree’s identifiable net assets. The choice of measurement basis is made on an acquisition-by-acquisition basis. Subsequent to acquisition, the carrying amount of non- controlling interests is the amount of those interests at initial recognition plus the non-controlling interests’ share of subsequent changes in equity of subsidiaries.
Business combinations between entities under common control is accounted for at carrying value of the assets acquired and liabilities assumed in the Group's consolidated financial statements.
The payments related to options issued by the Group over the non-controlling interests in its subsidiaries are accounted as financial liabilities and initially recognized at the estimated present value of gross obligations. Such options are subsequently measured at fair value in order to reflect the amount payable under the option at the date at which it becomes exercisable. In the event that the option expires unexercised, the liability is derecognized.
Acquisitions during the year ended March 31, 2023
During the year ended March 31, 2023 the Group, completed two business combinations to complement its digital offerings by acquiring 100% voting interests in:
1) oddity GmbH, oddity group services GmbH, oddity space GmbH, oddity jungle GmbH, oddity code GmbH and oddity waves GmbH (collectively known as oddity), a Germany-based digital marketing, experience, and commerce agencies on April 20, 2022.
2) BASE life science A/S, a consulting and technology firm in the life Science industry in Europe on September 1, 2022.
These acquisitions are expected to strengthen the Group’s creative, branding and experience design capabilities and augment the Group’s life sciences expertise, scales its digital transformation capabilities with cloud based industry solutions and expand its presence across Europe.
The purchase price is allocated to assets acquired and liabilities assumed based upon determination of fair values at the dates of acquisition as follows:
<table><thead><tr><th rowspan="2">Component</th><th>Acquiree's 
 amount</th><th>Fair value</th><th>(In % crore) Purchase price allocated</th></tr><tr><th>carrying</th><th>adjustments</th><th></th></tr></thead><tbody><tr><td>Net Assets?</td><td>103</td><td>-</td><td>103</td></tr><tr><td colspan="4">Intangible assets :</td></tr><tr><td>Customer contracts and relationships</td><td>-</td><td>274</td><td>274</td></tr><tr><td>Vendor relationships</td><td>-</td><td>30</td><td>30</td></tr><tr><td>Brand</td><td>-</td><td>24</td><td>24</td></tr><tr><td>Deferred tax liabilities on intangible assets</td><td>-</td><td>(80)</td><td>(80)</td></tr><tr><td>Total</td><td>103</td><td>248</td><td>351</td></tr><tr><td>Goodwill</td><td></td><td></td><td>630</td></tr><tr><td>Total purchase price</td><td></td><td></td><td>981</td></tr></tbody></table>
Includes cash and cash equivalents acquired of ¥ 26 crore.
The excess of the purchase consideration paid over the fair value of assets acquired has been attributed to goodwill. The primary items that generated this goodwill are the value of the acquired assembled workforce and estimated synergies, neither of which qualify as an intangible asset.
Goodwill is not tax-deductible. Goodwill pertaining to these business combinations is allocated to operating segments as more fully described in Note 2.3.1.
The purchase consideration of ₹981 crore includes cash of ₹936 crore and contingent consideration with an estimated fair value of ₹45 crore as on the date of acquisition.
At the acquisition date, the key inputs used in determination of the fair value of contingent consideration are the probabilities assigned towards achievement of financial targets and discount rate of 12.5%. As of March 31, 2024 the contingent consideration was fully paid.
Additionally, these acquisitions have shareholder and employee retention bonus payable to the employees of the acquiree over three years, subject to their continuous employment with the Group along with achievement of financial targets for the respective years. Performance and Retention Bonus is recognized in employee benefit expenses in the Interim Condensed Consolidated Statement of Statement of Profit or Loss over the period of service.
Fair value of trade receivables acquired, is ₹111 crore as of acquisition date and as of March 31, 2024 the amounts are fully collected.
Transaction costs that the Group incurs in connection with a business combination such as finder’s fees, legal fees, due diligence fees, and other professional and consulting fees are expensed as incurred. The transaction costs of ₹7 crore related to the acquisition have been included under administrative expenses in the Interim Condensed Consolidated Statement of Statement of Profit or Loss for the year ended March 31, 2023.
10
Proposed acquisitions
On January 11, 2024, Infosys Limited entered into a definitive agreement to acquire 100% of the equity share capital in InSemi Technology Services Private Limited, a semiconductor design services company headquartered in India, for a consideration including earn-outs, and management incentives and retention bonuses totaling up to ₹280 crore (approximately $34 million), subject to customary closing adjustments.
On April 18, 2024, Infosys Germany GmBH wholly owned step down subsidiary of Infosys Limited entered into a definitive agreement to acquire 100% of the equity share capital in in-tech Holding GmbH, leading provider of Engineering R&D services headquartered in Germany, for a consideration including earn-outs amounting up to EUR 450 million (approximately ₹4,045 crore), subject to customary closing adjustments.
11
2.2 PROPERTY, PLANT AND EQUIPMENT
Accounting policy
Property, plant and equipment are stated at cost, less accumulated depreciation and impairment, if any. Costs directly attributable to acquisition are capitalized until the property, plant and equipment are ready for use, as intended by the Management. The charge in respect of periodic depreciation is derived at after determining an estimate of an asset’s expected useful life and the expected residual value at the end of its life. The Group depreciates property, plant and equipment over their estimated useful lives using the straight-line method. The estimated useful lives of assets are as follows:
<table><tbody><tr><td>Buildings @</td><td>22-25 years</td></tr><tr><td>Plant and machinery (2)</td><td>5 years</td></tr><tr><td>Office equipment</td><td>5 years</td></tr><tr><td>Computer equipment</td><td>3-5 years</td></tr><tr><td>Furniture and fixtures</td><td>5 years</td></tr><tr><td>Vehicles(1)</td><td>5 years</td></tr><tr><td>’Iieasehold improvements</td><td>Lower of usefu</td></tr></tbody></table>
life of the asset or lease term
(1) Based on technical evaluation, the Management believes that the useful lives as given above best represent the period over which the Management expects to use these assets. Hence, the useful lives for these assets is different from the useful lives as prescribed under Part C of Schedule II of the Companies Act 2013
(2) Includes Solar plant with a useful life of 25 years
Depreciation methods, useful lives and residual values are reviewed periodically, including at each financial year end. The useful lives are based on historical experience with similar assets as well as anticipation of future events, which may impact their life, such as changes in technology.
Advances paid towards the acquisition of property, plant and equipment outstanding at each Balance Sheet date is classified as capital advances under other non-current assets and the cost of assets not ready to use before such date are disclosed under ‘Capital work-in-progress’. Subsequent expenditures relating to property, plant and equipment is capitalized only when it is probable that future economic benefits associated with these will flow to the Group and the cost of the item can be measured reliably. The cost and related accumulated depreciation are eliminated from the financial statements upon sale or retirement of the asset.
Impairment
Property, plant and equipment are evaluated for recoverability whenever events or changes in circumstances indicate that their carrying amounts may not be recoverable. For the purpose of impairment testing, the recoverable amount (i.e. the higher of the fair value less cost to sell and the value-in-use) is determined on an individual asset basis unless the asset does not generate cash flows that are largely independent of those from other assets. In such cases, the recoverable amount is determined for the Cash Generating Unit (CGU) to which the asset belongs.
If such assets are considered to be impaired, the impairment to be recognized in the Consolidated Statement of Profit and Loss is measured by the amount by which the carrying value of the assets exceeds the estimated recoverable amount of the asset. An impairment loss is reversed in the Consolidated Statement of Profit and Loss if there has been a change in the estimates used to determine the recoverable amount. The carrying amount of the asset is increased to its revised recoverable amount, provided that this amount does not exceed the carrying amount that would have been determined (net of any accumulated depreciation) had no impairment loss been recognized for the asset in prior years.
The changes in the carrying value of property, plant and equipment for the three months ended March 31, 2024 are as follows:
<table><thead><tr><th rowspan="2">Particulars</th><th rowspan="2">Land - Freehold</th><th rowspan="2">Buildings 
 (1)</th><th rowspan="2">Plant and machinery</th><th rowspan="2">Office Equipment</th><th></th><th></th><th rowspan="2">Leasehold 
 Improvements</th><th></th><th rowspan="2">(In % crore) Total</th></tr><tr><th>Computer 
 equipment</th><th>Furniture and fixtures</th><th>Vehicles</th></tr></thead><tbody><tr><td>Gross carrying value as at J anuary 1, 2024</td><td>1,432</td><td>11,498</td><td>3,305</td><td>1,510</td><td>8,497</td><td>2,308</td><td>1,456</td><td>45</td><td>30,051</td></tr><tr><td>Additions</td><td></td><td>287</td><td>140</td><td>33</td><td>345</td><td>54</td><td>35</td><td></td><td>894</td></tr><tr><td>Deletions**</td><td></td><td>—</td><td>(16)</td><td>(14)</td><td>(224)</td><td>(34)</td><td>(37)</td><td></td><td>(325)</td></tr><tr><td>Translation difference</td><td></td><td>(15)</td><td>(1)</td><td>1)</td><td>(7)</td><td>(2)</td><td>(7)</td><td></td><td>(33)</td></tr><tr><td>Gross carrying value as at March 31, 2024</td><td>1,432</td><td>11,770</td><td>3,428</td><td>1,528</td><td>8,611</td><td>2,326</td><td>1,447</td><td>45</td><td>30,587</td></tr><tr><td>Accumulated depreciation as at J anuary 1, 2024</td><td></td><td>(4,8149)</td><td>(2,584)</td><td>(1,253)</td><td>(6,267)</td><td>(1,807)</td><td>(1,131)</td><td>42)</td><td>(17,898)</td></tr><tr><td>Depreciation</td><td></td><td>(111)</td><td>(63)</td><td>(32)</td><td>(336)</td><td>(58)</td><td>(46)</td><td></td><td>(646)</td></tr><tr><td>Accumulated depreciation on deletions**</td><td></td><td>—</td><td>16</td><td>14</td><td>219</td><td>26</td><td>34</td><td></td><td>309</td></tr><tr><td>Translation difference</td><td>i</td><td>4</td><td>1</td><td>2</td><td>4</td><td>2</td><td>5</td><td></td><td>18</td></tr><tr><td>Accumulated depreciation as at March 31, 2024</td><td>—</td><td>(4,921)</td><td>(2,630)</td><td>(1,269)</td><td>(6,380)</td><td>(1,837)</td><td>(1,138)</td><td>(4;)</td><td>(18,217)</td></tr><tr><td>Carrying value as at January 1, 2024</td><td>1,432</td><td>6,684</td><td>721</td><td>257</td><td>2,230</td><td>501</td><td>325</td><td></td><td>12,153</td></tr><tr><td>Carrying value as at March 31, 2024</td><td>1,432</td><td>6,849</td><td>798</td><td>259</td><td>2,231</td><td>489</td><td>309</td><td></td><td>12,370</td></tr></tbody></table>
The changes in the carrying value of property, plant and equipment for the three months ended March 31, 2023 were as follows:
<table><thead><tr><th rowspan="2">Particulars</th><th rowspan="2">Land - Freehold</th><th rowspan="2">Buildings 
 (1)</th><th rowspan="2">Plant and machinery</th><th rowspan="2">Office Equipment</th><th></th><th></th><th rowspan="2">Leasehold Improvements</th><th></th><th rowspan="2">(In % crore) Total</th></tr><tr><th>Computer 
 equipment</th><th>Furniture and fixtures</th><th>Vehicles</th></tr></thead><tbody><tr><td>Gross carrying value as at J anuary 1, 2023</td><td>1,431</td><td>11,530</td><td>3,368</td><td>1,466</td><td>8,895</td><td>2,450</td><td>1,353</td><td></td><td>30,537</td></tr><tr><td>Additions</td><td>2</td><td>29</td><td>109</td><td>55</td><td>494</td><td>162</td><td>103</td><td></td><td>955</td></tr><tr><td>Deletions*</td><td>()</td><td>—</td><td>(175)</td><td>(40)</td><td>(877)</td><td>(311)</td><td>(13)</td><td></td><td>(1,418)</td></tr><tr><td>Translation difference</td><td></td><td>3</td><td>—</td><td>1</td><td>7</td><td>2</td><td>2</td><td></td><td>15</td></tr><tr><td>Gross carrying value as at March 31, 2023</td><td>1,4;</td><td>11,562</td><td>3,302</td><td>1,482</td><td>8,519</td><td>2,303</td><td>1,445</td><td>45</td><td>30,089</td></tr><tr><td>Accumulated depreciation as at January 1, 2023</td><td>—</td><td>(4,425)</td><td>(2,547)</td><td>(1,206)</td><td>(6,339)</td><td>(1,922)</td><td>(992)</td><td>39)</td><td>(17,470)</td></tr><tr><td>Depreciation</td><td></td><td>(109)</td><td>(65)</td><td>(31)</td><td>(354)</td><td>(62)</td><td>(48)</td><td>1)</td><td>(670)</td></tr><tr><td>Accumulated depreciation on deletions*</td><td></td><td>—</td><td>175</td><td>40</td><td>871</td><td>310</td><td>9</td><td></td><td>1,405</td></tr><tr><td>Translation difference</td><td></td><td>(1)</td><td>—</td><td>1)</td><td>(4)</td><td>(1)</td><td>1)</td><td></td><td>(8</td></tr><tr><td>Accumulated depreciation as at March 31, 2023</td><td></td><td>(4,535)</td><td>(2,437)</td><td>(1,198)</td><td>(5,826)</td><td>(1,675)</td><td>(1,032)</td><td>(4;)</td><td>(16,743)</td></tr><tr><td>Carrying value as at January 1, 2023</td><td>1,431</td><td>7,105</td><td>821</td><td>260</td><td>2,556</td><td>528</td><td>361</td><td></td><td>13,067</td></tr><tr><td>Carrying value as at March 31, 2023</td><td>1,431</td><td>7,027</td><td>865</td><td>284</td><td>2,693</td><td>628</td><td>413</td><td></td><td>13,346</td></tr></tbody></table>
(In % crore)
(In % crore)
12
The changes in the carrying value of property, plant and equipment for the year ended March 31, 2024 are as follows:
<table><thead><tr><th rowspan="2">Particulars</th><th rowspan="2">Land - Freehold</th><th rowspan="2">Buildings 
 (1)</th><th rowspan="2">Plant and machinery</th><th rowspan="2">Office Equipment</th><th></th><th></th><th rowspan="2">Leasehold 
 Improvements</th><th colspan="2">(In % crore)</th></tr><tr><th>Computer 
 equipment</th><th>Furniture and fixtures</th><th>Vehicles</th><th>Total</th></tr></thead><tbody><tr><td>Gross carrying value as at April 1, 2023</td><td>1,431</td><td>11,562</td><td>3,302</td><td>1,482</td><td>8,519</td><td>2,303</td><td>1,445</td><td>45</td><td>30,089</td></tr><tr><td>Additions</td><td>1</td><td>300</td><td>193</td><td>106</td><td>931</td><td>121</td><td>108</td><td></td><td>1,761</td></tr><tr><td>Deletions**</td><td>i</td><td>(55)</td><td>(64)</td><td>(60)</td><td>(846)</td><td>(99)</td><td>(102)</td><td>1)</td><td>(1,227)</td></tr><tr><td>Translation difference</td><td>—</td><td>(37)</td><td>[©)</td><td></td><td>7</td><td>1</td><td>4)</td><td></td><td>(36)</td></tr><tr><td>Gross carrying value as at March 31, 2024</td><td>1,432</td><td>11,770</td><td>3,428</td><td>1528</td><td>8,611</td><td>2,326</td><td>1,447</td><td>45</td><td>30,587</td></tr><tr><td>Accumulated depreciation as at April 1, 2023</td><td>—</td><td>(4,535)</td><td>(2,437)</td><td>(1,198)</td><td>(5,826)</td><td>(1,675)</td><td>(1,032)</td><td>40)</td><td>(16,743)</td></tr><tr><td>Depreciation</td><td>i</td><td>(450)</td><td>(259)</td><td>(130)</td><td>(1,387)</td><td>(250)</td><td>(206)</td><td>()</td><td>(2,685)</td></tr><tr><td>Accumulated depreciation on deletions**</td><td>—</td><td>55</td><td>64</td><td>59</td><td>836</td><td>89</td><td>97</td><td></td><td>1,201</td></tr><tr><td>Translation difference</td><td>i</td><td>9</td><td>2</td><td></td><td>(©)</td><td>(1)</td><td>3</td><td></td><td>10</td></tr><tr><td>Accumulated depreciation as at March 31, 2024</td><td>—</td><td>(4,921)</td><td>(2,630)</td><td>(1,26;)</td><td>(6,380)</td><td>(1,837)</td><td>(1,138)</td><td>(4;)</td><td>(18,217)</td></tr><tr><td>Carrying value as at April 1, 2023</td><td>1,431</td><td>7,027</td><td>865</td><td>284</td><td>2,693</td><td>628</td><td>413</td><td></td><td>13,346</td></tr><tr><td>Carrying value as at March 31, 2024</td><td>1,432</td><td>6,849</td><td>798</td><td>259</td><td>2,231</td><td>489</td><td>309</td><td></td><td>12,370</td></tr></tbody></table>
** During the three months and year ended March 31, 2024, certain assets which were not in use having gross book value of ₹181 crore (net book value: Nil) and ₹775 crore (net book value: Nil), respectively were retired.
The changes in the carrying value of property, plant and equipment for the year ended March 31, 2023 are as follows:
<table><thead><tr><th rowspan="2">Particulars</th><th rowspan="2">Land - Freehold</th><th rowspan="2">Buildings 
 (1)</th><th rowspan="2">Plant and machinery</th><th rowspan="2">Office Equipment</th><th colspan="2"></th><th rowspan="2">Leasehold 
 Improvements</th><th colspan="2">(In % crore)</th></tr><tr><th>Computer 
 equipment</th><th>Furniture and fixtures</th><th>Vehicles</th><th>Total</th></tr></thead><tbody><tr><td>Gross carrying value as at April 1, 2022</td><td>1,431</td><td>11,224</td><td>3,210</td><td>1,427</td><td>8,527</td><td>2,278</td><td>1,234</td><td></td><td>29,375</td></tr><tr><td>Additions - Business Combination (Refer to Note 2.1)</td><td>—                 —                —</td><td></td><td></td><td>5</td><td>6</td><td>1</td><td>2</td><td></td><td>14</td></tr><tr><td>Additions</td><td></td><td>337</td><td>273</td><td>122</td><td>1,510</td><td>364</td><td>220</td><td></td><td>2,830</td></tr><tr><td>Deletions*</td><td>2</td><td>—</td><td>(182)</td><td>(76)</td><td>(1,563)</td><td>(348)</td><td>(25)</td><td>1)</td><td>(2,197)</td></tr><tr><td>Translation difference</td><td></td><td>1</td><td>1</td><td>4</td><td>39</td><td>8</td><td>14</td><td></td><td>67</td></tr><tr><td>Gross carrying value as at March 31, 2023</td><td>1,4;</td><td>11,562</td><td>3,302</td><td>1,482</td><td>8,519</td><td>2,303</td><td>1,445</td><td>45</td><td>30,089</td></tr><tr><td>Accumulated depreciation as at April 1, 2022</td><td></td><td>(4,100)</td><td>(2,344)</td><td>(1,150)</td><td>(6,034</td><td>(1,779)</td><td>(856)</td><td>37</td><td>(16,300)</td></tr><tr><td>Depreciation</td><td></td><td>(434)</td><td>(273)</td><td>(121)</td><td>(1,322)</td><td>(236)</td><td>(187)</td><td>4)</td><td>(2,577)</td></tr><tr><td>Accumulated depreciation on deletions*</td><td></td><td>—</td><td>181</td><td>76</td><td>1,556</td><td>347</td><td>21</td><td></td><td>2,182</td></tr><tr><td>Translation difference</td><td></td><td>(1)</td><td>(1)</td><td>(©)</td><td>(26)</td><td>(7)</td><td>(10)</td><td></td><td>(48)</td></tr><tr><td>Accumulated depreciation as at March 31, 2023</td><td></td><td>(4,535)</td><td>(2,437)</td><td>(1,198)</td><td>(5,826)</td><td>(1,675)</td><td>(1,032)</td><td>(40)</td><td>(16,743)</td></tr><tr><td>Carrying value as at April 1, 2022</td><td>1,4;</td><td>7,124</td><td>866</td><td>277</td><td>2,493</td><td>499</td><td>378</td><td></td><td>13,075</td></tr><tr><td>Carrying value as at March 31, 2023</td><td>1,431</td><td>7,027</td><td>865</td><td>284</td><td>2,693</td><td>628</td><td>413</td><td></td><td>13,346</td></tr></tbody></table>
* During the three months and year ended March 31, 2023, certain assets which were old and not in use having gross book value of ₹1,414 crore (net book value: Nil) and ₹1,918 crore (net book value: Nil), respectively were retired.
(1) Buildings include ₹250/- being the value of five shares of ₹50/- each in Mittal Towers Premises Co-operative Society Limited.
The aggregate depreciation has been included under depreciation and amortization expense in the interim condensed Consolidated Statement of Profit and Loss.
Repairs and maintenance costs are recognized in the Consolidated Statement of Profit and Loss when incurred.
Consequent to the Companies (Corporate Social Responsibility Policy) Amendment Rules, 2021 (“the Rules”), the Company was required to transfer its CSR capital assets installed prior to January 2021. Towards this the Company had incorporated a subsidiary ‘Infosys Green Forum’ (IGF) under Section 8 of the Companies Act, 2013. During the year ended March 31, 2022 the Company had completed the transfer of assets upon obtaining the required approvals from regulatory authorities, as applicable. During March 31, 2024, the application filed by IGF for registration u/s.12AB of the Income Tax Act was rejected and registration cancelled. IGF is in the process of challenging the rejection order.
(In % crore)
13
2.3 GOODWILL AND OTHER INTANGIBLE ASSETS
2.3.1 Goodwill
Accounting policy
Goodwill represents the purchase consideration in excess of the Group's interest in the net fair value of identifiable assets, liabilities and contingent liabilities of the acquired entity. When the net fair value of the identifiable assets, liabilities and contingent liabilities acquired exceeds purchase consideration, the fair value of net assets acquired is reassessed and the bargain purchase gain is recognized in capital reserve. Goodwill is measured at cost less accumulated impairment losses.
Impairment
Goodwill is tested for impairment on an annual basis and whenever there is an indication that the recoverable amount of a cash generating unit (CGU) is less than its carrying amount. For the impairment test, goodwill is allocated to the CGU or groups of CGUs which benefit from the synergies of the acquisition and which represents the lowest level at which goodwill is monitored for internal management purposes. A CGU is the smallest identifiable group of assets that generates cash inflows that are largely independent of the cash inflows from other assets or group of assets. Impairment occurs when the carrying amount of a CGU including the goodwill, exceeds the estimated recoverable amount of the CGU. The recoverable amount of a CGU is the higher of its fair value less cost to sell and its value-in-use. Value-in-use is the present value of future cash flows expected to be derived from the CGU. Key assumptions in the cash flow projections are prepared based on current economic conditions and includes estimated long term growth rates, weighted average cost of capital and estimated operating margins.
Following is a summary of changes in the carrying amount of goodwill:
<table><thead><tr><th rowspan="2">Particulars</th><th>As at</th><th></th></tr><tr><th>March 31, 2024</th><th>March 31, 2023</th></tr></thead><tbody><tr><td>Carrying value at the beginning</td><td>7,248</td><td>6,195</td></tr><tr><td>Goodwill on acquisitions (Refer to note 2.1)</td><td>—</td><td>630</td></tr><tr><td>Translation differences</td><td>55</td><td>423</td></tr><tr><td>Carrying value at the end</td><td>7,303</td><td>7,248</td></tr></tbody></table>
For the purpose of impairment testing, goodwill acquired in a business combination is allocated to the CGU or groups of CGUs, which benefit from the synergies of the acquisition.
2.3.2 Intangible Assets
Accounting policy
Intangible assets are stated at cost less accumulated amortization and impairment. Intangible assets are amortized over their respective individual estimated useful lives on a straight-line basis, from the date that they are available for use. The estimated useful life of an identifiable intangible asset is based on a number of factors including the effects of obsolescence, demand, competition, and other economic factors (such as the stability of the industry, and known technological advances) and the level of maintenance expenditures required to obtain the expected future cash flows from the asset. Amortization methods and useful lives are reviewed periodically including at each financial year end.
Research costs are expensed as incurred. Software product development costs are expensed as incurred unless technical and commercial feasibility of the project is demonstrated, future economic benefits are probable, the Group has an intention and ability to complete and use or sell the software and the costs can be measured reliably. The costs which can be capitalized include the cost of material, direct labor, overhead costs that are directly attributable to prepare the asset for its intended use.
Impairment
Intangible assets are evaluated for recoverability whenever events or changes in circumstances indicate that their carrying amounts may not be recoverable. For the purpose of impairment testing, the recoverable amount (i.e. the higher of the fair value less cost to sell and the value-in-use) is determined on an individual asset basis unless the asset does not generate cash flows that are largely independent of those from other assets. In such cases, the recoverable amount is determined for the CGU to which the asset belongs.
If such assets are considered to be impaired, the impairment to be recognized in the Consolidated Statement of Profit and Loss is measured by the amount by which the carrying value of the assets exceeds the estimated recoverable amount of the asset. An impairment loss is reversed in the Consolidated Statement of Profit and Loss if there has been a change in the estimates used to determine the recoverable amount. The carrying amount of the asset is increased to its revised recoverable amount, provided that this amount does not exceed the carrying amount that would have been determined (net of any accumulated amortization) had no impairment loss been recognized for the asset in prior years.
14
2.4 INVESTMENTS
<table><thead><tr><th rowspan="2">Particulars</th><th>Asat</th><th>(4 N Crore,</th></tr><tr><th>March 31, 2024</th><th>March 31, 2023</th></tr></thead><tbody><tr><td colspan="3">Non-current Investments</td></tr><tr><td colspan="3">Unquoted</td></tr><tr><td colspan="3">Investments carried at fair value through other comprehensive income</td></tr><tr><td>Preference securities</td><td>91</td><td>193</td></tr><tr><td>Equity securities</td><td>2                                      3</td><td></td></tr><tr><td></td><td>93</td><td>196</td></tr><tr><td>Investments carried at fair value through profit or loss Target maturity fund units</td><td>431</td><td>402</td></tr><tr><td>Others</td><td>198</td><td>169</td></tr><tr><td></td><td>629</td><td>571</td></tr><tr><td colspan="3">Quoted</td></tr><tr><td>Investments carried at amortized cost</td><td></td><td></td></tr><tr><td>Government bonds Tax free bonds</td><td>28</td><td>28</td></tr><tr><td></td><td>1,731 1,759</td><td>1,742 1,770</td></tr><tr><td colspan="3">Investments carried at fair value through other comprehensive income</td></tr><tr><td>Non convertible debentures</td><td>2,217</td><td>2,713</td></tr><tr><td>Equity securities</td><td>113</td><td>—</td></tr><tr><td>Government securities</td><td>6,897 9,227</td><td>7,319 10,032</td></tr><tr><td colspan="3" rowspan="2">Total non-current investments 11,708 12,569 Current Investments</td></tr><tr><td colspan="3">Unquoted</td></tr><tr><td colspan="3">Investments carried at fair value through profit or loss</td></tr><tr><td>Liquid mutual fund units</td><td>2,615</td><td>975</td></tr><tr><td></td><td></td><td>975</td></tr><tr><td></td><td>2,615</td><td></td></tr><tr><td colspan="3">Investments carried at fair value through other comprehensive income</td></tr><tr><td>Commercial</td><td>4,830</td><td>742</td></tr><tr><td>Papers</td><td></td><td></td></tr><tr><td>Certificates of deposit</td><td>3,043</td><td>3,574</td></tr><tr><td></td><td>7,873</td><td>4,316</td></tr><tr><td>Quoted</td><td></td><td></td></tr><tr><td colspan="3">Investments carried at amortized cost</td></tr><tr><td>Tax free bonds</td><td>—</td><td>150</td></tr><tr><td colspan="3">150 Investments carried at fair value through other comprehensive income                                  —</td></tr><tr><td>Non convertible debentures</td><td>1,962</td><td>1,155</td></tr><tr><td>Government securities</td><td>465</td><td>313</td></tr><tr><td></td><td>2,427</td><td>1,468</td></tr><tr><td>Total current investments</td><td>12,915</td><td>6,909</td></tr><tr><td colspan="3">Total investments 24,623 19,478</td></tr><tr><td>Aggregate amount of quoted investments</td><td>13,413</td><td>13,420</td></tr><tr><td>Market value of quoted investments (including interest accrued), current</td><td>2,428</td><td>1,637</td></tr><tr><td>Market value of quoted investments (including interest accrued), non current</td><td>11,201</td><td>12,042</td></tr><tr><td>Aggregate amount of unquoted investments</td><td>11,210</td><td>6,058</td></tr><tr><td>Investments carried at amortized cost</td><td>1,759</td><td>1,920</td></tr><tr><td>Investments carried at fair value through other comprehensive income</td><td>19,620</td><td>16,012</td></tr><tr><td>Investments carried at fair value through profit or loss</td><td>3,244</td><td>1,546</td></tr></tbody></table>
(1) Uncalled capital commitments outstanding as at March 31, 2024 and March 31, 2023 was ₹79 crore and ₹92 crore, respectively.
Refer to Note 2.10 for Accounting policies on Financial Instruments.
Method of fair valuation:
<table><thead><tr><th rowspan="3">Class of investment</th><th rowspan="3">Method</th><th colspan="2">(In X crore)</th></tr><tr><th colspan="2">Fair value as at</th></tr><tr><th>March 31, 2024</th><th>March 31, 2023</th></tr></thead><tbody><tr><td>Liquid mutual fund units - carried at fair value through profit or loss</td><td>Quoted price</td><td>2,615</td><td>975</td></tr><tr><td>Target maturity fund units - carried at fair value through profit or loss</td><td>Quoted price</td><td>431</td><td>402</td></tr><tr><td>Tax free bonds and government bonds - carried at amortized cost</td><td>Quoted price and market observable inputs</td><td>1,973</td><td>2,148</td></tr><tr><td>Non-convertible debentures - carried at fair value through other comprehensive income</td><td>Quoted price and market observable inputs</td><td>4,179</td><td>3,868</td></tr><tr><td>Government securities - carried at fair value through other comprehensive income</td><td>Quoted price and market observable inputs</td><td>7,362</td><td>7,632</td></tr><tr><td>Commercial Papers - carried at fair value through other comprehensive income</td><td>Market observable inputs</td><td>4,830</td><td>742</td></tr><tr><td>Certificates of deposit - carried at fair value through other comprehensive income</td><td>Market observable inputs</td><td>3,043</td><td>3,574</td></tr><tr><td>Quoted Equity securities - carried at fair value through other comprehensive income</td><td>Quoted price</td><td>113</td><td>—</td></tr><tr><td>Unquoted equity and preference securities - carried at fair value through other comprehensive income</td><td>Discounted cash flows method, Market multiples method, Option pricing model</td><td>93</td><td>196</td></tr><tr><td>Others - carried at fair value through profit or loss</td><td>Discounted cash flows method, Market multiples method, Option pricing model</td><td>198</td><td>169</td></tr><tr><td>Total</td><td></td><td>24,837</td><td>19,706</td></tr></tbody></table>
Note: Certain quoted investments are classified as Level 2 in the absence of active market for such investments.
(In ₹ crore)
X
15
2.5 LOANS
<table><thead><tr><th>Particulars</th><th>Asat March 31, 2024</th><th>March 31, 2023</th></tr></thead><tbody><tr><td colspan="3">Non Current</td></tr><tr><td colspan="3">Loans considered good - Unsecured</td></tr><tr><td colspan="3">Other loans</td></tr><tr><td>Loans to employees</td><td>34</td><td>39</td></tr><tr><td></td><td>34</td><td>39</td></tr><tr><td colspan="3">Loans credit impaired - Unsecured</td></tr><tr><td colspan="3">Other loans</td></tr><tr><td>Loans to employees</td><td>2</td><td>2</td></tr><tr><td>Less: Allowance for credit impairment</td><td>()</td><td>(2</td></tr><tr><td>Total non-current loans</td><td>34                                  —</td><td>39                               —</td></tr><tr><td colspan="3">Current</td></tr><tr><td colspan="3">Loans considered good - Unsecured</td></tr><tr><td colspan="3">Other loans</td></tr><tr><td>Loans to employees</td><td>248</td><td>289</td></tr><tr><td>Total current loans</td><td>248</td><td>289</td></tr><tr><td>Total loans</td><td>282</td><td>328</td></tr></tbody></table>
2.6 OTHER FINANCIAL ASSETS
<table><thead><tr><th rowspan="2">Particulars</th><th>Asat</th><th></th></tr><tr><th>March 31, 2024</th><th>March 31, 2023</th></tr></thead><tbody><tr><td colspan="3">Non Current</td></tr><tr><td>Security deposits</td><td>259</td><td>287</td></tr><tr><td># Unbilled revenues</td><td>1,677</td><td>1,185</td></tr><tr><td>) Net investment in sublease of right-of-use asset a</td><td>3</td><td>305</td></tr><tr><td>Restricted deposits "</td><td>47</td><td>96</td></tr><tr><td>Others</td><td>1,119</td><td>925</td></tr><tr><td>Total non-current other financial assets</td><td>3,105</td><td>2,798</td></tr><tr><td colspan="3">Current</td></tr><tr><td>Security deposits</td><td>75</td><td>42</td></tr><tr><td>Restricted deposits V"</td><td>2,535</td><td>2,348</td></tr><tr><td># Unbilled revenues</td><td>7,923</td><td>8,317</td></tr><tr><td>Interest accrued but not due</td><td>537</td><td>488</td></tr><tr><td>Foreign currency forward and options contracts</td><td>84</td><td>101</td></tr><tr><td>Net investment in sublease of right of-use-asset</td><td>6</td><td>53</td></tr><tr><td>Others V™"</td><td>925</td><td>255</td></tr><tr><td>Total current other financial assets</td><td>12,085</td><td>11,604</td></tr><tr><td>Total other financial assets</td><td>15,190</td><td>14,402</td></tr><tr><td>O Financial assets carried at amortized cost</td><td>15,106</td><td>14,301</td></tr><tr><td>@ Financial assets carried at fair value through other comprehensive income</td><td>23</td><td>32</td></tr><tr><td>) Financial assets carried at fair value through profit or loss</td><td>61</td><td>69</td></tr></tbody></table>
* Restricted deposits represent deposits with financial institutions to settle employee related obligations as and when they arise during the normal course of business. # Classified as financial asset as right to consideration is unconditional and is due only after a passage of time.
** Primarily includes net investment in lease
2.7 TRADE RECEIVABLES
<table><thead><tr><th rowspan="2">Particulars</th><th>Asat</th><th>(In % crore)</th></tr><tr><th>March 31, 2024</th><th>March 31, 2023</th></tr></thead><tbody><tr><td colspan="3">Current</td></tr><tr><td>Trade Receivable considered good - Unsecured</td><td>30,713</td><td>25,965</td></tr><tr><td>Less: Allowance for expected credit loss</td><td>520</td><td>541</td></tr><tr><td>Trade Receivable considered good - Unsecured</td><td>30,193</td><td>25,424</td></tr><tr><td>Trade Receivable - credit impaired - Unsecured</td><td>196</td><td>142</td></tr><tr><td>Less: Allowance for credit impairment</td><td>196</td><td>142</td></tr><tr><td>Trade Receivable - credit impaired - Unsecured</td><td>.</td><td>i</td></tr><tr><td>Total trade receivables</td><td>30,193</td><td>25,424</td></tr></tbody></table>
(In X crore)
X
16
2.8 CASH AND CASH EQUIVALENTS
<table><thead><tr><th>Particulars</th><th>Asat March 31, 2024</th><th>(In % crore) March 31, 2023</th></tr></thead><tbody><tr><td colspan="3">Balances with banks</td></tr><tr><td>In current and deposit accounts</td><td>14,786</td><td>10,026</td></tr><tr><td>Cash on hand</td><td>—</td><td>—</td></tr><tr><td colspan="3">Others</td></tr><tr><td>Deposits with financial institutions</td><td>—</td><td>2,147</td></tr><tr><td>Total cash and cash equivalents</td><td>14,786</td><td>12,173</td></tr><tr><td>Balances with banks in unpaid dividend accounts</td><td>37</td><td>37</td></tr><tr><td>Deposit with more than 12 months maturity</td><td>57</td><td>833</td></tr></tbody></table>
Cash and cash equivalents as at March 31, 2024 and March 31, 2023 include restricted cash and bank balances of ₹348 crore and ₹362 crore respectively. The restrictions are primarily on account of bank balances held by irrevocable trusts controlled by the company.
The deposits maintained by the Group with banks and financial institutions comprise of time deposits, which can be withdrawn by the Group at any point without prior notice or penalty on the principal.
2.9 OTHER ASSETS
<table><thead><tr><th rowspan="2">Particulars</th><th>Asat</th><th>(In % crore)</th></tr><tr><th>March 31, 2024</th><th>March 31, 2023</th></tr></thead><tbody><tr><td colspan="3">Non-current</td></tr><tr><td>Capital advances</td><td>155</td><td>159</td></tr><tr><td colspan="3">Advances other than capital advances</td></tr><tr><td colspan="3">Others</td></tr><tr><td>Withholding taxes and others</td><td>673</td><td>684</td></tr><tr><td>Unbilled revenues *</td><td>103</td><td>264</td></tr><tr><td>Defined benefit plan assets</td><td>31</td><td>36</td></tr><tr><td>Prepaid expenses</td><td>343</td><td>332</td></tr><tr><td colspan="3">Deferred Contract Cost</td></tr><tr><td>Cost of obtaining a contract *</td><td>129</td><td>191</td></tr><tr><td>Cost of fulfillment</td><td>687</td><td>652</td></tr><tr><td>Other receivables</td><td>—                                   —</td><td></td></tr><tr><td>Total non-current other assets</td><td>2,121</td><td>2,318</td></tr><tr><td colspan="3">Current</td></tr><tr><td colspan="3">Advances other than capital advances</td></tr><tr><td>Payment to vendors for supply of goods</td><td>356</td><td>202</td></tr><tr><td colspan="3">Others</td></tr><tr><td>Unbilled revenues *</td><td>4,845</td><td>6,972</td></tr><tr><td>Withholding taxes and others</td><td>3,540</td><td>3,268</td></tr><tr><td>Prepaid expenses</td><td>3,329</td><td>2,745</td></tr><tr><td colspan="3">Deferred Contract Cost</td></tr><tr><td>Cost of obtaining a contract *</td><td>200</td><td>853</td></tr><tr><td>Cost of fulfillment</td><td>358</td><td>175</td></tr><tr><td>Other receivables</td><td>180</td><td>261</td></tr><tr><td>Total current other assets</td><td>12,808</td><td>14,476</td></tr><tr><td>Total other assets</td><td>14,929</td><td>16,794</td></tr></tbody></table>
# Classified as non financial asset as the contractual right to consideration is dependent on completion of contractual milestones.
* Includes technology assets taken over by the Group from a customer as a part of transformation project which is not considered as distinct goods or services and the control related to the assets is not transferred to the Group in accordance with Ind AS 115 - Revenue from contract with customers. Accordingly, the same has been considered as a reduction to the total contract value and accounted as Deferred contract cost. The Group has entered into financing arrangements with a third party for these assets. As at March 31, 2024 and March 31, 2023, the financial liability pertaining to such arrangements amounts to ₹372 crore and ₹731 crore, respectively. For the year ended March 31, 2023 ₹118 crore was settled directly by the third party to the customer on behalf of the Group and accordingly considered as non-cash transaction (Refer to note 2.13)
Withholding taxes and others primarily consist of input tax credits and Cenvat/VAT recoverable from Government of India.
17
2.10 FINANCIAL INSTRUMENTS
Accounting policy
2.10.1 Initial recognition
The Group recognizes financial assets and financial liabilities when it becomes a party to the contractual provisions of the instrument. All financial assets and liabilities are recognized at fair value on initial recognition, except for trade receivables which are initially measured at transaction price. Transaction costs that are directly attributable to the acquisition or issue of financial assets and financial liabilities, which are not at fair value through profit or loss, are added to the fair value on initial recognition. Regular way purchase and sale of financial assets are accounted for at trade date.
2.10.2 Subsequent measurement
a. Non-derivative financial instruments
(i) Financial assets carried at amortized cost
A financial asset is subsequently measured at amortized cost if it is held within a business model whose objective is to hold the asset in order to collect contractual cash flows and the contractual terms of the financial asset give rise on specified dates to cash flows that are solely payments of principal and interest on the principal amount outstanding.
(ii) Financial assets carried at fair value through other comprehensive income (FVOCI)
A financial asset is subsequently measured at fair value through other comprehensive income if it is held within a business model whose objective is achieved by both collecting contractual cash flows and selling financial assets and the contractual terms of the financial asset give rise on specified dates to cash flows that are solely payments of principal and interest on the principal amount outstanding. The Group has made an irrevocable election for its investments which are classified as equity instruments to present the subsequent changes in fair value in other comprehensive income based on its business model.
(iii) Financial assets carried at fair value through profit or loss (FVTPL)
A financial asset which is not classified in any of the above categories is subsequently fair valued through profit or loss.
(iv) Financial liabilities
Financial liabilities are subsequently carried at amortized cost using the effective interest method, except for contingent consideration and financial liability under option arrangements recognized in a business combination which is subsequently measured at fair value through profit or loss.
b. Derivative financial instruments
The Group holds derivative financial instruments such as foreign exchange forward and option contracts to mitigate the risk of changes in exchange rates on foreign currency exposures. The counterparty for such contracts is generally a bank.
(i) Financial assets or financial liabilities, carried at fair value through profit or loss.
This category includes derivative financial assets or liabilities which are not designated as hedges.
Although the Group believes that these derivatives constitute hedges from an economic perspective, they may not qualify for hedge accounting under Ind AS 109, Financial Instruments. Any derivative that is either not designated as hedge, or is so designated but is ineffective as per Ind AS 109, is categorized as a financial asset or financial liability, at fair value through profit or loss.
Derivatives not designated as hedges are recognized initially at fair value and attributable transaction costs are recognized in net profit in the Consolidated Statement of Profit and Loss when incurred. Subsequent to initial recognition, these derivatives are measured at fair value through profit or loss and the resulting exchange gains or losses are included in other income. Assets/ liabilities in this category are presented as current assets/current liabilities if they are either held for trading or are expected to be realized within 12 months after the Balance Sheet date.
(ii) Cash flow hedge
The Group designates certain foreign exchange forward and options contracts as cash flow hedges to mitigate the risk of foreign exchange exposure on highly probable forecast cash transactions.
When a derivative is designated as a cash flow hedging instrument, the effective portion of changes in the fair value of the derivative is recognized in other comprehensive income and accumulated in the cash flow hedging reserve. Any ineffective portion of changes in the fair value of the derivative is recognized immediately in the net profit in the Consolidated Statement of Profit and Loss. If the hedging instrument no longer meets the criteria for hedge accounting, then hedge accounting is discontinued prospectively. If the hedging instrument expires or is sold, terminated or exercised, the cumulative gain or loss on the hedging instrument recognized in cash flow hedging reserve till the period the hedge was effective remains in cash flow hedging reserve until the forecasted transaction occurs. The cumulative gain or loss previously recognized in the cash flow hedging reserve is transferred to the net profit in the Consolidated Statement of Profit and Loss upon the occurrence of the related forecasted transaction. If the forecasted transaction is no longer expected to occur, then the amount accumulated in cash flow hedging reserve is reclassified to net profit in the Consolidated Statement of Profit and Loss.
2.10.3 Derecognition of financial instruments
The Group derecognizes a financial asset when the contractual rights to the cash flows from the financial asset expire or it transfers the financial asset and the transfer qualifies for derecognition under Ind AS 109. A financial liability (or a part of a financial liability) is derecognized from the Group's Balance Sheet when the obligation specified in the contract is discharged or cancelled or expires.
18
2.10.4 Fair value of financial instruments
In determining the fair value of its financial instruments, the Group uses a variety of methods and assumptions that are based on market conditions and risks existing at each reporting date. The methods used to determine fair value include discounted cash flow analysis, option pricing model, market multiples, available quoted market prices and dealer quotes. All methods of assessing fair value result in general approximation of value, and such value may never actually be realized.
Refer to table 'Financial instruments by category' below for the disclosure on carrying value and fair value of financial assets and liabilities. For financial assets and liabilities maturing within one year from the Balance Sheet date and which are not carried at fair value, the carrying amounts approximates fair value due to the short maturity of these instruments.
2.10.5 Impairment
The Group recognizes loss allowances using the expected credit loss (ECL) model for the financial assets and unbilled revenue which are not fair valued through profit or loss. Loss allowance for trade receivables and unbilled revenues with no significant financing component is measured at an amount equal to lifetime ECL. For all other financial assets, ECLs are measured at an amount equal to the 12-month ECL, unless there has been a significant increase in credit risk from initial recognition in which case those are measured at lifetime ECL.
The Group determines the allowance for credit losses based on historical loss experience adjusted to reflect current and estimated future economic conditions. The Group considers current and anticipated future economic conditions relating to industries the Group deals with and the countries where it operates.
The amount of ECL (or reversal) that is required to adjust the loss allowance at the reporting date to the amount that is required to be recorded is recognized as an impairment loss or gain in Consolidated Statement of Profit and Loss.
Financial instruments by category
The carrying value and fair value of financial instruments by categories as at March 31, 2024 are as follows:
<table><thead><tr><th rowspan="2">Particulars</th><th rowspan="2">Amortized cost</th><th colspan="2">Financial assets/ liabilities at fair value through profit or loss</th><th colspan="2">Financial assets/liabilities at fair value through OCI</th><th rowspan="2">Total carrying value</th><th rowspan="2">Total fair v'alue</th></tr><tr><th>Designated 
 upon initial recognition</th><th>Mandatory</th><th>Equity instruments designated upon initial recognition</th><th>Mandatory</th></tr></thead><tbody><tr><td colspan="8">Assets:</td></tr><tr><td>Cash and cash equivalents (Refer to Note 2.8)</td><td>14,786</td><td>—</td><td></td><td></td><td></td><td>14,786</td><td>14,786</td></tr><tr><td colspan="8">Investments (Refer to Note 2.4)</td></tr><tr><td>Equity and preference securities</td><td>—</td><td>—</td><td></td><td>206</td><td></td><td>206</td><td>206</td></tr><tr><td>Tax free bonds and government bonds</td><td>1,759</td><td>—</td><td></td><td></td><td></td><td>,759</td><td>973 (1</td></tr><tr><td>Liquid mutual fund units</td><td>—</td><td></td><td>2,615</td><td></td><td></td><td>2,615</td><td>2,615</td></tr><tr><td>Target maturity fund units</td><td>—</td><td></td><td>431</td><td></td><td></td><td>431</td><td>431</td></tr><tr><td>Non convertible debentures</td><td>—</td><td>—</td><td></td><td></td><td>4,179</td><td>4,179</td><td>4,179</td></tr><tr><td>Government securities</td><td>—</td><td>—</td><td></td><td></td><td>7,362</td><td>7,362</td><td>7,362</td></tr><tr><td>Commercial papers</td><td>—</td><td>—</td><td></td><td></td><td>4,830</td><td>4,830</td><td>4,830</td></tr><tr><td>Certificates of deposit</td><td>—</td><td>—</td><td></td><td></td><td>3,043</td><td>3,043</td><td>3,043</td></tr><tr><td>Other investments</td><td>—</td><td></td><td>198</td><td></td><td></td><td>98</td><td>98</td></tr><tr><td>Trade receivables (Refer to Note 2.7)</td><td>30,193</td><td>—</td><td></td><td></td><td></td><td>30,193</td><td>30,193</td></tr><tr><td>Loans (Refer to Note 2.5)</td><td>282</td><td>—</td><td></td><td></td><td></td><td>282</td><td>282</td></tr><tr><td>Other financials assets (Refer to Note 2.6) @</td><td>15,106</td><td></td><td>61</td><td></td><td>23</td><td>15,190</td><td>= 15,106</td></tr><tr><td>Total</td><td>62,126</td><td></td><td>3,305</td><td>206</td><td>19,437</td><td>85,074</td><td>85,204</td></tr><tr><td colspan="8">Liabilities:</td></tr><tr><td>Trade payables</td><td>3,956</td><td>—</td><td></td><td></td><td></td><td>3,956</td><td>3,956</td></tr><tr><td>Lease liabilities (Refer to Note 2.19)</td><td>8,359</td><td>—</td><td></td><td></td><td></td><td>8,359</td><td>8,359</td></tr><tr><td>Financial Liability under option arrangements (Refer to Note 2.12)</td><td>—</td><td></td><td>597</td><td></td><td></td><td>597</td><td>597</td></tr><tr><td>Other financial liabilities (Refer to Note 2.12)</td><td>15,750</td><td></td><td>30</td><td></td><td></td><td>15,781</td><td>15,781</td></tr><tr><td>Total</td><td>28,065</td><td></td><td>627</td><td></td><td></td><td>28,693</td><td>28,693</td></tr></tbody></table>
(1) On account of fair value changes including interest accrued
(2) Excludes interest accrued on tax free bonds and government bonds carried at amortized cost of ₹84 crore
(In ₹ crore)
[89]
(2)
(3) Excludes unbilled revenue on contracts where the right to consideration is dependent on completion of contractual milestones
19
The carrying value and fair value of financial instruments by categories as at March 31, 2023 were as follows:
<table><thead><tr><th rowspan="3">Particulars</th><th rowspan="3">Amortized 
 cost</th><th colspan="2" rowspan="2">Financial assets/ liabilities at fair value through profit or loss</th><th></th><th></th><th></th><th rowspan="3">(In X crore) Total fair value</th></tr><tr><th colspan="2">Financial assets/liabilities at fair value through OCI</th><th rowspan="2">Total carrying value</th></tr><tr><th>Designated 
 upon initial recognition</th><th>Mandatory</th><th>Equity instruments designated upon initial recognition</th><th>Mandatory</th></tr></thead><tbody><tr><td colspan="8">Assets:</td></tr><tr><td>Cash and cash equivalents (Refer to Note 2.8)</td><td>12,173</td><td>—</td><td></td><td></td><td></td><td>12,173</td><td>12,173</td></tr><tr><td colspan="8">Investments (Refer to Note 2.4)</td></tr><tr><td>Equity and preference securities</td><td>—</td><td>—</td><td></td><td>196</td><td></td><td>196</td><td>196</td></tr><tr><td>Tax free bonds and government bonds</td><td>1,920</td><td>—</td><td></td><td></td><td></td><td>1,920</td><td>(1) 2,148</td></tr><tr><td>Liquid mutual fund units</td><td>—</td><td></td><td>975</td><td></td><td></td><td>975</td><td>975</td></tr><tr><td>Target maturity fund units</td><td>—</td><td></td><td>402</td><td></td><td></td><td>402</td><td>402</td></tr><tr><td>Non convertible debentures</td><td>—</td><td>—</td><td></td><td></td><td>3,868</td><td>3,868</td><td>3,868</td></tr><tr><td>Government securities</td><td>—</td><td>—</td><td></td><td></td><td>7,632</td><td>7,632</td><td>7,632</td></tr><tr><td>Commercial papers</td><td>—</td><td>—</td><td></td><td></td><td>742</td><td>742</td><td>742</td></tr><tr><td>Certificates of deposit</td><td>—</td><td>—</td><td></td><td></td><td>3,574</td><td>3,574</td><td>3,574</td></tr><tr><td>Other investments</td><td>—</td><td></td><td>169</td><td></td><td></td><td>169</td><td>169</td></tr><tr><td>Trade receivables (Refer to Note 2.7)</td><td>25,424</td><td>—</td><td></td><td></td><td></td><td>25,424</td><td>25,424</td></tr><tr><td>Loans (Refer to Note 2.5)</td><td>328</td><td>—</td><td></td><td></td><td></td><td>328</td><td>328</td></tr><tr><td>Other financials assets (Refer to Note 2.6) @</td><td>14,301</td><td></td><td>69</td><td></td><td>32</td><td>14,402</td><td>14,318 (2)</td></tr><tr><td>Total</td><td>54,146</td><td></td><td>1,615</td><td>196</td><td>15,848</td><td>71,805</td><td>71,949</td></tr><tr><td colspan="8">Liabilities:</td></tr><tr><td>Trade payables</td><td>3,865</td><td>—</td><td></td><td></td><td></td><td>3,865</td><td>3,865</td></tr><tr><td>Lease liahilities (Refer to Note 2.19)</td><td>8,299</td><td>—</td><td></td><td></td><td></td><td>8,299</td><td>8,299</td></tr><tr><td>Financial Liability under option arrangements Refer to Note 2.12)</td><td>.</td><td></td><td>600</td><td></td><td></td><td>600</td><td>600</td></tr><tr><td>Other financial liabilities (Refer to Note 2.12)</td><td>17,359</td><td></td><td>161</td><td></td><td>14</td><td>17,534</td><td>17,534</td></tr><tr><td>Total</td><td>29,523</td><td></td><td>761</td><td></td><td>14</td><td>30,298</td><td>30,298</td></tr></tbody></table>
(1) On account of fair value changes including interest accrued
(2) Excludes interest accrued on tax free bonds and government bonds carried at amortized cost of ₹84 crore
(3) Excludes unbilled revenue on contracts where the right to consideration is dependent on completion of contractual milestones
For trade receivables, trade payables, other assets and payables maturing within one year from the Balance Sheet date, the carrying amounts approximate the fair value due to the short maturity of these instruments.
Fair value hierarchy
Level 1 - Quoted prices (unadjusted) in active markets for identical assets or liabilities.
Level 2 - Inputs other than quoted prices included within Level 1 that are observable for the asset or liability, either directly (i.e. as prices) or indirectly (i.e. derived from prices). Level 3 - Inputs for the assets or liabilities that are not based on observable market data (unobservable inputs).
The fair value hierarchy of assets and liabilities measured at fair value on a recurring basis as at March 31, 2024 is as follows:
<table><thead><tr><th rowspan="2">Particulars</th><th rowspan="2">Asat March 31, 2024</th><th colspan="3">Fair value measurement at end of the reporting period using</th></tr><tr><th>Level 1</th><th>Level 2</th><th>Level 3</th></tr></thead><tbody><tr><td colspan="5">Assets</td></tr><tr><td colspan="5">Investments (Refer to note 2.4)</td></tr><tr><td>nvestments in liquid mutual funds</td><td>2,615</td><td>2,615</td><td></td><td>—</td></tr><tr><td>nvestments in target maturity fund units</td><td>431</td><td>431</td><td></td><td>—</td></tr><tr><td>nvestments in tax free bonds</td><td>1,944</td><td>1,944</td><td></td><td>—</td></tr><tr><td>nvestments in government bonds</td><td>29</td><td>29</td><td></td><td>—</td></tr><tr><td>nvestments in non convertible debentures</td><td>4,179</td><td>3,922</td><td>257</td><td>—</td></tr><tr><td>nvestment in government securities</td><td>7,362</td><td>7,289</td><td>73</td><td>—</td></tr><tr><td>nvestments in equity securities</td><td>15</td><td></td><td></td><td>115</td></tr><tr><td>nvestments in preference securities</td><td>91</td><td></td><td></td><td>91</td></tr><tr><td>nvestments in commercial papers</td><td>4,830</td><td></td><td>4,830</td><td>—</td></tr><tr><td>nvestments in certificates of deposit</td><td>3,043</td><td></td><td>3,043</td><td>—</td></tr><tr><td>Other investments</td><td>98</td><td></td><td></td><td>198</td></tr><tr><td colspan="5">Others</td></tr><tr><td>Derivative financial instruments - gain on outstanding foreign exchange forward and option</td><td>84</td><td></td><td>84</td><td></td></tr><tr><td>contracts (Refer to Note 2.6)</td><td></td><td>—</td><td></td><td>—</td></tr><tr><td colspan="5">Liabilities</td></tr><tr><td>Derivative financial instruments - loss on outstanding foreign exchange forward and option</td><td>31</td><td></td><td>31</td><td></td></tr><tr><td colspan="5">contracts (Refer to Note 2.12)                         — 
                             —</td></tr><tr><td>Financial liability under option arrangements (Refer to Note 2.12) M</td><td>597</td><td></td><td></td><td>597</td></tr></tbody></table>
M Discount rate ranges from 9% to 15%
(In X crore)
[89]
(2)
20
During the year ended March 31, 2024, government securities , non convertible debentures and tax free bonds of ₹2,143 crore was transferred from Level 2 to Level 1 of fair value hierarchy, since these were valued based on quoted price. Further, government securities of ₹ 73 crore were transferred from Level 1 to Level 2 of fair value hierarchy, since these were valued based on market observable inputs.
The fair value hierarchy of assets and liabilities measured at fair value on a recurring basis as at March 31, 2023 was as follows:
<table><thead><tr><th rowspan="2">Particulars</th><th rowspan="2">Asat March 31, 2023</th><th colspan="3">Fair value measurement at end of the reporting period using</th></tr><tr><th>Level 1</th><th>Level 2</th><th>Level 3</th></tr></thead><tbody><tr><td colspan="5">Assets</td></tr><tr><td colspan="5">Investments (Refer to note 2.4)</td></tr><tr><td>Investments in liquid mutual funds</td><td>975</td><td>975</td><td></td><td></td></tr><tr><td>nvestments in target maturity fund units</td><td>402</td><td>402</td><td></td><td></td></tr><tr><td>Investments in tax free bonds</td><td>2,120</td><td>1,331</td><td>789</td><td></td></tr><tr><td>nvestments in government bonds</td><td>28</td><td>28</td><td></td><td></td></tr><tr><td>Investments in non convertible debentures</td><td>3,868</td><td>1,793</td><td>2,075</td><td></td></tr><tr><td>nvestment in government securities</td><td>7,632</td><td>7,549</td><td>83</td><td></td></tr><tr><td>Investments in equity securities</td><td></td><td>3                             —                                 —                                    3</td><td></td><td></td></tr><tr><td>nvestments in preference securities</td><td>193</td><td></td><td></td><td>193</td></tr><tr><td>Investments in commercial papers</td><td>742</td><td></td><td>742</td><td>—</td></tr><tr><td>nvestments in certificates of deposit</td><td>3,574</td><td></td><td>3,574</td><td>—</td></tr><tr><td>Other investments</td><td>169</td><td></td><td></td><td>169</td></tr><tr><td colspan="5">Others</td></tr><tr><td>Derivative financial instruments - gain on outstanding foreign exchange forward and option contracts (Refer to Note 2.6)</td><td>101</td><td></td><td>101</td><td></td></tr><tr><td colspan="5">Liabilities</td></tr><tr><td>Derivative financial instruments - loss on outstanding foreign exchange forward and option contracts (Refer to Note 2.12)</td><td>78</td><td></td><td>78</td><td></td></tr><tr><td>Financial liability under option arrangements (Refer to Note 2.12) M</td><td>600</td><td></td><td></td><td>600</td></tr><tr><td>Liability towards contingent consideration (Refer to Note 2.12)</td><td>97</td><td></td><td></td><td>97</td></tr></tbody></table>
Discount rate ranges from 10% to 15%
During the year ended March 31, 2023, government securities and tax free bonds of ₹383 crore was transferred from Level 2 to Level 1 of fair value hierarchy, since these were valued based on quoted price. Further, non-convertible debentures of ₹1,611 crore were transferred from Level 1 to Level 2 of fair value hierarchy, since these were valued based on market observable inputs.
A one percentage point change in the unobservable inputs used in fair valuation of Level 3 assets and liabilities does not have a significant impact in its value.
Majority of investments of the Group are fair valued based on Level 1 or Level 2 inputs. These investments primarily include investment in liquid mutual fund units, target maturity fund units, tax free bonds, certificates of deposit, commercial papers, treasury bills, government securities, non-convertible debentures, quoted bonds issued by government and quasi-government organizations. The Group invests after considering counterparty risks based on multiple criteria including Tier I capital, Capital Adequacy Ratio, Credit Rating, Profitability, NPA levels and Deposit base of banks and financial institutions. These risks are monitored regularly as per Group's risk management program.
(In X crore)
21
2.11 EQUITY
Accounting policy
Ordinary Shares
Ordinary shares are classified as equity share capital. Incremental costs directly attributable to the issuance of new ordinary shares, share options and buyback are recognized as a deduction from equity, net of any tax effects.
Treasury Shares
When any entity within the Group purchases the company's ordinary shares, the consideration paid including any directly attributable incremental cost is presented as a deduction from total equity, until they are cancelled, sold or reissued. When treasury shares are sold or reissued subsequently, the amount received is recognized as an increase in equity, and the resulting surplus or deficit on the transaction is transferred to / from securities premium.
Description of reserves
Capital Redemption Reserve
In accordance with section 69 of the Indian Companies Act, 2013, the Company creates capital redemption reserve equal to the nominal value of the shares bought back as an appropriation from general reserve / retained earnings.
Retained earnings
Retained earnings represent the amount of accumulated earnings of the Group.
Securities premium
The amount received in excess of the par value of equity shares has been classified as securities premium. Amounts have been utilized for bonus issue and share buyback from share premium account.
Share options outstanding account
The share options outstanding account is used to record the fair value of equity-settled share based payment transactions with employees. The amounts recorded in share options outstanding account are transferred to securities premium upon exercise of stock options and transferred to general reserve on account of stock options not exercised by employees.
Special Economic Zone Re-investment reserve
The Special Economic Zone Re-investment reserve has been created out of the profit of the eligible SEZ unit in terms of the provisions of Sec 10AA (1)(ii) of Income Tax Act, 1961. The reserve should be utilized by the Company for acquiring new plant and machinery for the purpose of its business in terms of the provisions of the Sec 10AA (2) of the Income Tax Act, 1961.
Other components of equity
Other components of equity include currency translation, remeasurement of net defined benefit liability / asset, equity instruments fair valued through other comprehensive income, changes on fair valuation of investments and changes in fair value of derivatives designated as cash flow hedges, net of taxes.
Currency translation reserve
The exchange differences arising from the translation of financial statements of foreign subsidiaries with functional currency other than Indian rupees is recognized in other comprehensive income and is presented within equity.
Cash flow hedge reserve
When a derivative is designated as a cash flow hedging instrument, the effective portion of changes in the fair value of the derivative is recognized in other comprehensive income and accumulated in the cash flow hedging reserve. The cumulative gain or loss previously recognized in the cash flow hedging reserve is transferred to the interim condensed Consolidated Statement of Profit and Loss upon the occurrence of the related forecasted transaction.
EQUITY SHARE CAPITAL
<table><thead><tr><th rowspan="2">Particulars</th><th colspan="2">(In &lt; crore, except as otherwise stated) As at</th></tr><tr><th>March 31, 2024</th><th>March 31, 2023</th></tr></thead><tbody><tr><td colspan="3">Authorized</td></tr><tr><td colspan="3">Equity shares, Z5/- par value</td></tr><tr><td>4,80,00,00,000 (4,80,00,00,000) equity shares</td><td>2,400</td><td>2,400</td></tr><tr><td colspan="3">Issued, Subscribed and Paid-Up</td></tr><tr><td>Equity shares, Z5/- par value®</td><td>2,071</td><td>2,069</td></tr><tr><td colspan="3">4,13,99,50,635 (4,13,63,87,925) equity shares fully paid-up®</td></tr><tr><td></td><td>2,071</td><td>2,069</td></tr></tbody></table>
Note: Forfeited shares amounted to ₹1,500 (₹1,500)
(1) Refer to Note 2.20 for details of basic and diluted shares
(2) Net of treasury shares 1,09,16,829 (1,21,72,119)
The Company has only one class of shares referred to as equity shares having a par value of ₹5/-. Each holder of equity shares is entitled to one vote per share. The equity shares represented by American Depositary Shares (ADS) carry similar rights to voting and dividends as the other equity shares. Each ADS represents one underlying equity share.
In the event of liquidation of the Company, the holders of equity shares will be entitled to receive any of the remaining assets of the Company in proportion to the number of equity shares held by the shareholders, after distribution of all preferential amounts. However, no such preferential amounts exist currently, other than the amounts held by irrevocable controlled trusts. For irrevocable controlled trusts, the corpus would be settled in favor of the beneficiaries.
There are no voting, dividend or liquidation rights to the holders of options issued under the company's share option plans
For details of shares reserved for issue under the employee stock option plan of the Company refer to the note below.
22
The reconciliation of the number of shares outstanding and the amount of share capital as at March 31, 2024 and March 31, 2023 are as follows:
<table><thead><tr><th rowspan="2">Particulars</th><th colspan="2">Asat March 31, 2024</th><th colspan="2">(In &lt; crore, except as stated otherwise) As at March 31, 2023</th></tr><tr><th>Number of shares</th><th>Amount</th><th>Number of shares</th><th>Amount</th></tr></thead><tbody><tr><td>As at the beginning of the period</td><td>413,63,87,925</td><td>2,069</td><td>419,30,12,929</td><td>2,098</td></tr><tr><td>Add: Shares issued on exercise of employee stock options</td><td>35,62,710</td><td>2</td><td>38,01,344</td><td>1</td></tr><tr><td>Less: Shares bought back</td><td>—</td><td>—</td><td>6,04,26,348</td><td>30</td></tr><tr><td>As at the end of the period</td><td>413,99,50,635</td><td>2,071</td><td>413,63,87,925</td><td>2,069</td></tr></tbody></table>
Capital allocation policy
Effective from financial year 2025, the Company expects to continue the policy of returning approximately 85% of the free cash flow cumulatively over a 5-year period through a combination of semi-annual dividends and/or share buyback/ special dividends subject to applicable laws and requisite approvals, if any. Under this policy, the Company expects to progressively increase its annual dividend per share (excluding special dividend if any).
Free cash flow is defined as net cash provided by operating activities less capital expenditure as per the consolidated statement of cash flows prepared under IFRS. Dividend and buyback include applicable taxes
Buyback completed in February 2023
In line with the capital allocation policy, the Board, at its meeting held on October 13, 2022, approved the buyback of equity shares, from the open market route through the Indian stock exchanges, amounting to ₹9,300 crore (Maximum Buyback Size, excluding buyback tax) at a price not exceeding ₹1,850 per share (Maximum Buyback Price), subject to shareholders' approval by way of Postal Ballot.
The shareholders approved the proposal of buyback of Equity Shares recommended by its Board of Directors by way of e-voting on the postal ballot, the results of which were declared on December 3, 2022. The buyback was offered to all equity shareholders of the Company (other than the Promoters, the Promoter Group and Persons in Control of the Company) under the open market route through the stock exchange. The buyback of equity shares through the stock exchange commenced on December 7, 2022 and was completed on February 13, 2023. During this buyback period the Company had purchased and extinguished a total of 60,426,348 equity shares from the stock exchange at a volume weighted average buyback price of ₹1,539.06/- per equity share comprising 1.44% of the pre buyback paid-up equity share capital of the Company. The buyback resulted in a cash outflow of ₹9,300 crore (excluding transaction costs and tax on buyback). The Company funded the buyback from its free reserves including Securities Premium as explained in Section 68 of the Companies Act, 2013.
In accordance with section 69 of the Companies Act, 2013, as at March 31, 2023, the Company has created ‘Capital Redemption Reserve’ of ₹30 crore equal to the nominal value of the shares bought back as an appropriation from general reserve and retained earnings.
The Company’s objective when managing capital is to safeguard its ability to continue as a going concern and to maintain an optimal capital structure so as to maximize shareholder value. In order to maintain or achieve an optimal capital structure, the Company may adjust the amount of dividend payment, return capital to shareholders, issue new shares or buy back issued shares. As of March 31, 2024, the Company has only one class of equity shares and has no debt. Consequent to the above capital structure there are no externally imposed capital requirements.
Dividend
The final dividend on shares is recorded as a liability on the date of approval by the shareholders and interim dividends are recorded as a liability on the date of declaration by the Company's Board of Directors. Income tax consequences of dividends on financial instruments classified as equity will be recognized according to where the entity originally recognized those past transactions or events that generated distributable profits.
The Company declares and pays dividends in Indian rupees. Companies are required to pay/distribute dividend after deducting applicable taxes. The remittance of dividends outside India is governed by Indian law on foreign exchange and is also subject to withholding tax at applicable rates.
The amount of per share dividend recognized as distribution to equity shareholders is as follows:
<table><thead><tr><th rowspan="2">Particulars</th><th colspan="2">Three months ended March 31,</th><th colspan="2">Y ear ended March 31,</th></tr><tr><th></th><th></th><th></th><th>2023</th></tr></thead><tbody><tr><td>Final dividend for fiscal 2022</td><td>20%                          —</td><td>20?</td><td>20274</td><td>16.00</td></tr><tr><td>Interim dividend for fiscal 2023</td><td>—</td><td>—</td><td>—</td><td>16.50</td></tr><tr><td>Final dividend for fiscal 2023</td><td>—</td><td>—</td><td>17.50</td><td></td></tr><tr><td>Interim dividend for fiscal 2024</td><td>—</td><td>—</td><td>18.00</td><td></td></tr></tbody></table>
During the year ended March 31, 2024, on account of the final dividend for fiscal 2023 and interim dividend for fiscal 2024, the Company has incurred a net cash outflow of ₹14,692 crore (excluding dividend paid on treasury shares)
The Board of Directors in their meeting held on April 18, 2024 recommended a final dividend of ₹20/- per equity share for the financial year ended March 31, 2024 and a special dividend of ₹8/- per equity share. The payment is subject to the approval of shareholders in the AGM of the Company to be held on June 26, 2024 and if approved, would result in a net cash outflow of approximately ₹11,592 crore (excluding dividend paid on treasury shares).
Employee Stock Option Plan (ESOP):
Accounting policy
The Group recognizes compensation expense relating to share-based payments in net profit based on estimated fair values of the awards on the grant date. The estimated fair value of awards is recognized as an expense in the statement of profit and loss on a straight-line basis over the requisite service period for each separately vesting portion of the award as if the award was in- substance, multiple awards with a corresponding increase to share options outstanding account.
Infosys Expanded Stock Ownership Program 2019 (the 2019 Plan) :
On June 22, 2019 pursuant to approval by the shareholders in the Annual General Meeting, the Board has been authorized to introduce, offer, issue and provide share-based incentives to eligible employees of the Company and its subsidiaries under the 2019 Plan. The maximum number of shares under the 2019 Plan shall not exceed 5,00,00,000 equity shares. To implement the 2019 Plan, up to 4,50,00,000 equity shares may be issued by way of secondary acquisition of shares by Infosys Expanded Stock Ownership Trust. The Restricted Stock Units (RSUs) granted under the 2019 Plan shall vest based on the achievement of defined annual performance parameters as determined by the administrator (Nomination and Remuneration Committee). The performance parameters will be based on a combination of relative Total Shareholder Return (TSR) against selected industry peers and certain broader market domestic and global indices and operating performance metrics of the Company as decided by administrator. Each of the above performance parameters will be distinct for the purposes of calculation of quantity of shares to vest based on performance. These instruments will generally vest between a minimum of 1 to maximum of 3 years from the grant date.
2015 Stock Incentive Compensation Plan (the 2015 Plan) :
On March 31, 2016, pursuant to the approval by the shareholders through postal ballot, the Board was authorized to introduce, offer, issue and allot share-based incentives to eligible employees of the Company and its subsidiaries under the 2015 Stock Incentive Compensation Plan. The maximum number of shares under the 2015 Plan shall not exceed 2,40,38,883 equity shares (this includes 1,12,23,576 equity shares which are held by the trust towards the 2011 Plan as at March 31, 2016). These instruments will generally vest over a period of 4 years. The plan numbers mentioned above are further adjusted with the September 2018 bonus issue.
The equity settled and cash settled RSUs and stock options would vest generally over a period of 4 years and shall be exercisable within the period as approved by the Nomination and Remuneration Committee (NARC). The exercise price of the RSUs will be equal to the par value of the shares and the exercise price of the stock options would be the market price as on the date of grant.
Controlled trust holds 10,916,829 and 1,21,72,119 shares as at March 31, 2024 and March 31, 2023, respectively, under the 2015 Plan. Out of these shares, 2,00,000 equity shares each have been earmarked for welfare activities of the employees as at March 31, 2024 and March 31, 2023.
23
The following is the summary of grants made during the three months and year ended March 31, 2024 and March 31, 2023:
<table><thead><tr><th rowspan="3">Particulars</th><th colspan="4">2019 Plan</th><th colspan="4">2015 Plan</th></tr><tr><th colspan="2">Three months ended March 31,</th><th colspan="2">Y ear ended March 31,</th><th colspan="2">Three months ended March 31,</th><th colspan="2">Year ended March 31,</th></tr><tr><th>2024</th><th>2023</th><th>2024</th><th>2023</th><th>2024</th><th>2023</th><th>2024</th><th>2023</th></tr></thead><tbody><tr><td colspan="9">Equity Settled RSUs</td></tr><tr><td>Key Management Personnel (KMP)</td><td>26,900</td><td>33,750</td><td>141,171</td><td>210,643</td><td>77,094</td><td>80,154</td><td>498,730</td><td>367,479</td></tr><tr><td>Employees other than KMP</td><td>3,582,471</td><td>3,329,240</td><td>4,046,731</td><td>3,704,014</td><td>3,442,700</td><td>1,736,925</td><td>4,640,640</td><td>1,784,975</td></tr><tr><td></td><td>3,609,371</td><td>3,362,990</td><td>4,187,902</td><td>3,914,657</td><td>3,519,794</td><td>1,817,079</td><td>5,139,370</td><td>2,152,454</td></tr><tr><td colspan="9">Cash settled RSU</td></tr><tr><td>Key Management Personnel (KMP)</td><td></td><td></td><td>-                     -                      -</td><td></td><td></td><td></td><td>-                              -                          -                                  -                                  -</td><td></td></tr><tr><td>Employees other than KMP</td><td></td><td></td><td>-                     -                      -</td><td></td><td>169,04(;                   -                   169,040</td><td>92,40(;</td><td>176,99(;               92,400                      176,990                        92,400</td><td>92,40(;</td></tr><tr><td></td><td>-</td><td></td><td>-                      -</td><td></td><td>169,040</td><td>92,400</td><td>176,990</td><td>92,400</td></tr><tr><td>Total Grants</td><td>3,609,371</td><td>3,362,990</td><td>4,187,902</td><td>3,914,657</td><td>3,688,834</td><td>1,909,479</td><td>5,316,360</td><td>2,244,854</td></tr></tbody></table>
Notes on grants to KMP:
CEO & MD
Under the 2015 Plan:
The Board, on April 13, 2023, based on the recommendations of the Nomination and Remuneration Committee approved the following grants for fiscal 2024. In accordance with such approval the following grants were made effective May 2, 2023.
- 2,72,026 performance-based RSUs (Annual performance equity grant) of fair value of ₹34.75 crore. These RSUs will vest in line with the employment agreement based on achievement of certain performance targets.
15,656 performance-based grant of RSUs (Annual performance equity ESG grant) of fair value of 22 crore. These RSUs will vest in line with the employment agreement based on achievement
of certain environment, social and governance milestones as determined by the Board. - 39,140 performance-based grant of RSUs (Annual performance Equity TSR grant) of fair value of ₹5 crore . These RSUs will vest in line with the employment agreement based on Company’s performance on cumulative relative TSR over the years and as determined by the Board.
Further, in accordance with the employee agreement which has been approved by the shareholders, the CEO is eligible to receive an annual grant of RSUs of fair value ₹3 crore which will vest overtime in three equal annual installments upon the completion of each year of service from the respective grant date. Accordingly, annual time-based grant of 18,104 RSUs was made effective February 1, 2024 for fiscal 2024.
Though the annual time based grants and annual performance equity TSR grant for the remaining employment term ending on March 31, 2027 have not been granted as of March 31, 2024, since the service commencement date precedes the grant date, the company has recorded employment stock compensation expense in accordance with Ind AS 102, Share based payment. The grant date for this purpose in accordance with Ind AS 102, Share based payment is July 1, 2022.
Under the 2019 Plan:
The Board, on April 13, 2023, based on the recommendations of the Nomination and Remuneration Committee, approved performance-based grant of RSUs amounting to ₹10 crore for fiscal 2024 under the 2019 Plan. These RSUs will vest based on achievement of certain performance targets. Accordingly, 78,281 performance based RSU’s were granted effective May 2, 2023.
Other KMP
Under the 2015 Plan:
During the year ended March 31, 2024, based on recommendations of Nomination and Remuneration Committee, the Board approved 1,47,030 time based RSUs and 6,774 performance based RSUs to other KMP under the 2015 plan. Time based RSUs will vest over three to four years and performance based RSUs will vest over three years based on certain performance targets.
Under the 2019 Plan:
During the year ended March 31, 2024, based on recommendations of Nomination and Remuneration Committee, the Board approved performance based grants of 62,890 RSUs to other KMPs under the 2019 plan. These RSUs will vest over three years based on achievement of certain performance targets.
The break-up of employee stock compensation expense is as follows:
<table><thead><tr><th rowspan="2">Particulars</th><th colspan="2" rowspan="2">Three months ended March 31, 2024
 2023</th><th colspan="2" rowspan="2">(in X crore) Year ended March 31, 2024
 2023</th></tr></thead><tbody><tr><td colspan="5">Granted to:</td></tr><tr><td>KMP</td><td>17</td><td>8</td><td>68</td><td>49</td></tr><tr><td>Employees other than KMP</td><td>208</td><td>125</td><td>584</td><td>470</td></tr><tr><td>Total ®</td><td>225</td><td>133</td><td>652</td><td>519</td></tr><tr><td>C ash-settled stock compensation expense included in the above</td><td>4</td><td>2</td><td>13</td><td>5</td></tr></tbody></table>
The fair value of the awards are estimated using the Black-Scholes Model for time and non-market performance based options and Monte Carlo simulation model is used for TSR based options.
The inputs to the model include the share price at date of grant, exercise price, expected volatility, expected dividends, expected term and the risk free rate of interest. Expected volatility during the expected term of the options is based on historical volatility of the observed market prices of the Company's publicly traded equity shares during a period equivalent to the expected term of the options. Expected volatility of the comparative company have been modelled based on historical movements in the market prices of their publicly traded equity shares during a period equivalent to the expected term of the options. Correlation coefficient is calculated between each peer entity and the indices as a whole or between each entity in the peer group.
The fair value of each equity settled award is estimated on the date of grant using the following assumptions:
<table><thead><tr><th rowspan="2">Particulars</th><th colspan="4">For options granted in</th></tr><tr><th>Fiscal 2024- Equity Shares- RSU</th><th>Fiscal 2024- ADS-RSU</th><th>Fiscal 2023- Equity Shares-RSU</th><th>Fiscal 2023- ADS-RSU</th></tr></thead><tbody><tr><td>Weighted average share price (%) / ($ ADS)</td><td>1,588</td><td>19.19</td><td>1,525</td><td>18.08</td></tr><tr><td>Exercise price (%) / (§ ADS)</td><td>5.00</td><td>0.07</td><td>5.00</td><td>0.07</td></tr><tr><td>Expected volatility (%)</td><td>23-31</td><td>25-33</td><td>23-32</td><td>27-34</td></tr><tr><td>Expected life of the option (years)</td><td>1-4</td><td>1-4</td><td>1-4</td><td>1-4</td></tr><tr><td>Expected dividends (%)</td><td>2-3</td><td>2-3</td><td>2-3</td><td>2-3</td></tr><tr><td>Risk-free interest rate (%)</td><td>7</td><td>4-5</td><td>5-7</td><td>2-5</td></tr><tr><td>Weighted average fair value as on grant date () / ($ ADS)</td><td>1,317</td><td>16.27</td><td>1,210</td><td>13.69</td></tr></tbody></table>
The expected life of the RSU/ESOP is estimated based on the vesting term and contractual term of the RSU/ESOP, as well as expected exercise behavior of the employee who receives the RSU/ESOP.
24
2.12 OTHER FINANCIAL LIABILITIES
<table><thead><tr><th rowspan="3">Particulars</th><th colspan="2">(In X crore)</th></tr><tr><th>Asat</th><th></th></tr><tr><th>March 31, 2024</th><th>March 31, 2023</th></tr></thead><tbody><tr><td colspan="3">Non-current</td></tr><tr><td colspan="3">Others</td></tr><tr><td>Accrued compensation to employees m</td><td>7                                       5</td><td></td></tr><tr><td>(6] Accrued expenses</td><td>1,779</td><td>1,628</td></tr><tr><td>Compensated absences</td><td>89</td><td>83</td></tr><tr><td>Financial liability under option arrangements B</td><td>98</td><td></td></tr><tr><td>Other Payables (V@</td><td>157</td><td>342</td></tr><tr><td>Total non-current other financial liabilities</td><td>2,130</td><td>2,058</td></tr><tr><td>Current</td><td></td><td></td></tr><tr><td>Unpaid dividends @</td><td>37</td><td>37</td></tr><tr><td colspan="3">Others</td></tr><tr><td>Accrued compensation to employees ¥</td><td>4,454</td><td>4,174</td></tr><tr><td>(6] Accrued expenses</td><td>8,224</td><td>7,802</td></tr><tr><td>Payable for acquisition of business - Contingent consideration !</td><td></td><td>97</td></tr><tr><td>Payable by controlled trusts</td><td>21</td><td>211</td></tr><tr><td>Compensated absences</td><td>2,622</td><td>2,399</td></tr><tr><td>Financial liability under option arrangements B</td><td>499</td><td>600</td></tr><tr><td>Foreign currency forward and options contracts</td><td></td><td>78</td></tr><tr><td>Capital creditors ¥</td><td>310</td><td>674</td></tr><tr><td>Other payables V@</td><td>57</td><td>2,486</td></tr><tr><td colspan="3">Total current other financial liabilities 16,959 18,558</td></tr><tr><td>Total other financial liabilities</td><td>19,089</td><td>20,616</td></tr><tr><td>O Financial liability carried at amortized cost</td><td>15,750</td><td>17,359</td></tr><tr><td>@ Financial liability carried at fair value through profit or loss</td><td>627</td><td>761</td></tr><tr><td>) Financial liability carried at fair value through other comprehensive income</td><td></td><td>14</td></tr><tr><td>Financial liability under option arrangements on an undiscounted basis</td><td>690</td><td>676</td></tr><tr><td>Contingent consideration on undiscounted basis</td><td></td><td>101</td></tr></tbody></table>
(4) Deferred contract cost in note 2.9 includes technology assets taken over by the Group from a customer as a part of transformation project which is not considered as distinct goods or services and the control related to the assets is not transferred to the Group in accordance with Ind AS 115 - Revenue from contract with customers. Accordingly, the same has been considered as a reduction to the total contract value and accounted as Deferred contract cost. The Group has entered into financing arrangements with a third party for these assets. As at March 31, 2024 and March 31, 2023, the financial liability pertaining to such arrangements amounts to ₹372 crore and ₹731 crore, respectively. For the year ended March 31, 2023 ₹118 crore was settled directly by the third party to the customer on behalf of the Group and accordingly considered as non-cash transaction.
# Represents liability related to options issued by the Group over the non-controlling interests in its subsidiaries
Accrued expenses primarily relate to cost of technical sub-contractors, telecommunication charges, legal and professional charges, brand building expenses, overseas travel expenses, office maintenance and cost of third party software and hardware.
2.13 OTHER LIABILITIES
<table><thead><tr><th rowspan="2">Particulars</th><th>Asat March 2024</th><th>(In % crore) March 2023</th></tr><tr><th>31,</th><th>31,</th></tr></thead><tbody><tr><td colspan="3">Non-current</td></tr><tr><td>Others</td><td></td><td></td></tr><tr><td>Accrued defined benefit liability</td><td>159</td><td>445</td></tr><tr><td>Others</td><td>76</td><td>55</td></tr><tr><td>Total non-current other liabilities</td><td>235</td><td>500</td></tr><tr><td colspan="3">Current</td></tr><tr><td>Unearned revenue</td><td>7,341</td><td>7,163</td></tr><tr><td colspan="3">Others</td></tr><tr><td>Withholding taxes and others</td><td>3,185</td><td>3,632</td></tr><tr><td>Accrued defined benefit liability</td><td>5</td><td>4</td></tr><tr><td>Others</td><td>8</td><td>31</td></tr><tr><td>Total current other liabilities</td><td>10,539</td><td>10,830</td></tr><tr><td>Total other liabilities</td><td>10,774</td><td>11,330</td></tr></tbody></table>
(In X crore)
X
25
2.14 PROVISIONS
Accounting policy
A provision is recognized if, as a result of a past event, the Group has a present legal or constructive obligation that is reasonably estimable, and it is probable that an outflow of economic benefits will be required to settle the obligation. Provisions are determined by discounting the expected future cash flows at a pre-tax rate that reflects current market assessments of the time value of money and the risks specific to the liability.
a. Post sales client support
The Group provides its clients with a fixed-period post sales support on its fixed-price, fixed-timeframe contracts. Costs associated with such support services are accrued at the time related revenues are recorded and included in Consolidated Statement of Profit and Loss. The Group estimates such costs based on historical experience and estimates are reviewed on a periodic basis for any material changes in assumptions and likelihood of occurrence.
b. Onerous contracts
Provisions for onerous contracts are recognized when the expected benefits to be derived by the Group from a contract are lower than the unavoidable costs of meeting the future obligations under the contract. Provisions for estimated losses, if any, on incomplete contracts are recorded in the period in which such losses become probable based on the estimated efforts or costs to complete the contract. The provision is measured at the present value of the lower of the expected cost of terminating the contract and the expected net cost of continuing with the contract. Before a provision is established the Group recognizes any impairment loss on the assets associated with that contract.
Provision for post-sales client support and other provisions:
<table><thead><tr><th rowspan="2">Particulars</th><th>Asat</th><th>(In % crore)</th></tr><tr><th>March 31, 2024</th><th>March 31, 2023</th></tr></thead><tbody><tr><td colspan="3">Current</td></tr><tr><td colspan="3">Others</td></tr><tr><td>Post-sales client support and other provisions</td><td>1,796</td><td>1,307</td></tr><tr><td>Total provisions</td><td>1,796</td><td>1,307</td></tr></tbody></table>
Provision for post sales client support and other provisions majorly represents costs associated with providing sales support services which are accrued at the time of recognition of revenues and are expected to be utilized over a period of 1 year.
Provision for post sales client support and other provisions is included in cost of sales in the condensed consolidated statement of profit and loss.
26
2.15 INCOME TAXES
Accounting policy
Income tax expense comprises current and deferred income tax. Income tax expense is recognized in net profit in the Consolidated Statement of Profit and Loss except to the extent that it relates to items recognized directly in equity, in which case it is recognized in equity or other comprehensive income. Current income tax for current and prior periods is recognized at the amount expected to be paid to or recovered from the tax authorities, using the tax rates and tax laws that have been enacted or substantively enacted by the Balance Sheet date. Deferred income tax assets and liabilities are recognized for all temporary differences arising between the tax bases of assets and liabilities and their carrying amounts in the financial statements except when the deferred income tax arises from the initial recognition of goodwill or an asset or liability in a transaction that is not a business combination and affects neither accounting nor taxable profit or loss at the time of the transaction. Deferred tax assets are reviewed at each reporting date and are reduced to the extent that it is no longer probable that the related tax benefit will be realized.
Deferred income tax assets and liabilities are measured using tax rates and tax laws that have been enacted or substantively enacted by the Balance Sheet date and are expected to apply to taxable income in the years in which those temporary differences are expected to be recovered or settled. The effect of changes in tax rates on deferred income tax assets and liabilities is recognized as income or expense in the period that includes the enactment or the substantive enactment date. A deferred income tax asset is recognized to the extent that it is probable that future taxable profit will be available against which the deductible temporary differences and tax losses can be utilized. Deferred income taxes are not provided on the undistributed earnings of subsidiaries and branches where it is expected that the earnings of the subsidiary or branch will not be distributed in the foreseeable future.
The Group offsets current tax assets and current tax liabilities; deferred tax assets and deferred tax liabilities, where it has a legally enforceable right to set off the recognized amounts and where it intends either to settle on a net basis, or to realize the asset and settle the liability simultaneously. Tax benefits of deductions earned on exercise of employee share options in excess of compensation charged to income are credited to equity.
Income tax expense in the Consolidated Statement of Profit and Loss comprises:
<table><thead><tr><th rowspan="2">Particulars</th><th colspan="2">Three months ended March 31,</th><th>Y ear ended March</th><th>(In % crore) 31,</th></tr><tr><th>2024</th><th>2023</th><th>2024</th><th>2023</th></tr></thead><tbody><tr><td>Current taxes</td><td>1,173</td><td>2,260</td><td>8,390</td><td>9,287</td></tr><tr><td>Deferred taxes</td><td>1,092</td><td>72</td><td>1,350</td><td>(73)</td></tr><tr><td>Income tax expense</td><td>2,265</td><td>2,332</td><td>9,740</td><td>9,214</td></tr></tbody></table>
Income tax expense for the three months ended March 31, 2024 and March 31, 2023 includes reversal (net of provisions) of ₹871 crore and ₹71 crore, respectively. Income tax expense for the year ended March 31, 2024 and March 31, 2023 includes reversal (net of provisions) of ₹937 crore and ₹106 crore, respectively. These reversals pertaining to prior periods are primarily on account of adjudication of certain disputed matters, upon filing of tax return and completion of assessments, across various jurisdictions.
During the quarter ending March 31, 2024, the Company received orders under sections 250 and 254 of the Income Tax Act, 1961, from the Income Tax Authorities in India for the assessment years, 2007-08 to 2015-16, 2017-18 and 2018-19. These orders confirmed the Company's position with respect to tax treatment of certain contentious matters. As a result interest income (pre-tax) of ₹1,933 crore was recognised and provision for income tax aggregating ₹525 crore was reversed with a corresponding credit to the Statement of Profit and Loss. Also, upon resolution of the disputes, an amount aggregating to ₹ 1,628 crore has been reduced from contingent liabilities.
Deferred income tax for three months and year ended March 31, 2024 and March 31, 2023 substantially relates to origination and reversal of temporary differences.
The Company’s Advanced Pricing Arrangement (APA) with the Internal Revenue Service (IRS) for US branch income tax expired in March 2021. The Company has applied for renewal of APA and currently the US taxable income is based on the Company’s best estimate determined based on the expected value method.
27
2.16 REVENUE FROM OPERATIONS
Accounting policy
The Group derives revenues primarily from IT services comprising software development and related services, cloud and infrastructure services, maintenance, consulting and package implementation, licensing of software products and platforms across the Group’s core and digital offerings (together called as “software related services”) and business process management services. Contracts with customers are either on a time-and-material, unit of work, fixed-price or on a fixed-timeframe basis.
Revenues from customer contracts are considered for recognition and measurement when the contract has been approved in writing by the parties, to the contract, the parties to contract are committed to perform their respective obligations under the contract, and the contract is legally enforceable. Revenue is recognized upon transfer of control of promised products or services (“performance obligations”) to customers in an amount that reflects the consideration the Group has received or expects to receive in exchange for these products or services (“transaction price”). When there is uncertainty as to collectability, revenue recognition is postponed until such uncertainty is resolved.
The Group assesses the services promised in a contract and identifies distinct performance obligations in the contract. The Group allocates the transaction price to each distinct performance obligation based on the relative standalone selling price. The price that is regularly charged for an item when sold separately is the best evidence of its standalone selling price. In the absence of such evidence, the primary method used to estimate standalone selling price is the expected cost plus a margin, under which the Group estimates the cost of satisfying the performance obligation and then adds an appropriate margin based on similar services.
The Group’s contracts may include variable consideration including rebates, volume discounts and penalties. The Group includes variable consideration as part of transaction price when there is a basis to reasonably estimate the amount of the variable consideration and when it is probable that a significant reversal of cumulative revenue recognized will not occur when the uncertainty associated with the variable consideration is resolved.
Revenue on time-and-material and unit of work based contracts, are recognized as the related services are performed. Fixed price maintenance revenue is recognized ratably either on a straight-line basis when services are performed through an indefinite number of repetitive acts over a specified period or ratably using a percentage of completion method when the pattern of benefits from the services rendered to the customer and the Group’s costs to fulfil the contract is not even through the period of contract because the services are generally discrete in nature and not repetitive. Revenue from other fixed-price, fixed-timeframe contracts, where the performance obligations are satisfied over time is recognized using the percentage-of-completion method. Efforts or costs expended are used to determine progress towards completion as there is a direct relationship between input and productivity. Progress towards completion is measured as the ratio of costs or efforts incurred to date (representing work performed) to the estimated total costs or efforts. Estimates of transaction price and total costs or efforts are continuously monitored over the term of the contracts and are recognized in net profit in the period when these estimates change or when the estimates are revised. Revenues and the estimated total costs or efforts are subject to revision as the contract progresses. Provisions for estimated losses, if any, on incomplete contracts are recorded in the period in which such losses become probable based on the estimated efforts or costs to complete the contract.
The billing schedules agreed with customers include periodic performance based billing and / or milestone based progress billings. Revenues in excess of billing are classified as unbilled revenue while billing in excess of revenues are classified as contract liabilities (which we refer to as unearned revenues).
In arrangements for software development and related services and maintenance services, by applying the revenue recognition criteria for each distinct performance obligation, the arrangements with customers generally meet the criteria for considering software development and related services as distinct performance obligations. For allocating the transaction price, the Group measures the revenue in respect of each performance obligation of a contract at its relative standalone selling price. The price that is regularly charged for an item when sold separately is the best evidence of its standalone selling price. In cases where the Group is unable to determine the standalone selling price, the Group uses the expected cost plus margin approach in estimating the standalone selling price. For software development and related services, the performance obligations are satisfied as and when the services are rendered since the customer generally obtains control of the work as it progresses.
Certain cloud and infrastructure services contracts include multiple elements which may be subject to other specific accounting guidance, such as leasing guidance. These contracts are accounted in accordance with such specific accounting guidance. In such arrangements where the Group is able to determine that hardware and services are distinct performance obligations, it allocates the consideration to these performance obligations on a relative standalone selling price basis. In the absence of standalone selling price, the Group uses the expected cost-plus margin approach in estimating the standalone selling price. When such arrangements are considered as a single performance obligation, revenue is recognized over the period and measure of progress is determined based on promise in the contract.
Revenue from licenses where the customer obtains a “right to use” the licenses is recognized at the time the license are made available to the customer. Revenue from licenses where the customer obtains a “right to access” is recognized over the access period.
Arrangements to deliver software products generally have three elements: license, implementation and Annual Technical Services (ATS).When implementation services are provided in conjunction with the licensing arrangement and the license and implementation have been identified as two distinct separate performance obligations, the transaction price for such contracts are allocated to each performance obligation of the contract based on their relative standalone selling prices. In the absence of standalone selling price for implementation, the Group uses the expected cost plus margin approach in estimating the standalone selling price. Where the license is required to be substantially customized as part of the implementation service the entire arrangement fee for license and implementation is considered to be a single performance obligation and the revenue is recognized using the percentage-of-completion method as the implementation is performed. Revenue from client training, support and other services arising due to the sale of software products is recognized as the performance obligations are satisfied. ATS revenue is recognized ratably on a straight line basis over the period in which the services are rendered.
Contracts with customers includes subcontractor services or third-party vendor equipment or software in certain integrated services arrangements. In these types of arrangements, revenue from sales of third-party vendor products or services is recorded net of costs when the Group is acting as an agent between the customer and the vendor, and gross when the Group is the principal for the transaction. In doing so, the Group first evaluates whether it obtains control of the specified goods or services before they are transferred to the customer. The Group considers whether it is primarily responsible for fulfilling the promise to provide the specified goods or services, inventory risk, pricing discretion and other factors to determine whether it controls the specified goods or services and therefore, is acting as a principal or an agent.
A contract modification is a change in the scope or price or both of a contract that is approved by the parties to the contract. A contract modification that results in the addition of distinct performance obligations are accounted for either as a separate contract if the additional services are priced at the standalone selling price or as a termination of the existing contract and creation of a new contract if they are not priced at the standalone selling price. If the modification does not result in a distinct performance obligation, it is accounted for as part of the existing contract on a cumulative catch-up basis.
28
The incremental costs of obtaining a contract (i.e., costs that would not have been incurred if the contract had not been obtained) are recognized as an asset if the Group expects to recover them.
Certain eligible, nonrecurring costs (e.g. set-up or transition or transformation costs) that do not represent a separate performance obligation are recognized as an asset when such costs (a) relate directly to the contract; (b) generate or enhance resources of the Group that will be used in satisfying the performance obligation in the future; and (c) are expected to be recovered.
Capitalized contract costs relating to upfront payments to customers are amortized to revenue and other capitalized costs are amortized to expenses over the respective contract life on a systematic basis consistent with the transfer of goods or services to customer to which the asset relates. Capitalized costs are monitored regularly for impairment. Impairment losses are recorded when present value of projected remaining operating cash flows is not sufficient to recover the carrying amount of the capitalized costs.
The Group presents revenues net of indirect taxes in its Consolidated Statement of Profit and Loss.
Revenue from operation for the three months and year ended March 31, 2024 and March 31, 2023 are as follows:
<table><thead><tr><th rowspan="3">Particulars</th><th colspan="2" rowspan="2">Three months ended March 31,</th><th>Y ear ended</th><th>(In % crore)</th></tr><tr><th colspan="2">March 31,</th></tr><tr><th>2024</th><th>2023</th><th>2024</th><th>2023</th></tr></thead><tbody><tr><td>Revenue from software services</td><td>36,064</td><td>35,199</td><td>145,285</td><td>137,575</td></tr><tr><td>Revenue from products and platforms</td><td>1,859</td><td>2,242</td><td>8,385</td><td>9,192</td></tr><tr><td>Total revenue from operations</td><td>37,923</td><td>37,441</td><td>153,670</td><td>146,767</td></tr></tbody></table>
Products & platforms
The Group also derives revenues from the sale of products and platforms including Finacle – core banking solution, Edge Suite of products, Panaya platform, Infosys Equinox, Infosys Helix, Infosys Applied AI, Infosys Cortex, Stater digital platform and Infosys McCamish – insurance platform.
Disaggregated revenue information
Revenue disaggregation by business segments has been included in segment information (Refer to Note 2.23) . The table below presents disaggregated revenues from contracts with customers by geography and contract type. The Group believes that this disaggregation best depicts how the nature, amount, timing and uncertainty of revenues and cash flows are affected by industry, market and other economic factors.
For the three months and year ended March 31, 2024 and March 31, 2023:
<table><thead><tr><th rowspan="2">Particulars</th><th colspan="2">Three months ended March 31,</th><th colspan="2">(In X crore) Y ear ended March 31,</th></tr><tr><th>2024</th><th>2023</th><th>2024</th><th>2023</th></tr></thead><tbody><tr><td colspan="5">Revenues by G eography*</td></tr><tr><td>North America</td><td>22,606</td><td>22,842</td><td>92,411</td><td>90,724</td></tr><tr><td>Europe</td><td>10,861</td><td>10,088</td><td>42,267</td><td>37,675</td></tr><tr><td>India</td><td>833</td><td>981</td><td>3,881</td><td>3,861</td></tr><tr><td>Rest of the world</td><td>3,623</td><td>3,530</td><td>15,111</td><td>14,507</td></tr><tr><td>Total</td><td>37,923</td><td>37,441</td><td>153,670</td><td>146,767</td></tr></tbody></table>
Geographical revenues is based on the domicile of customer.
The percentage of revenue from fixed-price contracts for the three months ended March 31, 2024 and March 31, 2023 is 54% and 52%, respectively. The percentage of revenue from fixed-price contracts for the year ended March 31, 2024 and March 31, 2023 is 53% and 52%, respectively.
Trade Receivables and Contract Balances
The timing of revenue recognition, billings and cash collections results in receivables, unbilled revenue, and unearned revenue on the Group’s Consolidated Balance Sheet. Amounts are billed as work progresses in accordance with agreed-upon contractual terms, either at periodic intervals (e.g., monthly or quarterly) or upon achievement of contractual milestones.
The Group’s receivables are rights to consideration that are unconditional. Unbilled revenues comprising revenues in excess of billings from time and material contracts and fixed price maintenance contracts are classified as financial asset when the right to consideration is unconditional and is due only after a passage of time.
Invoicing to the clients for other fixed price contracts is based on milestones as defined in the contract and therefore the timing of revenue recognition is different from the timing of invoicing to the customers. Therefore unbilled revenues for other fixed price contracts (contract asset) are classified as non-financial asset because the right to consideration is dependent on completion of contractual milestones.
Invoicing in excess of earnings are classified as unearned revenue.
Trade receivables and unbilled revenues are presented net of impairment in the consolidated Balance Sheet.
X
29
2.17 OTHER INCOME, NET
Accounting policy
Other income is comprised primarily of interest income, dividend income, gain/loss on investment and exchange gain/loss on forward and options contracts and on translation of foreign currency assets and liabilities. Interest income is recognized using the effective interest method. Dividend income is recognized when the right to receive payment is established.
Foreign currency
Accounting policy
Functional currency
The functional currency of Infosys, Infosys BPM, EdgeVerve, Skava, Infosys Green Forum, Danske IT and controlled trusts is the Indian rupee. The functional currencies for foreign subsidiaries are their respective local currencies. These financial statements are presented in Indian rupees (rounded off to crore; one crore equals ten million).
Transactions and translations
Foreign-currency denominated monetary assets and liabilities are translated into the relevant functional currency at exchange rates in effect at the Balance Sheet date. The gains or losses resulting from such translations are recognized in the Condensed Consolidated Statement of Profit and Loss and reported within exchange gains/ (losses) on translation of assets and liabilities, net, except when deferred in Other Comprehensive Income as qualifying cash flow hedges. Non-monetary assets and non-monetary liabilities denominated in a foreign currency and measured at fair value are translated at the exchange rate prevalent at the date when the fair value was determined. Non-monetary assets and non-monetary liabilities denominated in a foreign currency and measured at historical cost are translated at the exchange rate prevalent at the date of transaction. The related revenue and expense are recognized using the same exchange rate.
Transaction gains or losses realized upon settlement of foreign currency transactions are included in determining net profit for the period in which the transaction is settled. Revenue, expense and cash-flow items denominated in foreign currencies are translated into the relevant functional currencies using the exchange rate in effect on the date of the transaction.
The translation of financial statements of the foreign subsidiaries to the presentation currency is performed for assets and liabilities using the exchange rate in effect at the Balance Sheet date and for revenue, expense and cash-flow items using the average exchange rate for the respective periods. The gains or losses resulting from such translation are included in currency translation reserves under other components of equity. When a subsidiary is disposed off, in full, the relevant amount is transferred to net profit in the Condensed Consolidated Statement of Profit and Loss. However when a change in the parent's ownership does not result in loss of control of a subsidiary, such changes are recorded through equity.
Other Comprehensive Income, net of taxes includes translation differences on non-monetary financial assets measured at fair value at the reporting date, such as equities classified as financial instruments and measured at fair value through other comprehensive income (FVOCI).
Goodwill and fair value adjustments arising on the acquisition of a foreign entity are treated as assets and liabilities of the foreign entity and translated at the exchange rate in effect at the Balance Sheet date.
Government grant
The Group recognizes government grants only when there is reasonable assurance that the conditions attached to them shall be complied with, and the grants will be received. Government grants related to assets are treated as deferred income and are recognized in net profit in the Consolidated Statement of Profit and Loss on a systematic and rational basis over the useful life of the asset. Government grants related to revenue are recognized on a systematic basis in net profit in the Consolidated Statement of Profit and Loss over the periods necessary to match them with the related costs which they are intended to compensate.
Other income for the three months and year ended March 31, 2024 and March 31, 2023 is as follows:
<table><thead><tr><th rowspan="2">Particulars</th><th colspan="2">Three months ended March 31,</th><th colspan="2">Y ear ended March 31,</th></tr><tr><th>2024</th><th>2023</th><th>2024</th><th>2023</th></tr></thead><tbody><tr><td colspan="5">Interest income on financial assets carried at amortized cost</td></tr><tr><td>Tax free bonds and Government bonds</td><td>31</td><td>36</td><td>131</td><td>149</td></tr><tr><td>Deposit with Bank and others</td><td>222</td><td>161</td><td>929</td><td>712</td></tr><tr><td colspan="5">Interest income on financial assets carried at fair value through other comprehensive income</td></tr><tr><td>Non-convertible debentures, commercial papers, certificates of deposit and government securities</td><td>318</td><td>231</td><td>1,007</td><td>955</td></tr><tr><td colspan="5">Income on investments carried at fair value through profit or loss</td></tr><tr><td>Dividend income on liquid mutual funds</td><td>—                          —                               —</td><td></td><td></td><td>—</td></tr><tr><td>Gain / (loss) on liquid mutual funds and other investments</td><td>88</td><td>61</td><td>285</td><td>148</td></tr><tr><td>Income on investments carried at fair value through other comprehensive income</td><td>—                          —                               —</td><td></td><td></td><td>1</td></tr><tr><td>Interest on income tax refund</td><td>1,916</td><td></td><td>1,965</td><td>3</td></tr><tr><td>Exchange gains / (losses) on forward and options contracts</td><td>190</td><td>142</td><td>100</td><td>(647)</td></tr><tr><td>Exchange gains / (losses) on translation of other assets and liabilities</td><td>(123)</td><td>(91)</td><td>87</td><td>1,062</td></tr><tr><td>Miscellaneous income, net</td><td>87</td><td>129</td><td>207</td><td>318</td></tr><tr><td>Total other income</td><td>2,729</td><td>671</td><td>4,711</td><td>2,701</td></tr></tbody></table>
(In % crore)
30
2.18 EXPENSES
Accounting policy
Gratuity and Pensions
The Group provides for gratuity, a defined benefit retirement plan ('the Gratuity Plan') covering eligible employees majorly of Infosys and its Indian subsidiaries. The Gratuity Plan provides a lump-sum payment to vested employees at retirement, death, incapacitation or termination of employment, of an amount based on the respective employee's salary and the tenure of employment with the Group. The Company contributes Gratuity liabilities to the Infosys Limited Employees' Gratuity Fund Trust (the Trust). In case of Infosys BPM and EdgeVerve, contributions are made to the Infosys BPM Employees' Gratuity Fund Trust and EdgeVerve Systems Limited Employees' Gratuity Fund Trust, respectively. Trustees administer contributions made to the Trusts and contributions are invested in a scheme with the Life Insurance Corporation of India as permitted by Indian law.
The Group operates defined benefit pension plan in certain overseas jurisdictions, in accordance with the local laws. These plans are managed by third party fund managers. The plans provide for periodic payouts after retirement and/or for a lumpsum payment as set out in rules of each fund and includes death and disability benefits. The defined benefit plans require contributions which are based on a percentage of salary that varies depending on the age of the respective employees.
Liabilities with regard to these defined benefit plans are determined by actuarial valuation, performed by an external actuary, at each Balance Sheet date using the projected unit credit method. These defined benefit plans expose the Group to actuarial risks, such as longevity risk, interest rate risk and market risk.
The Group recognizes the net obligation of a defined benefit plan in its Balance Sheet as an asset or liability. Gains and losses through re-measurements of the net defined benefit liability / (asset) are recognized in other comprehensive income and are not reclassified to profit or loss in subsequent periods. The actual return of the portfolio of plan assets, in excess of the yields computed by applying the discount rate used to measure the defined benefit obligation is recognized in other comprehensive income. The effect of any plan amendments is recognized in net profit in the Consolidated Statement of Profit and Loss.
Provident fund
Eligible employees of Infosys receive benefits from a provident fund, which is a defined benefit plan. Both the eligible employee and the Company make monthly contributions to the provident fund plan equal to a specified percentage of the covered employee's salary. The Company contributes a portion to the Infosys Limited Employees' Provident Fund Trust. The trust invests in specific designated instruments as permitted by Indian law. The remaining portion is contributed to the government administered pension fund. The rate at which the annual interest is payable to the beneficiaries by the trust is being administered by the Government of India. The Company has an obligation to make good the shortfall, if any, between the return from the investments of the trust and the notified interest rate.
In respect of Indian subsidiaries, eligible employees receive benefits from a provident fund, which is a defined contribution plan. Both the eligible employee and the respective companies make monthly contributions to this provident fund plan equal to a specified percentage of the covered employee's salary. Amounts collected under the provident fund plan are deposited in a government administered provident fund. The Companies have no further obligation to the plan beyond its monthly contributions.
Superannuation
Certain employees of Infosys, Infosys BPM and EdgeVerve are participants in a defined contribution plan. The Group has no further obligations to the plan beyond its monthly contributions which are periodically contributed to a trust fund, the corpus of which is invested with the Life Insurance Corporation of India.
Compensated absences
The Group has a policy on compensated absences which are both accumulating and non-accumulating in nature. The expected cost of accumulating compensated absences is determined by actuarial valuation performed by an external actuary at each Balance Sheet date using projected unit credit method on the additional amount expected to be paid/availed as a result of the unused entitlement that has accumulated at the Balance Sheet date. Expense on non- accumulating compensated absences is recognized in the period in which the absences occur.
31
<table><thead><tr><th rowspan="2">Particulars</th><th colspan="2">Three months ended March 31,</th><th colspan="2">Y ear ended March 31,</th></tr><tr><th>2024</th><th>2023</th><th>2024</th><th>2023</th></tr></thead><tbody><tr><td colspan="5">Employee benefit expenses</td></tr><tr><td>Salaries including bonus</td><td>19,527</td><td>19,526</td><td>79,315</td><td>75,239</td></tr><tr><td>Contribution to provident and other funds</td><td>529</td><td>547</td><td>2,213</td><td>2,143</td></tr><tr><td>Share based payments to employees (Refer to Note 2.11)</td><td>225</td><td>133</td><td>652</td><td>519</td></tr><tr><td>Staff welfare</td><td>112</td><td>105</td><td>440</td><td>458</td></tr><tr><td></td><td>20,393</td><td>20,311</td><td>82,620</td><td>78,359</td></tr><tr><td colspan="5">Cost of software packages and others</td></tr><tr><td>For own use</td><td>555</td><td>496</td><td>2,145</td><td>1,937</td></tr><tr><td>Third party items bought for service delivery to clients</td><td>3,132</td><td>2,390</td><td>11,370</td><td>8,965</td></tr><tr><td></td><td>3,687</td><td>2,886</td><td>13,515</td><td>10,902</td></tr><tr><td colspan="5">Other expenses</td></tr><tr><td>Repairs and maintenance</td><td>316</td><td>331</td><td>1,278</td><td>1,208</td></tr><tr><td>Power and fuel</td><td>49</td><td>46</td><td>199</td><td>176</td></tr><tr><td>Brand and marketing</td><td>285</td><td>265</td><td>1,007</td><td>905</td></tr><tr><td>Rates and taxes</td><td>84</td><td>78</td><td>326</td><td>299</td></tr><tr><td>Consumables</td><td>47</td><td>41</td><td>170</td><td>158</td></tr><tr><td>nsurance</td><td>53</td><td>43</td><td>210</td><td>174</td></tr><tr><td>Provision for post-sales client support and others</td><td>(129)</td><td>(80)</td><td>75</td><td>120</td></tr><tr><td>Commission to non-whole time directors</td><td></td><td></td><td>16</td><td>15</td></tr><tr><td>mpairment loss recognized / (reversed) under expected credit loss model</td><td>(98)</td><td>86</td><td>121</td><td>283</td></tr><tr><td>Contributions towards Corporate Social Responsibility</td><td>182</td><td>151</td><td>533</td><td>471</td></tr><tr><td>Others</td><td>191</td><td>181</td><td>781</td><td>583</td></tr><tr><td></td><td>985</td><td>1,146</td><td>4,716</td><td>4,392</td></tr></tbody></table>
(In % crore)
2.19 Leases
Accounting Policy
The Group as a lessee
The Group’s lease asset classes consist of leases for land, buildings and computers. The Group assesses whether a contract contains a lease, at inception of a contract. A contract is, or contains, a lease if the contract conveys the right to control the use of an identified asset for a period of time in exchange for consideration. To assess whether a contract conveys the right to control the use of an identified asset, the group assesses whether: (1) the contract involves the use of an identified asset (2) the Group has substantially all of the economic benefits from use of the asset through the period of the lease and (3) the Group has the right to direct the use of the asset.
At the date of commencement of the lease, the Group recognizes a right-of-use asset (“ROU”) and a corresponding lease liability for all lease arrangements in which it is a lessee, except for leases with a term of twelve months or less (short-term leases) and low value leases. For these short-term and low value leases, the Group recognizes the lease payments as an operating expense on a straight-line basis over the term of the lease.
As a lessee, the Group determines the lease term as the non-cancellable period of a lease adjusted with any option to extend or terminate the lease, if the use of such option is reasonably certain. The Group makes an assessment on the expected lease term on a lease-by-lease basis and thereby assesses whether it is reasonably certain that any options to extend or terminate the contract will be exercised. In evaluating the lease term, the Group considers factors such as any significant leasehold improvements undertaken over the lease term, costs relating to the termination of the lease and the importance of the underlying asset to Group’s operations taking into account the location of the underlying asset and the availability of suitable alternatives. The lease term in future periods is reassessed to ensure that the lease term reflects the current economic circumstances.
Certain lease arrangements includes the options to extend or terminate the lease before the end of the lease term. ROU assets and lease liabilities includes these options when it is reasonably certain that they will be exercised.
The right-of-use assets are initially recognized at cost, which comprises the initial amount of the lease liability adjusted for any lease payments made at or prior to the commencement date of the lease plus any initial direct costs less any lease incentives. They are subsequently measured at cost less accumulated depreciation and impairment losses.
Right-of-use assets are depreciated from the commencement date on a straight-line basis over the shorter of the lease term and useful life of the underlying asset.
Right-of-use assets are evaluated for recoverability whenever events or changes in circumstances indicate that their carrying amounts may not be recoverable. For the purpose of impairment testing, the recoverable amount (i.e. the higher of the fair value less cost to sell and the value-in-use) is determined on an individual asset basis unless the asset does not generate cash flows that are largely independent of those from other assets. In such cases, the recoverable amount is determined for the Cash Generating Unit (CGU) to which the asset belongs.
The lease liability is initially measured at amortized cost at the present value of the future lease payments. The lease payments are discounted using the interest rate implicit in the lease or, if not readily determinable, using the incremental borrowing rates in the country of domicile of these leases. Lease liabilities are remeasured with a corresponding adjustment to the related right-of-use asset if the Group changes its assessment if whether it will exercise an extension or a termination option.
Lease liability and ROU asset have been separately presented in the Balance Sheet and lease payments have been classified as financing cash flows.
32
The Group as a lessor
Leases for which the Group is a lessor is classified as a finance or operating lease. Whenever the terms of the lease transfer substantially all the risks and rewards of ownership to the lessee, the contract is classified as a finance lease. All other leases are classified as operating leases.
When the Group is an intermediate lessor, it accounts for its interests in the head lease and the sublease separately. The sublease is classified as a finance or operating lease by reference to the right-of-use asset arising from the head lease.
For operating leases, rental income is recognized on a straight line basis over the term of the relevant lease.
Following are the changes in the carrying value of right-of-use assets for the three months ended March 31, 2024:
<table><thead><tr><th rowspan="2">Particulars</th><th colspan="4">Category of ROU asset</th><th rowspan="2">Total</th></tr><tr><th>Land</th><th>Buildings</th><th>V ehicles</th><th>Computers</th></tr></thead><tbody><tr><td>Balance as of J anuary 1, 2024</td><td>607</td><td>3,527</td><td>18</td><td>2,740</td><td>6,892</td></tr><tr><td>Additions*</td><td></td><td>61</td><td></td><td>376</td><td>439</td></tr><tr><td>Deletions</td><td></td><td>(92)</td><td></td><td>(215)</td><td>(307)</td></tr><tr><td>Impairment#</td><td></td><td>_</td><td></td><td>—</td><td>—</td></tr><tr><td>Depreciation</td><td>()</td><td>(185)</td><td>()</td><td>(234)</td><td>(423)</td></tr><tr><td>Translation difference</td><td></td><td>(13)</td><td>(©)</td><td>(35)</td><td>(49)</td></tr><tr><td>Balance as of March 31, 2024</td><td>605</td><td>3,298</td><td>17</td><td>2,632</td><td>6,552</td></tr></tbody></table>
* Net of adjustments on account of modifications
# included under other expenses. Refer note 2.18
Following are the changes in the carrying value of right-of-use assets for the three months ended March 31, 2023:
<table><thead><tr><th rowspan="2">Particulars</th><th colspan="4">Category of ROU asset</th><th rowspan="2">(In X crore) Total</th></tr><tr><th>Land</th><th>Buildings</th><th>Vehicles</th><th>Computers</th></tr></thead><tbody><tr><td>Balance as of J anuary 1, 2023</td><td>624</td><td>3,847</td><td>15</td><td>1,994</td><td>6,480</td></tr><tr><td>Additions*</td><td></td><td>228</td><td>2</td><td>651</td><td>881</td></tr><tr><td>Deletions</td><td></td><td>(33)</td><td>\</td><td>(124)</td><td>(157)</td></tr><tr><td>Depreciation</td><td></td><td>(171)</td><td>= w &lt;</td><td>(179)</td><td>(355)</td></tr><tr><td>Translation difference</td><td></td><td>25</td><td>—_</td><td>6</td><td>33</td></tr><tr><td>Balance as of March 31, 2023</td><td>623</td><td>3,896</td><td>15</td><td>2,348</td><td>6,882</td></tr></tbody></table>
* Net of adjustments on account of modifications and lease incentives
Following are the changes in the carrying value of right-of-use assets for the year ended March 31, 2024:
<table><thead><tr><th rowspan="2">Particulars</th><th colspan="4">Category of ROU asset</th><th rowspan="2">Total</th></tr><tr><th>Land</th><th>Buildings</th><th>V ehicles</th><th>Computers</th></tr></thead><tbody><tr><td>Balance as of April 1, 2023</td><td>623</td><td>3,896</td><td>15</td><td>2,348</td><td>6,882</td></tr><tr><td>Additions*</td><td></td><td>394</td><td>12</td><td>1,872</td><td>2,278</td></tr><tr><td>Deletions</td><td>(10)</td><td>(181)</td><td>1</td><td>(755)</td><td>(947)</td></tr><tr><td>Impairment *</td><td></td><td>(88)</td><td>—</td><td>—</td><td>(88)</td></tr><tr><td>Depreciation</td><td>(6)</td><td>(728)</td><td>(10)</td><td>(851)</td><td>(1,595)</td></tr><tr><td>Translation difference</td><td>2</td><td>5</td><td>1</td><td>18</td><td>22</td></tr><tr><td>Balance as of March 31, 2024</td><td>605</td><td>3,298</td><td>17</td><td>2,632</td><td>6,552</td></tr></tbody></table>
* Net of adjustments on account of modifications and lease incentives
# included under other expenses. Refer note 2.18
Following are the changes in the carrying value of right-of-use assets for the year ended March 31, 2023:
<table><thead><tr><th rowspan="2">Particulars</th><th colspan="4">Category of ROU asset</th><th rowspan="2">(In R crore) Total</th></tr><tr><th>Land</th><th>Buildings</th><th>Vehicles</th><th>Computers</th></tr></thead><tbody><tr><td>Balance as of April 1, 2022</td><td>628</td><td>3,711</td><td>16</td><td>468</td><td>4,823</td></tr><tr><td>Additions*</td><td></td><td>847</td><td>8</td><td>2,646</td><td>3,501</td></tr><tr><td>Deletions</td><td></td><td>(45)</td><td>—</td><td>(364)</td><td>(409)</td></tr><tr><td>Depreciation</td><td></td><td>(671)</td><td>(10)</td><td>(499)</td><td>(1,186)</td></tr><tr><td>Translation difference</td><td></td><td>54</td><td>1</td><td>97</td><td>153</td></tr><tr><td>Balance as of March 31, 2023</td><td>623</td><td>3,896</td><td>15</td><td>2,348</td><td>6,882</td></tr></tbody></table>
* Net of adjustments on account of modifications and lease incentives
The aggregate depreciation expense on ROU assets is included under depreciation and amortization expense in the interim condensed Consolidated Statement of Profit and Loss.
The following is the break-up of current and non-current lease liabilities as at March 31, 2024 and March 31, 2023:
<table><thead><tr><th rowspan="2">Particulars</th><th colspan="2">Asat</th></tr><tr><th>March 31, 2024</th><th>March 31, 2023</th></tr></thead><tbody><tr><td>Current lease liabilities</td><td>1,959</td><td>1,242</td></tr><tr><td>Non-current lease liabilities</td><td>6,400</td><td>7,057</td></tr><tr><td>Total</td><td>8,359</td><td>8,299</td></tr></tbody></table>
(In % crore)
(In X crore)
(InR crore)
(InR crore)
(In % crore)
33
2.20 BASIC AND DILUTED SHARES USED IN COMPUTING EARNINGS PER EQUITY SHARE
Accounting policy
Basic earnings per equity share is computed by dividing the net profit attributable to the equity holders of the Group by the weighted average number of equity shares outstanding during the period. Diluted earnings per equity share is computed by dividing the net profit attributable to the equity holders of the Group by the weighted average number of equity shares considered for deriving basic earnings per equity share and also the weighted average number of equity shares that could have been issued upon conversion of all dilutive potential equity shares. The dilutive potential equity shares are adjusted for the proceeds receivable had the equity shares been actually issued at fair value (i.e. the average market value of the outstanding equity shares). Dilutive potential equity shares are deemed converted as at the beginning of the period, unless issued at a later date. Dilutive potential equity shares are determined independently for each period presented.
The number of equity shares and potentially dilutive equity shares are adjusted retrospectively for all periods presented for any share splits and bonus shares issues including for changes effected prior to the approval of the financial statements by the Board of Directors.
2.21 CONTINGENT LIABILITIES AND COMMITMENTS
Accounting policy
Contingent liability is a possible obligation arising from past events and whose existence will be confirmed only by the occurrence or non-occurrence of one or more uncertain future events not wholly within the control of the entity or a present obligation that arises from past events but is not recognized because it is not probable that an outflow of resources embodying economic benefits will be required to settle the obligation or the amount of the obligation cannot be measured with sufficient reliability.
2.21.1 Contingent liability
<table><thead><tr><th rowspan="2">Particulars</th><th>Asat March 31, 2024</th><th>(In % crore) March 31, 2023</th></tr><tr><th></th><th></th></tr></thead><tbody><tr><td colspan="3">Contingent liabilities :</td></tr><tr><td>Claims against the Group, not acknowledged as debts</td><td>3,583</td><td>4,762</td></tr><tr><td>[Amount paid to statutory authorities 28,754 crore (36,539 crore) |</td><td></td><td></td></tr></tbody></table>
(1) As at March 31, 2024 and March 31, 2023, claims against the Group not acknowledged as debts in respect of income tax matters amounted to ₹2,794 crore and ₹4,062 crore, respectively.
The claims against the Group primarily represent demands arising on completion of assessment proceedings under the Income Tax Act, 1961. These claims are on account of issues of disallowance of expenditure towards software being held as capital in nature, payments made to Associated Enterprises held as liable for withholding of taxes, among others. These matters are pending before various Income Tax Authorities and the Management including its tax advisors expect that its position will likely be upheld on ultimate resolution and will not have a material adverse effect on the Group's financial position and results of operations.
Amount paid to statutory authorities against the tax claims amounted to ₹8,743 crore and ₹6,528 crore as at March 31, 2024 and March 31, 2023, respectively.
34
2.21.2 McCamish Cybersecurity incident
In November 2023, Infosys McCamish Systems (McCamish), a step-down subsidiary of Infosys Limited, experienced a cybersecurity incident resulting in the non-availability of certain applications and systems. McCamish initiated its incident response and engaged cybersecurity and other specialists to assist in its investigation of and response to the incident and remediation and restoration of impacted applications and systems. By December 31, 2023, McCamish, with external specialists’ assistance, substantially remediated and restored the affected applications and systems.
Loss of contracted revenues and costs incurred with respect to remediations, restoration, communication efforts, investigative processes and analysis, legal services and others amounted to $38 million (approximately ₹316 crore).
Actions taken by McCamish included investigative analysis conducted by a third-party cybersecurity firm to determine, among other things, whether and the extent to which company or customer data was subject to unauthorized access or exfiltration. McCamish also engaged a third-party eDiscovery vendor in assessing the extent and nature of such data. McCamish in coordination with its third-party eDiscovery vendor has identified corporate customers and individuals whose information was subject to unauthorized access and exfiltration. McCamish’s review process is ongoing. McCamish may incur additional costs including indemnities or damages/claims, which are indeterminable at this time.
On March 6, 2024, a class action complaint was filed in the U.S. District Court for the Northern District of Georgia against McCamish . The complaint arises out of the cybersecurity incident at McCamish initially disclosed on November 3, 2023. The complaint was purportedly filed on behalf of all individuals within the United States whose personally identifiable information was exposed to unauthorized third parties as a result of the incident.
2.21.3 Legal Proceedings
Apart from this, the Group is subject to legal proceedings and claims, which have arisen in the ordinary course of business. The Group’s management reasonably expects that such ordinary course legal actions, when ultimately concluded and determined, will not have a material and adverse effect on the Group’s results of operations or financial condition.
2.21.4 Commitments
<table><thead><tr><th>Particulars</th><th>Asat March 2024</th><th>(In % crore) March 2023</th></tr><tr><th></th><th>31,</th><th>31,</th></tr></thead><tbody><tr><td>Estimated amount of contracts remaining to be executed on capital contracts and not provided for (net of advances and deposits)"”</td><td>780</td><td>959</td></tr><tr><td>Other commitments*</td><td>79</td><td>92</td></tr></tbody></table>
(1) Capital contracts primarily comprises of commitments for infrastructure facilities and computer equipment.
* Uncalled capital pertaining to investments
35
2.22 RELATED PARTY TRANSACTIONS
Refer to the Company's Annual Report for the year ended March 31, 2023 for the full names and other details of the Company's subsidiaries and controlled trusts.
Changes in Subsidiaries
During the year ended March 31, 2024, the following are the changes in the subsidiaries.
Infosys Americas Inc., (Infosys Americas) a Wholly-owned subsidiary of Infosys Limited is liquidated effective July 14, 2023.
oddity GmbH renamed as WongDoody GmbH
- (formerly known as oddity GmbH ). On September 29, 2023, oddity space GmbH, oddity waves GmbH, oddity jungle GmbH, oddity group services GmbH and oddity code GmbH merged into WongDoody GmbH and oddity code d.o.o which was formerly a subsidiary of oddity code Gmbh has become a subsidiary of Wongdoody Gmbh (formerly known as oddity GmbH).
- On September 1, 2023 Infosys Ltd. acquired 100% of voting interests in Danske IT and Support Services India Private Limited (“Danske IT”). Danske IT renamed as Idunn Information Technology Private Limited from April 1, 2024.
Infosys BPM Canada Inc, a Wholly-owned subsidiary of Infosys BPM Limited was incorporated on August 11, 2023.

Kaleidoscope Prototyping LLC, a Wholly-owned subsidiary of Kaleidoscope Animations is liquidated effective November 1, 2023.
- oddity Code d.o.o renamed as WongDoody d.o.o
- On November 24, 2023 Stater Participations B.V (Wholly-owned subsidiary of Stater N.V) merged with Stater N.V and Stater Belgium N.V./S.A which was formerly a wholly owned subsidiary of Stater Participations B.V. became a wholly owned subsidiary of Stater N.V.

On March 15, 2024, Infosys BPM Canada Inc, a Wholly-owned subsidiary of Infosys BPM Limited was dissolved.
oddity Limited (Taipei) renamed as WongDoody limited (Taipei) and oddity (Shanghai) Co., Ltd. renamed as WongDoody (Shanghai) Co. - Limited.
Changes in key management personnel
The following are the changes in the key management personnel:
Non-whole-time Directors
- Uri Levine (retired as independent director effective April 19, 2023)
- Helene Auriol Potier (appointed as independent director effective May 26, 2023)
- Nitin Paranjpe (appointed as an additional and independent director effective January 1, 2024)
Executive Officers:
- Mohit Joshi (resigned as President effective March 11, 2023 and was on leave till June 9, 2023 which was his last date with the company)
- Nilanjan Roy (resigned as Chief Financial Officer of the Company effective March 31, 2024)
- Jayesh Sanghrajka (appointed as Chief Financial Officer effective April 1, 2024)
Transaction with key management personnel:
The table below describes the compensation to key management personnel which comprise directors and executive officers:
<table><thead><tr><th rowspan="2">Particulars</th><th colspan="2">Three months ended March 31,</th><th colspan="2">(In X crore) Y ear ended March 31,</th></tr><tr><th>2024</th><th>2023</th><th>2024</th><th>2023</th></tr></thead><tbody><tr><td>Salaries and other short term employee benefits to whole-time directors (B[] and executive officers</td><td>30</td><td>25</td><td>113</td><td>111</td></tr><tr><td>Commission and other benefits to non-executive/independent directors</td><td>5</td><td>4</td><td>17</td><td>16</td></tr><tr><td>Total</td><td>35</td><td>29</td><td>130</td><td>127</td></tr></tbody></table>
(1) Total employee stock compensation expense for the three months ended March 31, 2024 and March 31, 2023 includes a charge of ₹17 crore and 8 crore, respectively, towards key management personnel. For the year ended March 31, 2024 and March 31, 2023 includes a charge of ₹68 crore and ₹49 crore, respectively, towards key management personnel (Refer to Note 2.11).
(2) Does not include post-employment benefits and other long-term benefits based on actuarial valuation as these are done for the Company as a whole.
(In X crore)
36
2.23 SEGMENT REPORTING
Ind AS 108, Operating segments, establishes standards for the way that public business enterprises report information about operating segments and related disclosures about products and services, geographic areas, and major customers. The Group's operations predominantly relate to providing end-to-end business solutions to enable clients to enhance business performance. The Chief Operating Decision Maker (CODM) evaluates the Group's performance and allocates resources based on an analysis of various performance indicators by business segments. Accordingly, information has been presented along business segments. The accounting principles used in the preparation of the financial statements are consistently applied to record revenue and expenditure in individual segments, and are as set out in the accounting policies.
Business segments of the Group are primarily enterprises in Financial Services and Insurance, enterprises in Manufacturing, enterprises in Retail, Consumer Packaged Goods and Logistics, enterprises in the Energy, Utilities, Resources and Services, enterprises in Communication, Telecom OEM and Media, enterprises in Hi-Tech, enterprises in Life Sciences and Healthcare and all other segments. The Financial services reportable segments has been aggregated to include the Financial Services operating segment and Finacle operating segment because of the similarity of the economic characteristics. All other segments represent the operating segments of businesses in India, Japan, China, Infosys Public Services & other enterprises in Public Services.
Revenue and identifiable operating expenses in relation to segments are categorized based on items that are individually identifiable to that segment. Revenue for 'all other segments' represents revenue generated by Infosys Public services and revenue generated from customers located in India, Japan and China and other enterprises in Public services. Allocated expenses of segments include expenses incurred for rendering services from the Group's offshore software development centers and on-site expenses, which are categorized in relation to the associated efforts of the segment. Certain expenses such as depreciation and amortization, which form a significant component of total expenses, are not specifically allocable to specific segments as the underlying assets are used interchangeably. The Management believes that it is not practical to provide segment disclosures relating to those costs and expenses, and accordingly these expenses are separately disclosed as "unallocated" and adjusted against the total income of the Group.
Assets and liabilities used in the Group's business are not identified to any of the reportable segments, as these are used interchangeably between segments. The Management believes that it is currently not practicable to provide segment disclosures relating to total assets and liabilities since a meaningful segregation of the available data is onerous.
Business segment revenue information is collated based on individual customers invoiced or in relation to which the revenue is otherwise recognized.
Disclosure of revenue by geographic locations is given in note 2.16 Revenue from operations.
Business Segments
Three months ended March 31, 2024 and March 31, 2023:
<table><thead><tr><th rowspan="2">Particulars</th><th rowspan="2">Financial 
 Services *</th><th rowspan="2">Retail ®</th><th rowspan="2">Communic ation (&amp;)</th><th rowspan="2">Energy, 
 Utilities, 
 Resources and Services</th><th></th><th></th><th></th><th></th><th rowspan="2">(In % crore) Total</th></tr><tr><th>Manufacturing</th><th>Hi-Tech</th><th>Life Sciences @</th><th>All other segments ©</th></tr></thead><tbody><tr><td rowspan="2">Revenue from operations</td><td>10,010</td><td>5,429</td><td>4,666</td><td>5,068</td><td>5,589</td><td>3,316</td><td>2,762</td><td>1,083</td><td>37,923</td></tr><tr><td>10,818</td><td>5,537</td><td>4,411</td><td>4,825</td><td>5,078</td><td>2,989</td><td>2,681</td><td>1,102</td><td>37,441</td></tr><tr><td rowspan="2">Identifiable operating expenses</td><td>6,042</td><td>2,591</td><td>3,033</td><td>2,717</td><td>3,656</td><td>1,995</td><td>1,639</td><td>652</td><td>22,325</td></tr><tr><td>6,161</td><td>2,869</td><td>2,613</td><td>2,614</td><td>3,248</td><td>1,734</td><td>1,514</td><td>701</td><td>21,454</td></tr><tr><td rowspan="2">Allocated expenses</td><td>2,027</td><td>974</td><td>823</td><td>920</td><td>852</td><td>518</td><td>491</td><td>209</td><td>6,814</td></tr><tr><td>2,057</td><td>1,034</td><td>840</td><td>909</td><td>928</td><td>505</td><td>462</td><td>254</td><td>6,989</td></tr><tr><td rowspan="2">Segment operating income</td><td>1,941</td><td>1,864</td><td>810</td><td>1,431</td><td>1,081</td><td>803</td><td>632</td><td>222</td><td>8,784</td></tr><tr><td>2,600</td><td>1,634</td><td>958</td><td>1,302</td><td>902</td><td>750</td><td>705</td><td>147</td><td>8,998</td></tr><tr><td rowspan="2">Unallocable expenses</td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td>1,163</td></tr><tr><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td>1,121</td></tr><tr><td rowspan="2">Other income, net (Refer to Note 2.17)</td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td>2,729</td></tr><tr><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td>671</td></tr><tr><td rowspan="2">Finance cost</td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td>110</td></tr><tr><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td>82</td></tr><tr><td rowspan="2">Profit before tax</td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td>10,240</td></tr><tr><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td>8,466</td></tr><tr><td rowspan="2">ncome tax expense</td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td>2,265</td></tr><tr><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td>2,332</td></tr><tr><td rowspan="2">Net Profit</td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td>7,975</td></tr><tr><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td>6,134</td></tr><tr><td rowspan="2">Depreciation and amortization</td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td>1,163</td></tr><tr><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td>1,121</td></tr><tr><td>Non-cash expenses other than depreciation</td><td>and amortization</td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td>—</td></tr></tbody></table>
37
Year ended March 31, 2024 and March 31, 2023:
<table><thead><tr><th rowspan="2">Particulars</th><th rowspan="2">Financial 
 * Services</th><th rowspan="2">Retail ®</th><th rowspan="2">Communic (&amp;) ation</th><th rowspan="2">Energy, 
 Utilities, 
 Resources and Services</th><th></th><th></th><th></th><th></th><th>(In % crore)</th></tr><tr><th>Manufacturing</th><th>Hi-Tech</th><th>Life @ Sciences</th><th>All other segments ©</th><th>Total</th></tr></thead><tbody><tr><td rowspan="2">Revenue from operations</td><td>42,158</td><td>22,504</td><td>17,991</td><td>20,035</td><td>22,298</td><td>12,411</td><td>11,515</td><td>4,758</td><td>153,670</td></tr><tr><td>43,763</td><td>21,204</td><td>18,086</td><td>18,539</td><td>19,035</td><td>11,867</td><td>10,085</td><td>4,188</td><td>146,767</td></tr><tr><td rowspan="2">Identifiable operating expenses</td><td>24,782</td><td>11,704</td><td>11,071</td><td>10,838</td><td>14,596</td><td>7,232</td><td>6,716</td><td>2,938</td><td>89,877</td></tr><tr><td>24,990</td><td>10,892</td><td>11,101</td><td>9,923</td><td>12,493</td><td>6,959</td><td>5,834</td><td>2,801</td><td>84,993</td></tr><tr><td rowspan="2">Allocated expenses</td><td>8,052</td><td>3,918</td><td>3,232</td><td>3,674</td><td>3,505</td><td>2,026</td><td>1,901</td><td>1,060</td><td>27,368</td></tr><tr><td>7,930</td><td>3,916</td><td>3,226</td><td>3,461</td><td>3,429</td><td>1,949</td><td>1,685</td><td>1,048</td><td>26,644</td></tr><tr><td rowspan="2">Segment operating income</td><td>9,324</td><td>6,882</td><td>3,688</td><td>5,523</td><td>4,197</td><td>3,153</td><td>2,898</td><td>760</td><td>36,425</td></tr><tr><td>10,843</td><td>6,396</td><td>3,759</td><td>5,155</td><td>3,113</td><td>2,959</td><td>2,566</td><td>339</td><td>35,130</td></tr><tr><td rowspan="2">Unallocable expenses</td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td>4,678</td></tr><tr><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td>4,225</td></tr><tr><td rowspan="2">Other income, net (Refer to Note 2.17)</td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td>4,711</td></tr><tr><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td>2,701</td></tr><tr><td rowspan="2">Finance cost</td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td>470</td></tr><tr><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td>284</td></tr><tr><td rowspan="2">Profit before tax</td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td>35,988</td></tr><tr><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td>33,322</td></tr><tr><td>ncome tax expense</td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td>9,740</td></tr><tr><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td>9,214</td></tr><tr><td rowspan="2">Net Profit</td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td>26,248</td></tr><tr><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td>24,108</td></tr><tr><td rowspan="2">Depreciation and amortization expense</td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td>4,678</td></tr><tr><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td>4,225</td></tr><tr><td rowspan="2">Non-cash expenses other than depreciation</td><td>and amortization</td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td>—</td></tr><tr><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td>—</td></tr></tbody></table>
(1) Financial Services include enterprises in Financial Services and Insurance
(2) Retail includes enterprises in Retail, Consumer Packaged Goods and Logistics
E
§
(3) Communication includes enterprises in Communication, Telecom OEM and Media
(4) Life Sciences includes enterprises in Life sciences and Health care
(5) Others include operating segments of businesses in India, Japan, China, Infosys Public Services & other enterprises in Public Services
* Includes impact on account of McCamish cybersecurity incident. Refer note 2.21.3.
Significant clients
No client individually accounted for more than 10% of the revenues for the three months and year ended March 31, 2024 and March 31, 2023, respectively.
38
2.24 FUNCTION WISE CLASSIFICATION OF CONDENSED CONSOLIDATED STATEMENT OF PROFIT AND LOSS
<table><thead><tr><th rowspan="2">Particulars</th><th rowspan="2">Note No.</th><th colspan="2">Three months ended March 31,</th><th colspan="2">Y ear ended March 31,</th></tr><tr><th>2024</th><th>2023</th><th>2024</th><th>2023</th></tr></thead><tbody><tr><td>Revenue from operations</td><td>2.16</td><td>37,923</td><td>37,441</td><td>153,670</td><td>146,767</td></tr><tr><td>Cost of Sales</td><td></td><td>26,748</td><td>26,011</td><td>107,413</td><td>102,353</td></tr><tr><td>Gross profit</td><td></td><td>11,175</td><td>11,430</td><td>46,257</td><td>44,414</td></tr><tr><td colspan="6">Operating expenses</td></tr><tr><td>Selling and marketing expenses</td><td></td><td>1,735</td><td>1,659</td><td>6,973</td><td>6,249</td></tr><tr><td>General and administration expenses</td><td></td><td>1,819</td><td>1,894</td><td>7,537</td><td>7,260</td></tr><tr><td>Total operating expenses</td><td></td><td>3,554</td><td>3,553</td><td>14,510</td><td>13,509</td></tr><tr><td>Operating profit</td><td></td><td>7,621</td><td>7,877</td><td>31,747</td><td>30,905</td></tr><tr><td>Other income, net</td><td>2.17</td><td>2,729</td><td>671</td><td>4,711</td><td>2,701</td></tr><tr><td>Finance cost</td><td></td><td>110</td><td>82</td><td>470</td><td>284</td></tr><tr><td>Profit before tax</td><td></td><td>10,240</td><td>8,466</td><td>35,988</td><td>33,322</td></tr><tr><td colspan="6">Tax expense:</td></tr><tr><td>Current tax</td><td>2.15</td><td>1,173</td><td>2,260</td><td>8,390</td><td>9,287</td></tr><tr><td>Deferred tax</td><td>2.15</td><td>1,092</td><td>72</td><td>1,350</td><td>(73)</td></tr><tr><td>Profit for the period</td><td></td><td>7,975</td><td>6,134</td><td>26,248</td><td>24,108</td></tr><tr><td colspan="6"></td></tr><tr><td colspan="6" rowspan="2">Other comprehensive income Items that will not be reclassified to loss</td></tr><tr><td>subsequently profit or</td><td></td><td></td><td></td><td></td><td></td></tr><tr><td>of the defined benefit</td><td></td><td>26</td><td>25</td><td>120</td><td></td></tr><tr><td>Remeasurement net liability/asset, net</td><td></td><td></td><td></td><td></td><td>8</td></tr><tr><td>Equity instruments through other comprehensive income, net</td><td></td><td>(12)</td><td>(15)</td><td>19</td><td>™</td></tr><tr><td></td><td></td><td>14</td><td>10</td><td>139</td><td>1</td></tr><tr><td colspan="6" rowspan="2">Items that will be reclassified subsequently to profit or loss Fair value derivatives cash flow 28 36 11</td></tr><tr><td>changes on designated as hedge, net</td><td></td><td></td><td></td><td></td><td>(7</td></tr><tr><td>Exchange differences on translation of foreign operations</td><td></td><td>(231)</td><td>61</td><td>226</td><td>776</td></tr><tr><td>Fair value changes on investments, net</td><td></td><td>37</td><td>42</td><td>144</td><td>(256)</td></tr><tr><td></td><td></td><td>(166)</td><td>139</td><td>381</td><td>513</td></tr><tr><td>Total other comprehensive income / (loss), net of tax</td><td></td><td>(152)</td><td>149</td><td>520</td><td>514</td></tr><tr><td>Total comprehensive income for the period</td><td></td><td>7,823</td><td>6,283</td><td>26,768</td><td>24,622</td></tr><tr><td>Profit attributable to:</td><td></td><td></td><td></td><td></td><td></td></tr><tr><td>Owners of the Company</td><td></td><td>7,969</td><td>6,128</td><td>26,233</td><td>24,095</td></tr><tr><td>Non-controlling interests</td><td></td><td>6</td><td>6</td><td>15</td><td>13</td></tr><tr><td></td><td></td><td>7,975</td><td>6,134</td><td>26,248</td><td>24,108</td></tr><tr><td colspan="6">Total comprehensive income attributable to:</td></tr><tr><td>Owners of the Company</td><td></td><td>7,821</td><td>6,276</td><td>26,754</td><td>24,598</td></tr><tr><td rowspan="2">Non-controlling interests</td><td></td><td>2</td><td>7</td><td>14</td><td>24</td></tr><tr><td></td><td>7,823</td><td>6,283</td><td>26,768</td><td>24,622</td></tr></tbody></table>
for and on behalf of the Board of Directors of Infosys Limited
(In ₹ crore)
o
D. Sundaram Lead Independent Director
Salil Parekh Chief Executive Officer and Managing Director
Bobby Parikh Director
Bengaluru April 18, 2024
Jayesh Sanghrajka Chief Financial Officer
A.G.S. Manikantha Company Secretary
39


#### Preprocessing of Tables and Text

In [33]:
from typing import Any
from pydantic import BaseModel

class Element(BaseModel):
    type: str
    page_content: Any

# Categorize by type
categorized_elements = []

for element in elements:
    if "unstructured.documents.elements.Table" in str(type(element)):
        categorized_elements.append(Element(type="table", page_content=str(element.metadata.text_as_html)))
    elif "unstructured.documents.elements.NarrativeText" in str(type(element)):
        categorized_elements.append(Element(type="text", page_content=str(element)))
    elif "unstructured.documents.elements.ListItem" in str(type(element)):
        categorized_elements.append(Element(type="text", page_content=str(element)))
    elif "unstructured.documents.elements.Title" in str(type(element)):
        categorized_elements.append(Element(type="text", page_content=str(element)))
    elif "unstructured.documents.elements.Address" in str(type(element)):
        categorized_elements.append(Element(type="text", page_content=str(element)))
    elif "unstructured.documents.elements.EmailAddress" in str(type(element)):
        categorized_elements.append(Element(type="text", page_content=str(element)))
    elif "unstructured.documents.elements.Header" in str(type(element)):
        categorized_elements.append(Element(type="CodeSnippet", page_content=str(element)))
    elif "unstructured.documents.elements.CodeSnippet" in str(type(element)):
        categorized_elements.append(Element(type="text", page_content=str(element)))
    elif "unstructured.documents.elements.UncategorizedText" in str(type(element)):
        categorized_elements.append(Element(type="text", page_content=str(element)))

In [34]:
# Tables
table_elements = [element for element in categorized_elements if element.type == "table"]
print(len(table_elements))

# Text
text_elements = [element for element in categorized_elements if element.type == "text"]
print(len(text_elements))

49
528


In [35]:
for text_element in text_elements:
    print(text_element.page_content)

INFOSYS LIMITED AND SUBSIDIARIES Condensed Consolidated Financial Statements under Indian Accounting Standards (Ind AS) for the three months and year ended March 31, 2024
No.
INFOSYS LIMITED AND SUBSIDIARIES
The accompanying notes form an integral part of the interim condensed consolidated financial statements
As per our report of even date attached
for Deloitte Haskins & Sells LLP
for and on behalf of the Board of Directors of Infosys Limited
Chartered Accountants
Firm’s Registration No :
(In ₹ crore )
Sanjiv V. Pilgaonkar Partner Membership No. 039826
D. Sundaram Lead Independent Director
Salil Parekh Chief Executive Officer and Managing Director
Bobby Parikh Director
Bengaluru April 18, 2024
Jayesh Sanghrajka Chief Financial Officer
A.G.S. Manikantha Company Secretary
INFOSYS LIMITED AND SUBSIDIARIES
(In % crore, except equity share and per equity share data)
The accompanying notes form an integral part of the interim condensed consolidated financial statements
As per our report of 

In [36]:
all_table_html = ""

for table in table_elements:
    all_table_html += table.page_content + "</br></br>"
    print(table.page_content)

<table><thead><tr><th>ndex</th><th>Page No.</th></tr></thead><tbody><tr><td>“ondensed Consolidated Balance Sheet</td><td>1</td></tr><tr><td>“ondensed Consolidated Statement of Profit and Loss</td><td></td></tr><tr><td>“ondensed Consolidated Statement of Changes in Equity</td><td>E</td></tr><tr><td>“ondensed Consolidated Statement of Cash Flows</td><td>5</td></tr><tr><td colspan="2">)verview and Notes to the Interim Condensed Consolidated Financial Statements</td></tr><tr><td colspan="2">. Overview</td></tr><tr><td>1.1 Company overview</td><td></td></tr><tr><td>1.2 Basis of preparation of financial statements</td><td>R 7</td></tr><tr><td>1.3 Basis of consolidation</td><td>S</td></tr><tr><td>1.4 Use of estimates and judgments</td><td>7</td></tr><tr><td>1.5 Critical accounting estimates and judgments....</td><td>[ R 8</td></tr><tr><td colspan="2">. Notes to the Interim Condensed Consolidated Financial Statements</td></tr><tr><td>2.1 Business Combinations</td><td>10</td></tr><tr><td>2.2 Pr

In [37]:
Markdown(all_table_html)

<table><thead><tr><th>ndex</th><th>Page No.</th></tr></thead><tbody><tr><td>“ondensed Consolidated Balance Sheet</td><td>1</td></tr><tr><td>“ondensed Consolidated Statement of Profit and Loss</td><td></td></tr><tr><td>“ondensed Consolidated Statement of Changes in Equity</td><td>E</td></tr><tr><td>“ondensed Consolidated Statement of Cash Flows</td><td>5</td></tr><tr><td colspan="2">)verview and Notes to the Interim Condensed Consolidated Financial Statements</td></tr><tr><td colspan="2">. Overview</td></tr><tr><td>1.1 Company overview</td><td></td></tr><tr><td>1.2 Basis of preparation of financial statements</td><td>R 7</td></tr><tr><td>1.3 Basis of consolidation</td><td>S</td></tr><tr><td>1.4 Use of estimates and judgments</td><td>7</td></tr><tr><td>1.5 Critical accounting estimates and judgments....</td><td>[ R 8</td></tr><tr><td colspan="2">. Notes to the Interim Condensed Consolidated Financial Statements</td></tr><tr><td>2.1 Business Combinations</td><td>10</td></tr><tr><td>2.2 Property, plant and equipment</td><td>12</td></tr><tr><td>2.3 Goodwill and other intangible assets</td><td>14</td></tr><tr><td>2.4 Investments</td><td>15</td></tr><tr><td>2.5 Loans</td><td>16</td></tr><tr><td>2.6 Other financial assets</td><td>16</td></tr><tr><td>2.7 Trade receivables</td><td>16</td></tr><tr><td>2.8 Cash and cash equivalents ..</td><td>17</td></tr><tr><td></td><td></td></tr><tr><td>2.9 Other assets</td><td>17</td></tr><tr><td>2.10 Financial instruments</td><td>18</td></tr><tr><td>2.11 Equity</td><td>22</td></tr><tr><td>2.12 Other financial liabilities</td><td>25</td></tr><tr><td>2.13 Other liabilities</td><td>25</td></tr><tr><td>2.14 Provisions</td><td>26</td></tr><tr><td>2.15 Income taxes</td><td>27</td></tr><tr><td>2.16 Revenue from operations</td><td>28</td></tr><tr><td>2.17 Other income, net</td><td>30</td></tr><tr><td>2.18 Expenses</td><td>31</td></tr><tr><td>2.19 Leases</td><td>32</td></tr><tr><td>2.20 Basic and diluted shares used in computing earnings per equity share</td><td>34</td></tr><tr><td>2.21 Contingent liabilities and commitments</td><td>34</td></tr><tr><td>2.22 Related party transactions</td><td>36</td></tr><tr><td>2.23 Segment reporting</td><td>37</td></tr><tr><td>2.24 Function wise classification of Condensed Consolidated Statement of Profit and Loss</td><td>39</td></tr></tbody></table></br></br><table><thead><tr><th>Condensed Consolidated Balance Sheets as at</th><th>Note No.</th><th>March 31, 2024</th><th>(4~ crore March 31, 2023</th></tr></thead><tbody><tr><td colspan="4">ASSETS</td></tr><tr><td colspan="4">Non-current assets</td></tr><tr><td>Property, plant and equipment</td><td>22</td><td>12,370</td><td>13,346</td></tr><tr><td>Right-of-use assets</td><td>2.19</td><td>6,552</td><td>6,882</td></tr><tr><td>Capital work-in-progress</td><td></td><td>293</td><td>288</td></tr><tr><td>Goodwi</td><td>2.3</td><td>7,303</td><td>7,248</td></tr><tr><td>Other intangible assets</td><td></td><td>1,397</td><td>1,749</td></tr><tr><td colspan="4">Financial assets</td></tr><tr><td>Investments</td><td>2.4</td><td>11,708</td><td>12,569</td></tr><tr><td>Loans</td><td>2.5</td><td>34</td><td>39</td></tr><tr><td>Other financial assets</td><td>2.6</td><td>3,105</td><td>2,798</td></tr><tr><td>Deferred tax assets (net)</td><td></td><td>454</td><td>1,245</td></tr><tr><td>ncome tax assets (net)</td><td></td><td>3,045</td><td>6,453</td></tr><tr><td>Other non-current assets</td><td>2.9</td><td>2,121</td><td>2,318</td></tr><tr><td>Total non-current assets</td><td></td><td>48,382</td><td>54,935</td></tr><tr><td colspan="4">Current assets</td></tr><tr><td colspan="4">Financial assets</td></tr><tr><td>Investments</td><td>2.4</td><td>12,915</td><td>6,909</td></tr><tr><td>Trade receivables</td><td>2.7</td><td>30,193</td><td>25,424</td></tr><tr><td>Cash and cash equivalents</td><td>2.8</td><td>14,786</td><td>12,173</td></tr><tr><td>Loans</td><td>2.5</td><td>248</td><td>289</td></tr><tr><td>Other financial assets</td><td>2.6</td><td>12,085</td><td>11,604</td></tr><tr><td>Income tax assets (net)</td><td></td><td>6,397</td><td>6</td></tr><tr><td>Other current assets</td><td>2.9</td><td>12,808</td><td>14,476</td></tr><tr><td>Total current assets</td><td></td><td>89,432</td><td>70,881</td></tr><tr><td>Total assets</td><td></td><td>137,814</td><td>125,816</td></tr><tr><td colspan="4" rowspan="2">EQUITY AND LIABILITIES Equity</td></tr><tr><td>Equity share capital</td><td>2.11</td><td>2,071</td><td>2,069</td></tr><tr><td>Other equity</td><td></td><td>86,045</td><td>73,338</td></tr><tr><td>Total equity attributable to equity holders of the C ompany</td><td></td><td>88,116</td><td>75,407</td></tr><tr><td>Non-controlling interests</td><td></td><td>345</td><td>388</td></tr><tr><td>Total equity</td><td></td><td>88,461</td><td>75,795</td></tr><tr><td colspan="4">Liabilities
 Non-current liabilities</td></tr><tr><td>Financial Liabilities</td><td></td><td></td><td></td></tr><tr><td>Lease liabilities</td><td>2.19</td><td>6,400</td><td>7,057</td></tr><tr><td>Other financial liabilities</td><td>2.12</td><td>2,130</td><td>2,058</td></tr><tr><td>Deferred tax liabilities (net)</td><td></td><td>1,794</td><td>1,220</td></tr><tr><td>Other non-current liabilities</td><td>2.13</td><td>235</td><td>500</td></tr><tr><td>Total non-current liabilities</td><td></td><td>10,559</td><td>10,835</td></tr><tr><td colspan="4">Current liabilities</td></tr><tr><td>Financial Liabilities</td><td></td><td></td><td></td></tr><tr><td>Lease liabilities</td><td>2.19</td><td>1,959</td><td>1,242</td></tr><tr><td>Trade payables</td><td></td><td>3,956</td><td>3,865</td></tr><tr><td>Other financial liabilities</td><td>2.12</td><td>16,959</td><td>18,558</td></tr><tr><td>Other current liabilities</td><td>2.13</td><td>10,539</td><td>10,830</td></tr><tr><td>Provisions</td><td>2.14</td><td>1,796</td><td>1,307</td></tr><tr><td>Income tax liabilities (net)</td><td></td><td>3,585</td><td>3,384</td></tr><tr><td>Total current liabilities</td><td></td><td>38,794</td><td>39,186</td></tr><tr><td>Total equity and liabilities</td><td></td><td>137,814</td><td>125,816</td></tr></tbody></table></br></br><table><thead><tr><th rowspan="2">Condensed Consolidated Statement of Profit and L oss for the</th><th rowspan="2">Note No.</th><th colspan="4">in &lt; crore, except equily share ana per equity share data)</th></tr><tr><th>Three months ended</th><th colspan="3">March 31, Y ear ended March 31,</th></tr></thead><tbody><tr><td>Revenue from operations</td><td>2.16</td><td>2024 37,923</td><td>2023 37,44</td><td>2024 153,670</td><td>2023 146,767</td></tr><tr><td>Other income, net</td><td>2.17</td><td>2,729</td><td>67</td><td>4,711</td><td>2,701</td></tr><tr><td>Total income</td><td></td><td>40,652</td><td>38,112</td><td>158,381</td><td>149,468</td></tr><tr><td>Expenses</td><td></td><td></td><td></td><td></td><td></td></tr><tr><td>Employee benefit expenses</td><td>2.18</td><td>20,393</td><td>20,31</td><td>82,620</td><td>78,359</td></tr><tr><td>Cost of technical sub-contractors</td><td></td><td>2,967</td><td>3,116</td><td>12,232</td><td>14,062</td></tr><tr><td>Travel expenses</td><td></td><td>471</td><td>426</td><td>1,759</td><td>1,525</td></tr><tr><td>Cost of software packages and others</td><td>2.18</td><td>3,687</td><td>2,886</td><td>13,515</td><td>10,902</td></tr><tr><td>Communication expenses</td><td></td><td>147</td><td>7</td><td>677</td><td>713</td></tr><tr><td>Consultancy and professional charges</td><td></td><td>489</td><td>387</td><td>1,726</td><td>1,684</td></tr><tr><td>Depreciation and amortization expenses</td><td></td><td>1,163</td><td>1,12</td><td>4,678</td><td>4,225</td></tr><tr><td>Finance cost</td><td></td><td>110</td><td>82</td><td>470</td><td>284</td></tr><tr><td>Other expenses Total</td><td>2.18</td><td>985</td><td>1,146</td><td>4,716</td><td>4,392</td></tr><tr><td>expenses</td><td></td><td>30,412</td><td>29,646</td><td>122,393</td><td>116,146</td></tr><tr><td colspan="6">Profit before tax 10,240 8,466 35,988 33,322</td></tr><tr><td>Tax expense:</td><td></td><td></td><td></td><td></td><td></td></tr><tr><td>Current tax</td><td>2.15</td><td>1,173</td><td>2,260</td><td>8,390</td><td>9,287</td></tr><tr><td>Deferred tax</td><td>2.15</td><td>1,092</td><td>72</td><td>1,350</td><td>(73)</td></tr><tr><td>Profit for the period</td><td></td><td>7,975</td><td>6,134</td><td>26,248</td><td>24,108</td></tr><tr><td colspan="6">Other comprehensive income</td></tr><tr><td colspan="6">tems that will not be reclassified subsequently to profit or loss</td></tr><tr><td>Remeasurement of the net defined benefit liability/asset, net</td><td></td><td>26</td><td>25</td><td>120</td><td></td></tr><tr><td>Equity instruments through other comprehensive income, net</td><td></td><td>(12)</td><td>(15)</td><td>19</td><td>(N</td></tr><tr><td></td><td></td><td>14</td><td>10</td><td>139</td><td></td></tr><tr><td>that will be reclassified loss</td><td></td><td></td><td></td><td></td><td></td></tr><tr><td>tems subsequently to profit or</td><td></td><td></td><td></td><td></td><td></td></tr><tr><td>Fair value changes on derivatives designated as cash flow hedge, net</td><td></td><td>28</td><td>36</td><td>11</td><td>(7)</td></tr><tr><td>Exchange differences on translation of foreign operations</td><td></td><td>(231)</td><td>61</td><td>226</td><td>776</td></tr><tr><td>Fair value changes on investments, net</td><td></td><td>37</td><td>42</td><td>144</td><td>(256)</td></tr><tr><td></td><td></td><td>(166)</td><td>139</td><td>381</td><td>513</td></tr><tr><td>Total other comprehensive income /(loss), net of tax</td><td></td><td>(152)</td><td>149</td><td>520</td><td>514</td></tr><tr><td>Total comprehensive income for the period</td><td></td><td>7,823</td><td>6,283</td><td>26,768</td><td>24,622</td></tr><tr><td>Profit attributable to:</td><td></td><td></td><td></td><td></td><td></td></tr><tr><td>Owners of the Company</td><td></td><td>7,969</td><td>6,128</td><td>26,233</td><td>24,095</td></tr><tr><td>Non-controlling interests</td><td></td><td>6</td><td>6</td><td>15</td><td>13</td></tr><tr><td></td><td></td><td>7,975</td><td>6,134</td><td>26,248</td><td>24,108</td></tr><tr><td colspan="6">Total comprehensive income attributable to:</td></tr><tr><td>Owners of the Company</td><td></td><td>7,821</td><td>6,276</td><td>26,754</td><td>24,598</td></tr><tr><td>Non-controlling interests</td><td></td><td>2 7,823</td><td>7 6,283</td><td>14 26,768</td><td>24 24,622</td></tr><tr><td colspan="6">Earnings per equity share</td></tr><tr><td colspan="6">Equity shares of par value Z5/- each</td></tr><tr><td>Basic (in X per share)</td><td></td><td>19.25</td><td>14.79</td><td>63.39</td><td>57.63</td></tr><tr><td>Diluted (in % per share)</td><td></td><td>19.22</td><td>14.77</td><td>63.29</td><td>57.54</td></tr><tr><td colspan="6">Weighted average equity shares used in computing earnings per equity share</td></tr><tr><td>Basic (in shares)</td><td>2.20</td><td>4,139,432,133</td><td>4,144,013,195</td><td>4,138,568,090</td><td>4,180,897,857</td></tr><tr><td>Diluted (in shares)</td><td>2.20</td><td>4,145,052,370</td><td>4,149,555,426</td><td>4,144,680,425</td><td>4,187,731,070</td></tr></tbody></table></br></br><table><thead><tr><th rowspan="2">Particulars</th><th rowspan="2">Equity 
 Share capital</th><th rowspan="2">Capital 
 reserve</th><th colspan="5">OTHER EQUITY</th><th></th><th colspan="4"></th><th rowspan="2">Total equity attributable to equity holders of the Company</th><th rowspan="2">Non- controlling 
 interest</th><th rowspan="2">Total equity</th></tr><tr><th>Capital 
 redemption 
 reserve</th><th>Securities Premium</th><th>Reserves &amp; Surplus Retained earnings</th><th>General Share reserve Options 
 Outstanding 
 Account</th><th>Special 
 Economic Zone Re- investment 
 @ reserve</th><th>Other (&amp;) reserves</th><th>Equity 
 instruments 
 through other comprehensive 
 income</th><th>Other comprehensive Exchange 
 differences translating the financial statements of foreign operation</th><th>income Effective portion of Cash Flow Hedges</th><th>Other items of other comprehensive 
 income / (loss)</th></tr></thead><tbody><tr><td>Balance as at April 1, 2022</td><td>2,098</td><td>54</td><td>139</td><td>200</td><td>61,313</td><td>1,061</td><td>8,339</td><td>16</td><td>254</td><td>1,560</td><td></td><td>(292)</td><td>75,350</td><td>386</td><td>75,736</td></tr><tr><td>mpact on adoption of amendment to Ind AS 37*</td><td></td><td></td><td></td><td></td><td>(19)</td><td></td><td>—                 —                 —</td><td></td><td>—</td><td></td><td></td><td></td><td>(19)</td><td></td><td>(19)</td></tr><tr><td></td><td>2,098</td><td>54</td><td>139</td><td>200</td><td>61,294</td><td>1,061</td><td>8,339</td><td>16</td><td>254</td><td>1,560</td><td></td><td>(292)</td><td>75,331</td><td>386</td><td>75,717</td></tr><tr><td colspan="16">Changes in equity for the year ended March 31, 2023</td></tr><tr><td>Profit for the period</td><td></td><td></td><td></td><td></td><td>24,095</td><td></td><td>—                 —                 —</td><td></td><td></td><td>—                             —                      —</td><td></td><td>—</td><td>24,095</td><td>13</td><td>24,108</td></tr><tr><td>Remeasurement of the net defined benefit liability/asset, net*</td><td>—                  —                  —                 —                 —                 —</td><td></td><td></td><td></td><td></td><td></td><td>—                 —                 —</td><td></td><td></td><td>—                             —                      —                          8</td><td></td><td></td><td>8</td><td>—</td><td>8</td></tr><tr><td>Equity instruments through other comprehensive income, net*</td><td>—                  —                  —                 —                 —                 —</td><td></td><td></td><td></td><td></td><td></td><td>—                 —                 —</td><td></td><td>(7</td><td></td><td></td><td></td><td>(7)</td><td></td><td>(7)</td></tr><tr><td>Fair value changes on derivatives designated as cash flow hedge, net*</td><td></td><td></td><td></td><td></td><td></td><td></td><td>—                 —                 —</td><td></td><td></td><td></td><td>U]</td><td></td><td>(7</td><td></td><td>(7</td></tr><tr><td>Exchange differences on translation of foreign operations</td><td>—                  —                  —                 —                 —                 —</td><td></td><td></td><td></td><td></td><td></td><td>—                 —                 —</td><td></td><td>—</td><td>765</td><td></td><td>—</td><td>765</td><td>11</td><td>776</td></tr><tr><td>Fair value changes on investments, net*</td><td>—                  —                  —                 —                 —                 —</td><td></td><td></td><td></td><td></td><td></td><td>—                 —                 —</td><td></td><td></td><td></td><td></td><td>(256)</td><td>(256)</td><td></td><td>(256)</td></tr><tr><td>Total Comprehensive income for the period</td><td></td><td></td><td></td><td></td><td>24,095</td><td></td><td>—                 —                 —</td><td></td><td>(7)</td><td>765</td><td>(7)</td><td>(248)</td><td>24,598</td><td>24</td><td>24,622</td></tr><tr><td>Shares issued on exercise of employee stock options (Refer to Note 2.11)</td><td>1</td><td></td><td></td><td>34</td><td>.</td><td></td><td>—                 —                 —</td><td></td><td></td><td>—                             —                      —</td><td></td><td>—</td><td>35</td><td>—</td><td>35</td></tr><tr><td>Employee stock compensation expense (Refer to Note 2.11)</td><td>—                  —                  —                 —                 —                 —</td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td>—                             —                      —</td><td></td><td>—</td><td>514</td><td>—</td><td>514</td></tr><tr><td>Transferred on account of options not exercised</td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td>—                             —                      —</td><td></td><td>—</td><td>—</td><td>—</td><td>—</td></tr><tr><td>Buyback of equity shares (Refer to Note 2.11) **</td><td>(30)</td><td></td><td></td><td>(340)</td><td>(11,096)</td><td></td><td>—                 —                 —</td><td></td><td></td><td>—                             —                      —</td><td></td><td></td><td>(11,466)</td><td></td><td>(11,466)</td></tr><tr><td>Transaction costs relating to buyback*</td><td></td><td></td><td></td><td>(19)</td><td>(5)</td><td></td><td>—                 —                 —</td><td></td><td></td><td>—                             —                      —</td><td></td><td></td><td>(24)</td><td></td><td>(24)</td></tr><tr><td>Amount transferred to capital redemption reserve upon buyback</td><td></td><td></td><td>30</td><td></td><td>(21</td><td>9)</td><td>—                 —                 —</td><td></td><td></td><td>—                             —                      —</td><td></td><td>—</td><td>—</td><td>—</td><td>—</td></tr><tr><td>Transferred to Special Economic Zone Re-investment reserve</td><td></td><td></td><td></td><td></td><td>(3,139 i</td><td></td><td>3,139</td><td></td><td></td><td>—                             —                      —</td><td></td><td>—</td><td>—</td><td>—</td><td>—</td></tr><tr><td>Transfer to legal reserve</td><td></td><td></td><td></td><td></td><td>©)</td><td></td><td>—                 —                   3</td><td></td><td></td><td>—                             —                      —</td><td></td><td>—</td><td>—</td><td>—</td><td>—</td></tr><tr><td>Transferred on account of exercise of stock options (Refer to note 2.11)</td><td></td><td></td><td></td><td>291</td><td>.</td><td></td><td></td><td></td><td></td><td>—                             —                      —</td><td></td><td>—</td><td>—</td><td>—</td><td>—</td></tr><tr><td>ncome tax benefit arising on exercise of stock options</td><td>—                  —                  —                 —                 —                 —</td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td>—                             —                      —</td><td></td><td>—</td><td>51</td><td>—</td><td>51</td></tr><tr><td>Dividends "</td><td></td><td></td><td></td><td></td><td>(13,632)</td><td></td><td>—                 —                 —</td><td></td><td></td><td>—                             —                      —</td><td></td><td></td><td>(13,632)</td><td></td><td>(13,632)</td></tr><tr><td>Dividends paid to non controlling interest of subsidiary</td><td>—                  —                  —                 —                 —                 —</td><td></td><td></td><td></td><td></td><td></td><td>—                 —                 —</td><td></td><td></td><td>—                             —                      —</td><td></td><td>—</td><td></td><td>(22)</td><td>(22)</td></tr><tr><td>Transferred from Special Economic Zone Re-investment reserve on utilization</td><td></td><td></td><td></td><td></td><td>1,464</td><td></td><td>(1,464)</td><td></td><td></td><td>—                             —                      —</td><td></td><td>—</td><td>—</td><td>—</td><td>—</td></tr><tr><td>Balance as at March 31, 2023</td><td>2,069</td><td>54</td><td>169</td><td>166</td><td>58,957</td><td>1,054</td><td>10,014</td><td>19</td><td>247</td><td>2,325</td><td>()</td><td>(540)</td><td>75,407</td><td>388</td><td>75,795</td></tr></tbody></table></br></br><table><thead><tr><th rowspan="2">Particulars</th><th rowspan="2">Note No.</th><th colspan="2">(In X crore)</th></tr><tr><th>Y ear ended March</th><th>31,</th></tr></thead><tbody><tr><td colspan="4">2024
 2023
 Cash flow from operating activities</td></tr><tr><td colspan="4">Profit for the period 26,248 24,108</td></tr><tr><td colspan="4">Adjustments to reconcile net profit to net cash provided by operating activities:</td></tr><tr><td>ncome tax</td><td>2.15</td><td>9,740</td><td>9,214</td></tr><tr><td>expense and amortization</td><td></td><td>4,678</td><td>4,225</td></tr><tr><td>Depreciation nterest and dividend income</td><td></td><td>(2,067)</td><td>(1,817)</td></tr><tr><td>Finance cost</td><td></td><td>470</td><td>284</td></tr><tr><td>loss under credit loss model</td><td></td><td>121</td><td>283</td></tr><tr><td>mpairment recognized / (reversed) expected differences translation of and</td><td></td><td>76</td><td>161</td></tr><tr><td>Exchange on assets liabilities, net</td><td></td><td></td><td></td></tr><tr><td>Stock compensation expense</td><td></td><td>652</td><td>519</td></tr><tr><td>nterest on income tax refund</td><td></td><td>(1,934)</td><td>i</td></tr><tr><td>Provision for post sale client support</td><td></td><td>75</td><td>120</td></tr><tr><td colspan="4">Other adjustments 1,464 508</td></tr><tr><td>Changes in assets and liabilities</td><td></td><td></td><td></td></tr><tr><td>Trade receivables and unbilled revenue Loans, other financial assets and other assets</td><td></td><td>(2,667) (1,172)</td><td>(7,076) (3,108)</td></tr><tr><td>Trade payables</td><td></td><td>91</td><td>(279)</td></tr><tr><td>Other financial other liabilities and</td><td></td><td></td><td></td></tr><tr><td>liabilities, provisions</td><td></td><td>(1,334)</td><td>4,119</td></tr><tr><td>Cash generated from operations</td><td></td><td>34,441</td><td>31,261</td></tr><tr><td>Income taxes paid</td><td></td><td>(9,231)</td><td>(8,794)</td></tr><tr><td>Net cash generated by operating activities</td><td></td><td>25,210</td><td>22,467</td></tr><tr><td>investing Expenditure on property, plant and equipment and intangibles</td><td></td><td>(2,201)</td><td>(2,579)</td></tr><tr><td>Deposits placed with corporation</td><td></td><td>(847)</td><td>(996)</td></tr><tr><td>Redemption of deposits placed with Corporation</td><td></td><td>710</td><td>762</td></tr><tr><td>nterest and dividend received</td><td></td><td></td><td></td></tr><tr><td></td><td></td><td>1,768</td><td>1,525</td></tr><tr><td>towards of business, net of cash</td><td>2.1</td><td>—</td><td></td></tr><tr><td>Payment acquisition acquired</td><td></td><td></td><td>(910)</td></tr><tr><td>Payment of contingent consideration pertaining to acquisition of business</td><td></td><td>(101)</td><td>(60)</td></tr><tr><td>Escrow and other deposits pertaining to Buyback</td><td></td><td>—</td><td>(483)</td></tr><tr><td>Redemption of escrow and other deposits pertaining to Buyback</td><td></td><td>—</td><td>483</td></tr><tr><td>Other</td><td></td><td>128</td><td>7</td></tr><tr><td>receipts</td><td></td><td></td><td></td></tr><tr><td>Payments to acquire Investments mutual fund</td><td></td><td></td><td></td></tr><tr><td>Liquid units fund units</td><td></td><td>(66,191)</td><td>(70,631 (400)</td></tr><tr><td>Target maturity Certificates of</td><td></td><td></td><td></td></tr><tr><td>deposit</td><td></td><td>(8,509</td><td>(10,348</td></tr><tr><td>Commercial Papers</td><td></td><td>(10,387 g</td><td>(3,003</td></tr><tr><td>Non-convertible debentures</td><td></td><td>(1,526</td><td>(249)</td></tr><tr><td>Tax free bonds and government bonds</td><td></td><td></td><td>(@7)</td></tr><tr><td>Government securities</td><td></td><td></td><td>(1,569</td></tr><tr><td>Other Investments</td><td></td><td>(14)</td><td>(20)</td></tr><tr><td colspan="4">Proceeds on sale of Investments</td></tr><tr><td>Tax free bonds and government bonds</td><td></td><td>150</td><td>22</td></tr><tr><td>Liquid mutual funds units</td><td></td><td>64,767</td><td>71,85</td></tr><tr><td>Certificates of deposit</td><td></td><td>9,205</td><td>10,404</td></tr><tr><td>Commercial Papers</td><td></td><td>6,479</td><td>2,298</td></tr><tr><td>Non-convertible debentures</td><td></td><td>1,230</td><td>470</td></tr><tr><td>Government securities</td><td></td><td>304</td><td>1,882</td></tr><tr><td>Equity and preference securities</td><td></td><td>26</td><td>99</td></tr><tr><td>Net cash generated / (used in) from investing activities</td><td></td><td>(5,009)</td><td>(1,209)</td></tr></tbody></table></br></br><table><tbody><tr><td>Payment of lease liabilities</td><td>@ 2,024)</td><td>(1,231)</td></tr><tr><td>Payment of dividends</td><td>(14,692</td><td>(13,631)</td></tr><tr><td>Payment of dividend to non-controlling interest of subsidiary</td><td>(39)</td><td>(22)</td></tr><tr><td>Payment towards buyback of shares pertaining to non controlling interest of subsidiary</td><td>(18)</td><td>—</td></tr><tr><td>Shares issued on exercise of employee stock options</td><td></td><td>35</td></tr><tr><td>Other receipts</td><td></td><td>132</td></tr><tr><td>Other payments</td><td>(736)</td><td>(479)</td></tr><tr><td>Buyback of equity shares including transaction cost and tax on buyback</td><td></td><td>(11,499)</td></tr><tr><td>Net cash used in financing activities</td><td>(17,504;</td><td>(26,695)</td></tr><tr><td>Net increase / (decrease) in cash and cash equivalents</td><td>2,697</td><td>(5,437)</td></tr><tr><td>Effect of exchange rate changes on cash and cash equivalents</td><td>(84)</td><td>138</td></tr><tr><td>Cash and cash equivalents at the beginning of the period</td><td>12,173</td><td>17,472</td></tr><tr><td>Cash and cash equivalents at the end of the period</td><td>14,786</td><td>12,173</td></tr><tr><td colspan="3">Supplementary information:</td></tr><tr><td>Restricted cash balance</td><td>348</td><td>362</td></tr></tbody></table></br></br><table><thead><tr><th rowspan="2">Component</th><th>Acquiree's 
 amount</th><th>Fair value</th><th>(In % crore) Purchase price allocated</th></tr><tr><th>carrying</th><th>adjustments</th><th></th></tr></thead><tbody><tr><td>Net Assets?</td><td>103</td><td>-</td><td>103</td></tr><tr><td colspan="4">Intangible assets :</td></tr><tr><td>Customer contracts and relationships</td><td>-</td><td>274</td><td>274</td></tr><tr><td>Vendor relationships</td><td>-</td><td>30</td><td>30</td></tr><tr><td>Brand</td><td>-</td><td>24</td><td>24</td></tr><tr><td>Deferred tax liabilities on intangible assets</td><td>-</td><td>(80)</td><td>(80)</td></tr><tr><td>Total</td><td>103</td><td>248</td><td>351</td></tr><tr><td>Goodwill</td><td></td><td></td><td>630</td></tr><tr><td>Total purchase price</td><td></td><td></td><td>981</td></tr></tbody></table></br></br><table><tbody><tr><td>Buildings @</td><td>22-25 years</td></tr><tr><td>Plant and machinery (2)</td><td>5 years</td></tr><tr><td>Office equipment</td><td>5 years</td></tr><tr><td>Computer equipment</td><td>3-5 years</td></tr><tr><td>Furniture and fixtures</td><td>5 years</td></tr><tr><td>Vehicles(1)</td><td>5 years</td></tr><tr><td>’Iieasehold improvements</td><td>Lower of usefu</td></tr></tbody></table></br></br><table><thead><tr><th rowspan="2">Particulars</th><th rowspan="2">Land - Freehold</th><th rowspan="2">Buildings 
 (1)</th><th rowspan="2">Plant and machinery</th><th rowspan="2">Office Equipment</th><th></th><th></th><th rowspan="2">Leasehold 
 Improvements</th><th></th><th rowspan="2">(In % crore) Total</th></tr><tr><th>Computer 
 equipment</th><th>Furniture and fixtures</th><th>Vehicles</th></tr></thead><tbody><tr><td>Gross carrying value as at J anuary 1, 2024</td><td>1,432</td><td>11,498</td><td>3,305</td><td>1,510</td><td>8,497</td><td>2,308</td><td>1,456</td><td>45</td><td>30,051</td></tr><tr><td>Additions</td><td></td><td>287</td><td>140</td><td>33</td><td>345</td><td>54</td><td>35</td><td></td><td>894</td></tr><tr><td>Deletions**</td><td></td><td>—</td><td>(16)</td><td>(14)</td><td>(224)</td><td>(34)</td><td>(37)</td><td></td><td>(325)</td></tr><tr><td>Translation difference</td><td></td><td>(15)</td><td>(1)</td><td>1)</td><td>(7)</td><td>(2)</td><td>(7)</td><td></td><td>(33)</td></tr><tr><td>Gross carrying value as at March 31, 2024</td><td>1,432</td><td>11,770</td><td>3,428</td><td>1,528</td><td>8,611</td><td>2,326</td><td>1,447</td><td>45</td><td>30,587</td></tr><tr><td>Accumulated depreciation as at J anuary 1, 2024</td><td></td><td>(4,8149)</td><td>(2,584)</td><td>(1,253)</td><td>(6,267)</td><td>(1,807)</td><td>(1,131)</td><td>42)</td><td>(17,898)</td></tr><tr><td>Depreciation</td><td></td><td>(111)</td><td>(63)</td><td>(32)</td><td>(336)</td><td>(58)</td><td>(46)</td><td></td><td>(646)</td></tr><tr><td>Accumulated depreciation on deletions**</td><td></td><td>—</td><td>16</td><td>14</td><td>219</td><td>26</td><td>34</td><td></td><td>309</td></tr><tr><td>Translation difference</td><td>i</td><td>4</td><td>1</td><td>2</td><td>4</td><td>2</td><td>5</td><td></td><td>18</td></tr><tr><td>Accumulated depreciation as at March 31, 2024</td><td>—</td><td>(4,921)</td><td>(2,630)</td><td>(1,269)</td><td>(6,380)</td><td>(1,837)</td><td>(1,138)</td><td>(4;)</td><td>(18,217)</td></tr><tr><td>Carrying value as at January 1, 2024</td><td>1,432</td><td>6,684</td><td>721</td><td>257</td><td>2,230</td><td>501</td><td>325</td><td></td><td>12,153</td></tr><tr><td>Carrying value as at March 31, 2024</td><td>1,432</td><td>6,849</td><td>798</td><td>259</td><td>2,231</td><td>489</td><td>309</td><td></td><td>12,370</td></tr></tbody></table></br></br><table><thead><tr><th rowspan="2">Particulars</th><th rowspan="2">Land - Freehold</th><th rowspan="2">Buildings 
 (1)</th><th rowspan="2">Plant and machinery</th><th rowspan="2">Office Equipment</th><th></th><th></th><th rowspan="2">Leasehold Improvements</th><th></th><th rowspan="2">(In % crore) Total</th></tr><tr><th>Computer 
 equipment</th><th>Furniture and fixtures</th><th>Vehicles</th></tr></thead><tbody><tr><td>Gross carrying value as at J anuary 1, 2023</td><td>1,431</td><td>11,530</td><td>3,368</td><td>1,466</td><td>8,895</td><td>2,450</td><td>1,353</td><td></td><td>30,537</td></tr><tr><td>Additions</td><td>2</td><td>29</td><td>109</td><td>55</td><td>494</td><td>162</td><td>103</td><td></td><td>955</td></tr><tr><td>Deletions*</td><td>()</td><td>—</td><td>(175)</td><td>(40)</td><td>(877)</td><td>(311)</td><td>(13)</td><td></td><td>(1,418)</td></tr><tr><td>Translation difference</td><td></td><td>3</td><td>—</td><td>1</td><td>7</td><td>2</td><td>2</td><td></td><td>15</td></tr><tr><td>Gross carrying value as at March 31, 2023</td><td>1,4;</td><td>11,562</td><td>3,302</td><td>1,482</td><td>8,519</td><td>2,303</td><td>1,445</td><td>45</td><td>30,089</td></tr><tr><td>Accumulated depreciation as at January 1, 2023</td><td>—</td><td>(4,425)</td><td>(2,547)</td><td>(1,206)</td><td>(6,339)</td><td>(1,922)</td><td>(992)</td><td>39)</td><td>(17,470)</td></tr><tr><td>Depreciation</td><td></td><td>(109)</td><td>(65)</td><td>(31)</td><td>(354)</td><td>(62)</td><td>(48)</td><td>1)</td><td>(670)</td></tr><tr><td>Accumulated depreciation on deletions*</td><td></td><td>—</td><td>175</td><td>40</td><td>871</td><td>310</td><td>9</td><td></td><td>1,405</td></tr><tr><td>Translation difference</td><td></td><td>(1)</td><td>—</td><td>1)</td><td>(4)</td><td>(1)</td><td>1)</td><td></td><td>(8</td></tr><tr><td>Accumulated depreciation as at March 31, 2023</td><td></td><td>(4,535)</td><td>(2,437)</td><td>(1,198)</td><td>(5,826)</td><td>(1,675)</td><td>(1,032)</td><td>(4;)</td><td>(16,743)</td></tr><tr><td>Carrying value as at January 1, 2023</td><td>1,431</td><td>7,105</td><td>821</td><td>260</td><td>2,556</td><td>528</td><td>361</td><td></td><td>13,067</td></tr><tr><td>Carrying value as at March 31, 2023</td><td>1,431</td><td>7,027</td><td>865</td><td>284</td><td>2,693</td><td>628</td><td>413</td><td></td><td>13,346</td></tr></tbody></table></br></br><table><thead><tr><th rowspan="2">Particulars</th><th rowspan="2">Land - Freehold</th><th rowspan="2">Buildings 
 (1)</th><th rowspan="2">Plant and machinery</th><th rowspan="2">Office Equipment</th><th></th><th></th><th rowspan="2">Leasehold 
 Improvements</th><th colspan="2">(In % crore)</th></tr><tr><th>Computer 
 equipment</th><th>Furniture and fixtures</th><th>Vehicles</th><th>Total</th></tr></thead><tbody><tr><td>Gross carrying value as at April 1, 2023</td><td>1,431</td><td>11,562</td><td>3,302</td><td>1,482</td><td>8,519</td><td>2,303</td><td>1,445</td><td>45</td><td>30,089</td></tr><tr><td>Additions</td><td>1</td><td>300</td><td>193</td><td>106</td><td>931</td><td>121</td><td>108</td><td></td><td>1,761</td></tr><tr><td>Deletions**</td><td>i</td><td>(55)</td><td>(64)</td><td>(60)</td><td>(846)</td><td>(99)</td><td>(102)</td><td>1)</td><td>(1,227)</td></tr><tr><td>Translation difference</td><td>—</td><td>(37)</td><td>[©)</td><td></td><td>7</td><td>1</td><td>4)</td><td></td><td>(36)</td></tr><tr><td>Gross carrying value as at March 31, 2024</td><td>1,432</td><td>11,770</td><td>3,428</td><td>1528</td><td>8,611</td><td>2,326</td><td>1,447</td><td>45</td><td>30,587</td></tr><tr><td>Accumulated depreciation as at April 1, 2023</td><td>—</td><td>(4,535)</td><td>(2,437)</td><td>(1,198)</td><td>(5,826)</td><td>(1,675)</td><td>(1,032)</td><td>40)</td><td>(16,743)</td></tr><tr><td>Depreciation</td><td>i</td><td>(450)</td><td>(259)</td><td>(130)</td><td>(1,387)</td><td>(250)</td><td>(206)</td><td>()</td><td>(2,685)</td></tr><tr><td>Accumulated depreciation on deletions**</td><td>—</td><td>55</td><td>64</td><td>59</td><td>836</td><td>89</td><td>97</td><td></td><td>1,201</td></tr><tr><td>Translation difference</td><td>i</td><td>9</td><td>2</td><td></td><td>(©)</td><td>(1)</td><td>3</td><td></td><td>10</td></tr><tr><td>Accumulated depreciation as at March 31, 2024</td><td>—</td><td>(4,921)</td><td>(2,630)</td><td>(1,26;)</td><td>(6,380)</td><td>(1,837)</td><td>(1,138)</td><td>(4;)</td><td>(18,217)</td></tr><tr><td>Carrying value as at April 1, 2023</td><td>1,431</td><td>7,027</td><td>865</td><td>284</td><td>2,693</td><td>628</td><td>413</td><td></td><td>13,346</td></tr><tr><td>Carrying value as at March 31, 2024</td><td>1,432</td><td>6,849</td><td>798</td><td>259</td><td>2,231</td><td>489</td><td>309</td><td></td><td>12,370</td></tr></tbody></table></br></br><table><thead><tr><th rowspan="2">Particulars</th><th rowspan="2">Land - Freehold</th><th rowspan="2">Buildings 
 (1)</th><th rowspan="2">Plant and machinery</th><th rowspan="2">Office Equipment</th><th colspan="2"></th><th rowspan="2">Leasehold 
 Improvements</th><th colspan="2">(In % crore)</th></tr><tr><th>Computer 
 equipment</th><th>Furniture and fixtures</th><th>Vehicles</th><th>Total</th></tr></thead><tbody><tr><td>Gross carrying value as at April 1, 2022</td><td>1,431</td><td>11,224</td><td>3,210</td><td>1,427</td><td>8,527</td><td>2,278</td><td>1,234</td><td></td><td>29,375</td></tr><tr><td>Additions - Business Combination (Refer to Note 2.1)</td><td>—                 —                —</td><td></td><td></td><td>5</td><td>6</td><td>1</td><td>2</td><td></td><td>14</td></tr><tr><td>Additions</td><td></td><td>337</td><td>273</td><td>122</td><td>1,510</td><td>364</td><td>220</td><td></td><td>2,830</td></tr><tr><td>Deletions*</td><td>2</td><td>—</td><td>(182)</td><td>(76)</td><td>(1,563)</td><td>(348)</td><td>(25)</td><td>1)</td><td>(2,197)</td></tr><tr><td>Translation difference</td><td></td><td>1</td><td>1</td><td>4</td><td>39</td><td>8</td><td>14</td><td></td><td>67</td></tr><tr><td>Gross carrying value as at March 31, 2023</td><td>1,4;</td><td>11,562</td><td>3,302</td><td>1,482</td><td>8,519</td><td>2,303</td><td>1,445</td><td>45</td><td>30,089</td></tr><tr><td>Accumulated depreciation as at April 1, 2022</td><td></td><td>(4,100)</td><td>(2,344)</td><td>(1,150)</td><td>(6,034</td><td>(1,779)</td><td>(856)</td><td>37</td><td>(16,300)</td></tr><tr><td>Depreciation</td><td></td><td>(434)</td><td>(273)</td><td>(121)</td><td>(1,322)</td><td>(236)</td><td>(187)</td><td>4)</td><td>(2,577)</td></tr><tr><td>Accumulated depreciation on deletions*</td><td></td><td>—</td><td>181</td><td>76</td><td>1,556</td><td>347</td><td>21</td><td></td><td>2,182</td></tr><tr><td>Translation difference</td><td></td><td>(1)</td><td>(1)</td><td>(©)</td><td>(26)</td><td>(7)</td><td>(10)</td><td></td><td>(48)</td></tr><tr><td>Accumulated depreciation as at March 31, 2023</td><td></td><td>(4,535)</td><td>(2,437)</td><td>(1,198)</td><td>(5,826)</td><td>(1,675)</td><td>(1,032)</td><td>(40)</td><td>(16,743)</td></tr><tr><td>Carrying value as at April 1, 2022</td><td>1,4;</td><td>7,124</td><td>866</td><td>277</td><td>2,493</td><td>499</td><td>378</td><td></td><td>13,075</td></tr><tr><td>Carrying value as at March 31, 2023</td><td>1,431</td><td>7,027</td><td>865</td><td>284</td><td>2,693</td><td>628</td><td>413</td><td></td><td>13,346</td></tr></tbody></table></br></br><table><thead><tr><th rowspan="2">Particulars</th><th>As at</th><th></th></tr><tr><th>March 31, 2024</th><th>March 31, 2023</th></tr></thead><tbody><tr><td>Carrying value at the beginning</td><td>7,248</td><td>6,195</td></tr><tr><td>Goodwill on acquisitions (Refer to note 2.1)</td><td>—</td><td>630</td></tr><tr><td>Translation differences</td><td>55</td><td>423</td></tr><tr><td>Carrying value at the end</td><td>7,303</td><td>7,248</td></tr></tbody></table></br></br><table><thead><tr><th rowspan="2">Particulars</th><th>Asat</th><th>(4 N Crore,</th></tr><tr><th>March 31, 2024</th><th>March 31, 2023</th></tr></thead><tbody><tr><td colspan="3">Non-current Investments</td></tr><tr><td colspan="3">Unquoted</td></tr><tr><td colspan="3">Investments carried at fair value through other comprehensive income</td></tr><tr><td>Preference securities</td><td>91</td><td>193</td></tr><tr><td>Equity securities</td><td>2                                      3</td><td></td></tr><tr><td></td><td>93</td><td>196</td></tr><tr><td>Investments carried at fair value through profit or loss Target maturity fund units</td><td>431</td><td>402</td></tr><tr><td>Others</td><td>198</td><td>169</td></tr><tr><td></td><td>629</td><td>571</td></tr><tr><td colspan="3">Quoted</td></tr><tr><td>Investments carried at amortized cost</td><td></td><td></td></tr><tr><td>Government bonds Tax free bonds</td><td>28</td><td>28</td></tr><tr><td></td><td>1,731 1,759</td><td>1,742 1,770</td></tr><tr><td colspan="3">Investments carried at fair value through other comprehensive income</td></tr><tr><td>Non convertible debentures</td><td>2,217</td><td>2,713</td></tr><tr><td>Equity securities</td><td>113</td><td>—</td></tr><tr><td>Government securities</td><td>6,897 9,227</td><td>7,319 10,032</td></tr><tr><td colspan="3" rowspan="2">Total non-current investments 11,708 12,569 Current Investments</td></tr><tr><td colspan="3">Unquoted</td></tr><tr><td colspan="3">Investments carried at fair value through profit or loss</td></tr><tr><td>Liquid mutual fund units</td><td>2,615</td><td>975</td></tr><tr><td></td><td></td><td>975</td></tr><tr><td></td><td>2,615</td><td></td></tr><tr><td colspan="3">Investments carried at fair value through other comprehensive income</td></tr><tr><td>Commercial</td><td>4,830</td><td>742</td></tr><tr><td>Papers</td><td></td><td></td></tr><tr><td>Certificates of deposit</td><td>3,043</td><td>3,574</td></tr><tr><td></td><td>7,873</td><td>4,316</td></tr><tr><td>Quoted</td><td></td><td></td></tr><tr><td colspan="3">Investments carried at amortized cost</td></tr><tr><td>Tax free bonds</td><td>—</td><td>150</td></tr><tr><td colspan="3">150 Investments carried at fair value through other comprehensive income                                  —</td></tr><tr><td>Non convertible debentures</td><td>1,962</td><td>1,155</td></tr><tr><td>Government securities</td><td>465</td><td>313</td></tr><tr><td></td><td>2,427</td><td>1,468</td></tr><tr><td>Total current investments</td><td>12,915</td><td>6,909</td></tr><tr><td colspan="3">Total investments 24,623 19,478</td></tr><tr><td>Aggregate amount of quoted investments</td><td>13,413</td><td>13,420</td></tr><tr><td>Market value of quoted investments (including interest accrued), current</td><td>2,428</td><td>1,637</td></tr><tr><td>Market value of quoted investments (including interest accrued), non current</td><td>11,201</td><td>12,042</td></tr><tr><td>Aggregate amount of unquoted investments</td><td>11,210</td><td>6,058</td></tr><tr><td>Investments carried at amortized cost</td><td>1,759</td><td>1,920</td></tr><tr><td>Investments carried at fair value through other comprehensive income</td><td>19,620</td><td>16,012</td></tr><tr><td>Investments carried at fair value through profit or loss</td><td>3,244</td><td>1,546</td></tr></tbody></table></br></br><table><thead><tr><th rowspan="3">Class of investment</th><th rowspan="3">Method</th><th colspan="2">(In X crore)</th></tr><tr><th colspan="2">Fair value as at</th></tr><tr><th>March 31, 2024</th><th>March 31, 2023</th></tr></thead><tbody><tr><td>Liquid mutual fund units - carried at fair value through profit or loss</td><td>Quoted price</td><td>2,615</td><td>975</td></tr><tr><td>Target maturity fund units - carried at fair value through profit or loss</td><td>Quoted price</td><td>431</td><td>402</td></tr><tr><td>Tax free bonds and government bonds - carried at amortized cost</td><td>Quoted price and market observable inputs</td><td>1,973</td><td>2,148</td></tr><tr><td>Non-convertible debentures - carried at fair value through other comprehensive income</td><td>Quoted price and market observable inputs</td><td>4,179</td><td>3,868</td></tr><tr><td>Government securities - carried at fair value through other comprehensive income</td><td>Quoted price and market observable inputs</td><td>7,362</td><td>7,632</td></tr><tr><td>Commercial Papers - carried at fair value through other comprehensive income</td><td>Market observable inputs</td><td>4,830</td><td>742</td></tr><tr><td>Certificates of deposit - carried at fair value through other comprehensive income</td><td>Market observable inputs</td><td>3,043</td><td>3,574</td></tr><tr><td>Quoted Equity securities - carried at fair value through other comprehensive income</td><td>Quoted price</td><td>113</td><td>—</td></tr><tr><td>Unquoted equity and preference securities - carried at fair value through other comprehensive income</td><td>Discounted cash flows method, Market multiples method, Option pricing model</td><td>93</td><td>196</td></tr><tr><td>Others - carried at fair value through profit or loss</td><td>Discounted cash flows method, Market multiples method, Option pricing model</td><td>198</td><td>169</td></tr><tr><td>Total</td><td></td><td>24,837</td><td>19,706</td></tr></tbody></table></br></br><table><thead><tr><th>Particulars</th><th>Asat March 31, 2024</th><th>March 31, 2023</th></tr></thead><tbody><tr><td colspan="3">Non Current</td></tr><tr><td colspan="3">Loans considered good - Unsecured</td></tr><tr><td colspan="3">Other loans</td></tr><tr><td>Loans to employees</td><td>34</td><td>39</td></tr><tr><td></td><td>34</td><td>39</td></tr><tr><td colspan="3">Loans credit impaired - Unsecured</td></tr><tr><td colspan="3">Other loans</td></tr><tr><td>Loans to employees</td><td>2</td><td>2</td></tr><tr><td>Less: Allowance for credit impairment</td><td>()</td><td>(2</td></tr><tr><td>Total non-current loans</td><td>34                                  —</td><td>39                               —</td></tr><tr><td colspan="3">Current</td></tr><tr><td colspan="3">Loans considered good - Unsecured</td></tr><tr><td colspan="3">Other loans</td></tr><tr><td>Loans to employees</td><td>248</td><td>289</td></tr><tr><td>Total current loans</td><td>248</td><td>289</td></tr><tr><td>Total loans</td><td>282</td><td>328</td></tr></tbody></table></br></br><table><thead><tr><th rowspan="2">Particulars</th><th>Asat</th><th></th></tr><tr><th>March 31, 2024</th><th>March 31, 2023</th></tr></thead><tbody><tr><td colspan="3">Non Current</td></tr><tr><td>Security deposits</td><td>259</td><td>287</td></tr><tr><td># Unbilled revenues</td><td>1,677</td><td>1,185</td></tr><tr><td>) Net investment in sublease of right-of-use asset a</td><td>3</td><td>305</td></tr><tr><td>Restricted deposits "</td><td>47</td><td>96</td></tr><tr><td>Others</td><td>1,119</td><td>925</td></tr><tr><td>Total non-current other financial assets</td><td>3,105</td><td>2,798</td></tr><tr><td colspan="3">Current</td></tr><tr><td>Security deposits</td><td>75</td><td>42</td></tr><tr><td>Restricted deposits V"</td><td>2,535</td><td>2,348</td></tr><tr><td># Unbilled revenues</td><td>7,923</td><td>8,317</td></tr><tr><td>Interest accrued but not due</td><td>537</td><td>488</td></tr><tr><td>Foreign currency forward and options contracts</td><td>84</td><td>101</td></tr><tr><td>Net investment in sublease of right of-use-asset</td><td>6</td><td>53</td></tr><tr><td>Others V™"</td><td>925</td><td>255</td></tr><tr><td>Total current other financial assets</td><td>12,085</td><td>11,604</td></tr><tr><td>Total other financial assets</td><td>15,190</td><td>14,402</td></tr><tr><td>O Financial assets carried at amortized cost</td><td>15,106</td><td>14,301</td></tr><tr><td>@ Financial assets carried at fair value through other comprehensive income</td><td>23</td><td>32</td></tr><tr><td>) Financial assets carried at fair value through profit or loss</td><td>61</td><td>69</td></tr></tbody></table></br></br><table><thead><tr><th rowspan="2">Particulars</th><th>Asat</th><th>(In % crore)</th></tr><tr><th>March 31, 2024</th><th>March 31, 2023</th></tr></thead><tbody><tr><td colspan="3">Current</td></tr><tr><td>Trade Receivable considered good - Unsecured</td><td>30,713</td><td>25,965</td></tr><tr><td>Less: Allowance for expected credit loss</td><td>520</td><td>541</td></tr><tr><td>Trade Receivable considered good - Unsecured</td><td>30,193</td><td>25,424</td></tr><tr><td>Trade Receivable - credit impaired - Unsecured</td><td>196</td><td>142</td></tr><tr><td>Less: Allowance for credit impairment</td><td>196</td><td>142</td></tr><tr><td>Trade Receivable - credit impaired - Unsecured</td><td>.</td><td>i</td></tr><tr><td>Total trade receivables</td><td>30,193</td><td>25,424</td></tr></tbody></table></br></br><table><thead><tr><th>Particulars</th><th>Asat March 31, 2024</th><th>(In % crore) March 31, 2023</th></tr></thead><tbody><tr><td colspan="3">Balances with banks</td></tr><tr><td>In current and deposit accounts</td><td>14,786</td><td>10,026</td></tr><tr><td>Cash on hand</td><td>—</td><td>—</td></tr><tr><td colspan="3">Others</td></tr><tr><td>Deposits with financial institutions</td><td>—</td><td>2,147</td></tr><tr><td>Total cash and cash equivalents</td><td>14,786</td><td>12,173</td></tr><tr><td>Balances with banks in unpaid dividend accounts</td><td>37</td><td>37</td></tr><tr><td>Deposit with more than 12 months maturity</td><td>57</td><td>833</td></tr></tbody></table></br></br><table><thead><tr><th rowspan="2">Particulars</th><th>Asat</th><th>(In % crore)</th></tr><tr><th>March 31, 2024</th><th>March 31, 2023</th></tr></thead><tbody><tr><td colspan="3">Non-current</td></tr><tr><td>Capital advances</td><td>155</td><td>159</td></tr><tr><td colspan="3">Advances other than capital advances</td></tr><tr><td colspan="3">Others</td></tr><tr><td>Withholding taxes and others</td><td>673</td><td>684</td></tr><tr><td>Unbilled revenues *</td><td>103</td><td>264</td></tr><tr><td>Defined benefit plan assets</td><td>31</td><td>36</td></tr><tr><td>Prepaid expenses</td><td>343</td><td>332</td></tr><tr><td colspan="3">Deferred Contract Cost</td></tr><tr><td>Cost of obtaining a contract *</td><td>129</td><td>191</td></tr><tr><td>Cost of fulfillment</td><td>687</td><td>652</td></tr><tr><td>Other receivables</td><td>—                                   —</td><td></td></tr><tr><td>Total non-current other assets</td><td>2,121</td><td>2,318</td></tr><tr><td colspan="3">Current</td></tr><tr><td colspan="3">Advances other than capital advances</td></tr><tr><td>Payment to vendors for supply of goods</td><td>356</td><td>202</td></tr><tr><td colspan="3">Others</td></tr><tr><td>Unbilled revenues *</td><td>4,845</td><td>6,972</td></tr><tr><td>Withholding taxes and others</td><td>3,540</td><td>3,268</td></tr><tr><td>Prepaid expenses</td><td>3,329</td><td>2,745</td></tr><tr><td colspan="3">Deferred Contract Cost</td></tr><tr><td>Cost of obtaining a contract *</td><td>200</td><td>853</td></tr><tr><td>Cost of fulfillment</td><td>358</td><td>175</td></tr><tr><td>Other receivables</td><td>180</td><td>261</td></tr><tr><td>Total current other assets</td><td>12,808</td><td>14,476</td></tr><tr><td>Total other assets</td><td>14,929</td><td>16,794</td></tr></tbody></table></br></br><table><thead><tr><th rowspan="2">Particulars</th><th rowspan="2">Amortized cost</th><th colspan="2">Financial assets/ liabilities at fair value through profit or loss</th><th colspan="2">Financial assets/liabilities at fair value through OCI</th><th rowspan="2">Total carrying value</th><th rowspan="2">Total fair v'alue</th></tr><tr><th>Designated 
 upon initial recognition</th><th>Mandatory</th><th>Equity instruments designated upon initial recognition</th><th>Mandatory</th></tr></thead><tbody><tr><td colspan="8">Assets:</td></tr><tr><td>Cash and cash equivalents (Refer to Note 2.8)</td><td>14,786</td><td>—</td><td></td><td></td><td></td><td>14,786</td><td>14,786</td></tr><tr><td colspan="8">Investments (Refer to Note 2.4)</td></tr><tr><td>Equity and preference securities</td><td>—</td><td>—</td><td></td><td>206</td><td></td><td>206</td><td>206</td></tr><tr><td>Tax free bonds and government bonds</td><td>1,759</td><td>—</td><td></td><td></td><td></td><td>,759</td><td>973 (1</td></tr><tr><td>Liquid mutual fund units</td><td>—</td><td></td><td>2,615</td><td></td><td></td><td>2,615</td><td>2,615</td></tr><tr><td>Target maturity fund units</td><td>—</td><td></td><td>431</td><td></td><td></td><td>431</td><td>431</td></tr><tr><td>Non convertible debentures</td><td>—</td><td>—</td><td></td><td></td><td>4,179</td><td>4,179</td><td>4,179</td></tr><tr><td>Government securities</td><td>—</td><td>—</td><td></td><td></td><td>7,362</td><td>7,362</td><td>7,362</td></tr><tr><td>Commercial papers</td><td>—</td><td>—</td><td></td><td></td><td>4,830</td><td>4,830</td><td>4,830</td></tr><tr><td>Certificates of deposit</td><td>—</td><td>—</td><td></td><td></td><td>3,043</td><td>3,043</td><td>3,043</td></tr><tr><td>Other investments</td><td>—</td><td></td><td>198</td><td></td><td></td><td>98</td><td>98</td></tr><tr><td>Trade receivables (Refer to Note 2.7)</td><td>30,193</td><td>—</td><td></td><td></td><td></td><td>30,193</td><td>30,193</td></tr><tr><td>Loans (Refer to Note 2.5)</td><td>282</td><td>—</td><td></td><td></td><td></td><td>282</td><td>282</td></tr><tr><td>Other financials assets (Refer to Note 2.6) @</td><td>15,106</td><td></td><td>61</td><td></td><td>23</td><td>15,190</td><td>= 15,106</td></tr><tr><td>Total</td><td>62,126</td><td></td><td>3,305</td><td>206</td><td>19,437</td><td>85,074</td><td>85,204</td></tr><tr><td colspan="8">Liabilities:</td></tr><tr><td>Trade payables</td><td>3,956</td><td>—</td><td></td><td></td><td></td><td>3,956</td><td>3,956</td></tr><tr><td>Lease liabilities (Refer to Note 2.19)</td><td>8,359</td><td>—</td><td></td><td></td><td></td><td>8,359</td><td>8,359</td></tr><tr><td>Financial Liability under option arrangements (Refer to Note 2.12)</td><td>—</td><td></td><td>597</td><td></td><td></td><td>597</td><td>597</td></tr><tr><td>Other financial liabilities (Refer to Note 2.12)</td><td>15,750</td><td></td><td>30</td><td></td><td></td><td>15,781</td><td>15,781</td></tr><tr><td>Total</td><td>28,065</td><td></td><td>627</td><td></td><td></td><td>28,693</td><td>28,693</td></tr></tbody></table></br></br><table><thead><tr><th rowspan="3">Particulars</th><th rowspan="3">Amortized 
 cost</th><th colspan="2" rowspan="2">Financial assets/ liabilities at fair value through profit or loss</th><th></th><th></th><th></th><th rowspan="3">(In X crore) Total fair value</th></tr><tr><th colspan="2">Financial assets/liabilities at fair value through OCI</th><th rowspan="2">Total carrying value</th></tr><tr><th>Designated 
 upon initial recognition</th><th>Mandatory</th><th>Equity instruments designated upon initial recognition</th><th>Mandatory</th></tr></thead><tbody><tr><td colspan="8">Assets:</td></tr><tr><td>Cash and cash equivalents (Refer to Note 2.8)</td><td>12,173</td><td>—</td><td></td><td></td><td></td><td>12,173</td><td>12,173</td></tr><tr><td colspan="8">Investments (Refer to Note 2.4)</td></tr><tr><td>Equity and preference securities</td><td>—</td><td>—</td><td></td><td>196</td><td></td><td>196</td><td>196</td></tr><tr><td>Tax free bonds and government bonds</td><td>1,920</td><td>—</td><td></td><td></td><td></td><td>1,920</td><td>(1) 2,148</td></tr><tr><td>Liquid mutual fund units</td><td>—</td><td></td><td>975</td><td></td><td></td><td>975</td><td>975</td></tr><tr><td>Target maturity fund units</td><td>—</td><td></td><td>402</td><td></td><td></td><td>402</td><td>402</td></tr><tr><td>Non convertible debentures</td><td>—</td><td>—</td><td></td><td></td><td>3,868</td><td>3,868</td><td>3,868</td></tr><tr><td>Government securities</td><td>—</td><td>—</td><td></td><td></td><td>7,632</td><td>7,632</td><td>7,632</td></tr><tr><td>Commercial papers</td><td>—</td><td>—</td><td></td><td></td><td>742</td><td>742</td><td>742</td></tr><tr><td>Certificates of deposit</td><td>—</td><td>—</td><td></td><td></td><td>3,574</td><td>3,574</td><td>3,574</td></tr><tr><td>Other investments</td><td>—</td><td></td><td>169</td><td></td><td></td><td>169</td><td>169</td></tr><tr><td>Trade receivables (Refer to Note 2.7)</td><td>25,424</td><td>—</td><td></td><td></td><td></td><td>25,424</td><td>25,424</td></tr><tr><td>Loans (Refer to Note 2.5)</td><td>328</td><td>—</td><td></td><td></td><td></td><td>328</td><td>328</td></tr><tr><td>Other financials assets (Refer to Note 2.6) @</td><td>14,301</td><td></td><td>69</td><td></td><td>32</td><td>14,402</td><td>14,318 (2)</td></tr><tr><td>Total</td><td>54,146</td><td></td><td>1,615</td><td>196</td><td>15,848</td><td>71,805</td><td>71,949</td></tr><tr><td colspan="8">Liabilities:</td></tr><tr><td>Trade payables</td><td>3,865</td><td>—</td><td></td><td></td><td></td><td>3,865</td><td>3,865</td></tr><tr><td>Lease liahilities (Refer to Note 2.19)</td><td>8,299</td><td>—</td><td></td><td></td><td></td><td>8,299</td><td>8,299</td></tr><tr><td>Financial Liability under option arrangements Refer to Note 2.12)</td><td>.</td><td></td><td>600</td><td></td><td></td><td>600</td><td>600</td></tr><tr><td>Other financial liabilities (Refer to Note 2.12)</td><td>17,359</td><td></td><td>161</td><td></td><td>14</td><td>17,534</td><td>17,534</td></tr><tr><td>Total</td><td>29,523</td><td></td><td>761</td><td></td><td>14</td><td>30,298</td><td>30,298</td></tr></tbody></table></br></br><table><thead><tr><th rowspan="2">Particulars</th><th rowspan="2">Asat March 31, 2024</th><th colspan="3">Fair value measurement at end of the reporting period using</th></tr><tr><th>Level 1</th><th>Level 2</th><th>Level 3</th></tr></thead><tbody><tr><td colspan="5">Assets</td></tr><tr><td colspan="5">Investments (Refer to note 2.4)</td></tr><tr><td>nvestments in liquid mutual funds</td><td>2,615</td><td>2,615</td><td></td><td>—</td></tr><tr><td>nvestments in target maturity fund units</td><td>431</td><td>431</td><td></td><td>—</td></tr><tr><td>nvestments in tax free bonds</td><td>1,944</td><td>1,944</td><td></td><td>—</td></tr><tr><td>nvestments in government bonds</td><td>29</td><td>29</td><td></td><td>—</td></tr><tr><td>nvestments in non convertible debentures</td><td>4,179</td><td>3,922</td><td>257</td><td>—</td></tr><tr><td>nvestment in government securities</td><td>7,362</td><td>7,289</td><td>73</td><td>—</td></tr><tr><td>nvestments in equity securities</td><td>15</td><td></td><td></td><td>115</td></tr><tr><td>nvestments in preference securities</td><td>91</td><td></td><td></td><td>91</td></tr><tr><td>nvestments in commercial papers</td><td>4,830</td><td></td><td>4,830</td><td>—</td></tr><tr><td>nvestments in certificates of deposit</td><td>3,043</td><td></td><td>3,043</td><td>—</td></tr><tr><td>Other investments</td><td>98</td><td></td><td></td><td>198</td></tr><tr><td colspan="5">Others</td></tr><tr><td>Derivative financial instruments - gain on outstanding foreign exchange forward and option</td><td>84</td><td></td><td>84</td><td></td></tr><tr><td>contracts (Refer to Note 2.6)</td><td></td><td>—</td><td></td><td>—</td></tr><tr><td colspan="5">Liabilities</td></tr><tr><td>Derivative financial instruments - loss on outstanding foreign exchange forward and option</td><td>31</td><td></td><td>31</td><td></td></tr><tr><td colspan="5">contracts (Refer to Note 2.12)                         — 
                             —</td></tr><tr><td>Financial liability under option arrangements (Refer to Note 2.12) M</td><td>597</td><td></td><td></td><td>597</td></tr></tbody></table></br></br><table><thead><tr><th rowspan="2">Particulars</th><th rowspan="2">Asat March 31, 2023</th><th colspan="3">Fair value measurement at end of the reporting period using</th></tr><tr><th>Level 1</th><th>Level 2</th><th>Level 3</th></tr></thead><tbody><tr><td colspan="5">Assets</td></tr><tr><td colspan="5">Investments (Refer to note 2.4)</td></tr><tr><td>Investments in liquid mutual funds</td><td>975</td><td>975</td><td></td><td></td></tr><tr><td>nvestments in target maturity fund units</td><td>402</td><td>402</td><td></td><td></td></tr><tr><td>Investments in tax free bonds</td><td>2,120</td><td>1,331</td><td>789</td><td></td></tr><tr><td>nvestments in government bonds</td><td>28</td><td>28</td><td></td><td></td></tr><tr><td>Investments in non convertible debentures</td><td>3,868</td><td>1,793</td><td>2,075</td><td></td></tr><tr><td>nvestment in government securities</td><td>7,632</td><td>7,549</td><td>83</td><td></td></tr><tr><td>Investments in equity securities</td><td></td><td>3                             —                                 —                                    3</td><td></td><td></td></tr><tr><td>nvestments in preference securities</td><td>193</td><td></td><td></td><td>193</td></tr><tr><td>Investments in commercial papers</td><td>742</td><td></td><td>742</td><td>—</td></tr><tr><td>nvestments in certificates of deposit</td><td>3,574</td><td></td><td>3,574</td><td>—</td></tr><tr><td>Other investments</td><td>169</td><td></td><td></td><td>169</td></tr><tr><td colspan="5">Others</td></tr><tr><td>Derivative financial instruments - gain on outstanding foreign exchange forward and option contracts (Refer to Note 2.6)</td><td>101</td><td></td><td>101</td><td></td></tr><tr><td colspan="5">Liabilities</td></tr><tr><td>Derivative financial instruments - loss on outstanding foreign exchange forward and option contracts (Refer to Note 2.12)</td><td>78</td><td></td><td>78</td><td></td></tr><tr><td>Financial liability under option arrangements (Refer to Note 2.12) M</td><td>600</td><td></td><td></td><td>600</td></tr><tr><td>Liability towards contingent consideration (Refer to Note 2.12)</td><td>97</td><td></td><td></td><td>97</td></tr></tbody></table></br></br><table><thead><tr><th rowspan="2">Particulars</th><th colspan="2">(In &lt; crore, except as otherwise stated) As at</th></tr><tr><th>March 31, 2024</th><th>March 31, 2023</th></tr></thead><tbody><tr><td colspan="3">Authorized</td></tr><tr><td colspan="3">Equity shares, Z5/- par value</td></tr><tr><td>4,80,00,00,000 (4,80,00,00,000) equity shares</td><td>2,400</td><td>2,400</td></tr><tr><td colspan="3">Issued, Subscribed and Paid-Up</td></tr><tr><td>Equity shares, Z5/- par value®</td><td>2,071</td><td>2,069</td></tr><tr><td colspan="3">4,13,99,50,635 (4,13,63,87,925) equity shares fully paid-up®</td></tr><tr><td></td><td>2,071</td><td>2,069</td></tr></tbody></table></br></br><table><thead><tr><th rowspan="2">Particulars</th><th colspan="2">Asat March 31, 2024</th><th colspan="2">(In &lt; crore, except as stated otherwise) As at March 31, 2023</th></tr><tr><th>Number of shares</th><th>Amount</th><th>Number of shares</th><th>Amount</th></tr></thead><tbody><tr><td>As at the beginning of the period</td><td>413,63,87,925</td><td>2,069</td><td>419,30,12,929</td><td>2,098</td></tr><tr><td>Add: Shares issued on exercise of employee stock options</td><td>35,62,710</td><td>2</td><td>38,01,344</td><td>1</td></tr><tr><td>Less: Shares bought back</td><td>—</td><td>—</td><td>6,04,26,348</td><td>30</td></tr><tr><td>As at the end of the period</td><td>413,99,50,635</td><td>2,071</td><td>413,63,87,925</td><td>2,069</td></tr></tbody></table></br></br><table><thead><tr><th rowspan="2">Particulars</th><th colspan="2">Three months ended March 31,</th><th colspan="2">Y ear ended March 31,</th></tr><tr><th></th><th></th><th></th><th>2023</th></tr></thead><tbody><tr><td>Final dividend for fiscal 2022</td><td>20%                          —</td><td>20?</td><td>20274</td><td>16.00</td></tr><tr><td>Interim dividend for fiscal 2023</td><td>—</td><td>—</td><td>—</td><td>16.50</td></tr><tr><td>Final dividend for fiscal 2023</td><td>—</td><td>—</td><td>17.50</td><td></td></tr><tr><td>Interim dividend for fiscal 2024</td><td>—</td><td>—</td><td>18.00</td><td></td></tr></tbody></table></br></br><table><thead><tr><th rowspan="3">Particulars</th><th colspan="4">2019 Plan</th><th colspan="4">2015 Plan</th></tr><tr><th colspan="2">Three months ended March 31,</th><th colspan="2">Y ear ended March 31,</th><th colspan="2">Three months ended March 31,</th><th colspan="2">Year ended March 31,</th></tr><tr><th>2024</th><th>2023</th><th>2024</th><th>2023</th><th>2024</th><th>2023</th><th>2024</th><th>2023</th></tr></thead><tbody><tr><td colspan="9">Equity Settled RSUs</td></tr><tr><td>Key Management Personnel (KMP)</td><td>26,900</td><td>33,750</td><td>141,171</td><td>210,643</td><td>77,094</td><td>80,154</td><td>498,730</td><td>367,479</td></tr><tr><td>Employees other than KMP</td><td>3,582,471</td><td>3,329,240</td><td>4,046,731</td><td>3,704,014</td><td>3,442,700</td><td>1,736,925</td><td>4,640,640</td><td>1,784,975</td></tr><tr><td></td><td>3,609,371</td><td>3,362,990</td><td>4,187,902</td><td>3,914,657</td><td>3,519,794</td><td>1,817,079</td><td>5,139,370</td><td>2,152,454</td></tr><tr><td colspan="9">Cash settled RSU</td></tr><tr><td>Key Management Personnel (KMP)</td><td></td><td></td><td>-                     -                      -</td><td></td><td></td><td></td><td>-                              -                          -                                  -                                  -</td><td></td></tr><tr><td>Employees other than KMP</td><td></td><td></td><td>-                     -                      -</td><td></td><td>169,04(;                   -                   169,040</td><td>92,40(;</td><td>176,99(;               92,400                      176,990                        92,400</td><td>92,40(;</td></tr><tr><td></td><td>-</td><td></td><td>-                      -</td><td></td><td>169,040</td><td>92,400</td><td>176,990</td><td>92,400</td></tr><tr><td>Total Grants</td><td>3,609,371</td><td>3,362,990</td><td>4,187,902</td><td>3,914,657</td><td>3,688,834</td><td>1,909,479</td><td>5,316,360</td><td>2,244,854</td></tr></tbody></table></br></br><table><thead><tr><th rowspan="2">Particulars</th><th colspan="2" rowspan="2">Three months ended March 31, 2024
 2023</th><th colspan="2" rowspan="2">(in X crore) Year ended March 31, 2024
 2023</th></tr></thead><tbody><tr><td colspan="5">Granted to:</td></tr><tr><td>KMP</td><td>17</td><td>8</td><td>68</td><td>49</td></tr><tr><td>Employees other than KMP</td><td>208</td><td>125</td><td>584</td><td>470</td></tr><tr><td>Total ®</td><td>225</td><td>133</td><td>652</td><td>519</td></tr><tr><td>C ash-settled stock compensation expense included in the above</td><td>4</td><td>2</td><td>13</td><td>5</td></tr></tbody></table></br></br><table><thead><tr><th rowspan="2">Particulars</th><th colspan="4">For options granted in</th></tr><tr><th>Fiscal 2024- Equity Shares- RSU</th><th>Fiscal 2024- ADS-RSU</th><th>Fiscal 2023- Equity Shares-RSU</th><th>Fiscal 2023- ADS-RSU</th></tr></thead><tbody><tr><td>Weighted average share price (%) / ($ ADS)</td><td>1,588</td><td>19.19</td><td>1,525</td><td>18.08</td></tr><tr><td>Exercise price (%) / (§ ADS)</td><td>5.00</td><td>0.07</td><td>5.00</td><td>0.07</td></tr><tr><td>Expected volatility (%)</td><td>23-31</td><td>25-33</td><td>23-32</td><td>27-34</td></tr><tr><td>Expected life of the option (years)</td><td>1-4</td><td>1-4</td><td>1-4</td><td>1-4</td></tr><tr><td>Expected dividends (%)</td><td>2-3</td><td>2-3</td><td>2-3</td><td>2-3</td></tr><tr><td>Risk-free interest rate (%)</td><td>7</td><td>4-5</td><td>5-7</td><td>2-5</td></tr><tr><td>Weighted average fair value as on grant date () / ($ ADS)</td><td>1,317</td><td>16.27</td><td>1,210</td><td>13.69</td></tr></tbody></table></br></br><table><thead><tr><th rowspan="3">Particulars</th><th colspan="2">(In X crore)</th></tr><tr><th>Asat</th><th></th></tr><tr><th>March 31, 2024</th><th>March 31, 2023</th></tr></thead><tbody><tr><td colspan="3">Non-current</td></tr><tr><td colspan="3">Others</td></tr><tr><td>Accrued compensation to employees m</td><td>7                                       5</td><td></td></tr><tr><td>(6] Accrued expenses</td><td>1,779</td><td>1,628</td></tr><tr><td>Compensated absences</td><td>89</td><td>83</td></tr><tr><td>Financial liability under option arrangements B</td><td>98</td><td></td></tr><tr><td>Other Payables (V@</td><td>157</td><td>342</td></tr><tr><td>Total non-current other financial liabilities</td><td>2,130</td><td>2,058</td></tr><tr><td>Current</td><td></td><td></td></tr><tr><td>Unpaid dividends @</td><td>37</td><td>37</td></tr><tr><td colspan="3">Others</td></tr><tr><td>Accrued compensation to employees ¥</td><td>4,454</td><td>4,174</td></tr><tr><td>(6] Accrued expenses</td><td>8,224</td><td>7,802</td></tr><tr><td>Payable for acquisition of business - Contingent consideration !</td><td></td><td>97</td></tr><tr><td>Payable by controlled trusts</td><td>21</td><td>211</td></tr><tr><td>Compensated absences</td><td>2,622</td><td>2,399</td></tr><tr><td>Financial liability under option arrangements B</td><td>499</td><td>600</td></tr><tr><td>Foreign currency forward and options contracts</td><td></td><td>78</td></tr><tr><td>Capital creditors ¥</td><td>310</td><td>674</td></tr><tr><td>Other payables V@</td><td>57</td><td>2,486</td></tr><tr><td colspan="3">Total current other financial liabilities 16,959 18,558</td></tr><tr><td>Total other financial liabilities</td><td>19,089</td><td>20,616</td></tr><tr><td>O Financial liability carried at amortized cost</td><td>15,750</td><td>17,359</td></tr><tr><td>@ Financial liability carried at fair value through profit or loss</td><td>627</td><td>761</td></tr><tr><td>) Financial liability carried at fair value through other comprehensive income</td><td></td><td>14</td></tr><tr><td>Financial liability under option arrangements on an undiscounted basis</td><td>690</td><td>676</td></tr><tr><td>Contingent consideration on undiscounted basis</td><td></td><td>101</td></tr></tbody></table></br></br><table><thead><tr><th rowspan="2">Particulars</th><th>Asat March 2024</th><th>(In % crore) March 2023</th></tr><tr><th>31,</th><th>31,</th></tr></thead><tbody><tr><td colspan="3">Non-current</td></tr><tr><td>Others</td><td></td><td></td></tr><tr><td>Accrued defined benefit liability</td><td>159</td><td>445</td></tr><tr><td>Others</td><td>76</td><td>55</td></tr><tr><td>Total non-current other liabilities</td><td>235</td><td>500</td></tr><tr><td colspan="3">Current</td></tr><tr><td>Unearned revenue</td><td>7,341</td><td>7,163</td></tr><tr><td colspan="3">Others</td></tr><tr><td>Withholding taxes and others</td><td>3,185</td><td>3,632</td></tr><tr><td>Accrued defined benefit liability</td><td>5</td><td>4</td></tr><tr><td>Others</td><td>8</td><td>31</td></tr><tr><td>Total current other liabilities</td><td>10,539</td><td>10,830</td></tr><tr><td>Total other liabilities</td><td>10,774</td><td>11,330</td></tr></tbody></table></br></br><table><thead><tr><th rowspan="2">Particulars</th><th>Asat</th><th>(In % crore)</th></tr><tr><th>March 31, 2024</th><th>March 31, 2023</th></tr></thead><tbody><tr><td colspan="3">Current</td></tr><tr><td colspan="3">Others</td></tr><tr><td>Post-sales client support and other provisions</td><td>1,796</td><td>1,307</td></tr><tr><td>Total provisions</td><td>1,796</td><td>1,307</td></tr></tbody></table></br></br><table><thead><tr><th rowspan="2">Particulars</th><th colspan="2">Three months ended March 31,</th><th>Y ear ended March</th><th>(In % crore) 31,</th></tr><tr><th>2024</th><th>2023</th><th>2024</th><th>2023</th></tr></thead><tbody><tr><td>Current taxes</td><td>1,173</td><td>2,260</td><td>8,390</td><td>9,287</td></tr><tr><td>Deferred taxes</td><td>1,092</td><td>72</td><td>1,350</td><td>(73)</td></tr><tr><td>Income tax expense</td><td>2,265</td><td>2,332</td><td>9,740</td><td>9,214</td></tr></tbody></table></br></br><table><thead><tr><th rowspan="3">Particulars</th><th colspan="2" rowspan="2">Three months ended March 31,</th><th>Y ear ended</th><th>(In % crore)</th></tr><tr><th colspan="2">March 31,</th></tr><tr><th>2024</th><th>2023</th><th>2024</th><th>2023</th></tr></thead><tbody><tr><td>Revenue from software services</td><td>36,064</td><td>35,199</td><td>145,285</td><td>137,575</td></tr><tr><td>Revenue from products and platforms</td><td>1,859</td><td>2,242</td><td>8,385</td><td>9,192</td></tr><tr><td>Total revenue from operations</td><td>37,923</td><td>37,441</td><td>153,670</td><td>146,767</td></tr></tbody></table></br></br><table><thead><tr><th rowspan="2">Particulars</th><th colspan="2">Three months ended March 31,</th><th colspan="2">(In X crore) Y ear ended March 31,</th></tr><tr><th>2024</th><th>2023</th><th>2024</th><th>2023</th></tr></thead><tbody><tr><td colspan="5">Revenues by G eography*</td></tr><tr><td>North America</td><td>22,606</td><td>22,842</td><td>92,411</td><td>90,724</td></tr><tr><td>Europe</td><td>10,861</td><td>10,088</td><td>42,267</td><td>37,675</td></tr><tr><td>India</td><td>833</td><td>981</td><td>3,881</td><td>3,861</td></tr><tr><td>Rest of the world</td><td>3,623</td><td>3,530</td><td>15,111</td><td>14,507</td></tr><tr><td>Total</td><td>37,923</td><td>37,441</td><td>153,670</td><td>146,767</td></tr></tbody></table></br></br><table><thead><tr><th rowspan="2">Particulars</th><th colspan="2">Three months ended March 31,</th><th colspan="2">Y ear ended March 31,</th></tr><tr><th>2024</th><th>2023</th><th>2024</th><th>2023</th></tr></thead><tbody><tr><td colspan="5">Interest income on financial assets carried at amortized cost</td></tr><tr><td>Tax free bonds and Government bonds</td><td>31</td><td>36</td><td>131</td><td>149</td></tr><tr><td>Deposit with Bank and others</td><td>222</td><td>161</td><td>929</td><td>712</td></tr><tr><td colspan="5">Interest income on financial assets carried at fair value through other comprehensive income</td></tr><tr><td>Non-convertible debentures, commercial papers, certificates of deposit and government securities</td><td>318</td><td>231</td><td>1,007</td><td>955</td></tr><tr><td colspan="5">Income on investments carried at fair value through profit or loss</td></tr><tr><td>Dividend income on liquid mutual funds</td><td>—                          —                               —</td><td></td><td></td><td>—</td></tr><tr><td>Gain / (loss) on liquid mutual funds and other investments</td><td>88</td><td>61</td><td>285</td><td>148</td></tr><tr><td>Income on investments carried at fair value through other comprehensive income</td><td>—                          —                               —</td><td></td><td></td><td>1</td></tr><tr><td>Interest on income tax refund</td><td>1,916</td><td></td><td>1,965</td><td>3</td></tr><tr><td>Exchange gains / (losses) on forward and options contracts</td><td>190</td><td>142</td><td>100</td><td>(647)</td></tr><tr><td>Exchange gains / (losses) on translation of other assets and liabilities</td><td>(123)</td><td>(91)</td><td>87</td><td>1,062</td></tr><tr><td>Miscellaneous income, net</td><td>87</td><td>129</td><td>207</td><td>318</td></tr><tr><td>Total other income</td><td>2,729</td><td>671</td><td>4,711</td><td>2,701</td></tr></tbody></table></br></br><table><thead><tr><th rowspan="2">Particulars</th><th colspan="2">Three months ended March 31,</th><th colspan="2">Y ear ended March 31,</th></tr><tr><th>2024</th><th>2023</th><th>2024</th><th>2023</th></tr></thead><tbody><tr><td colspan="5">Employee benefit expenses</td></tr><tr><td>Salaries including bonus</td><td>19,527</td><td>19,526</td><td>79,315</td><td>75,239</td></tr><tr><td>Contribution to provident and other funds</td><td>529</td><td>547</td><td>2,213</td><td>2,143</td></tr><tr><td>Share based payments to employees (Refer to Note 2.11)</td><td>225</td><td>133</td><td>652</td><td>519</td></tr><tr><td>Staff welfare</td><td>112</td><td>105</td><td>440</td><td>458</td></tr><tr><td></td><td>20,393</td><td>20,311</td><td>82,620</td><td>78,359</td></tr><tr><td colspan="5">Cost of software packages and others</td></tr><tr><td>For own use</td><td>555</td><td>496</td><td>2,145</td><td>1,937</td></tr><tr><td>Third party items bought for service delivery to clients</td><td>3,132</td><td>2,390</td><td>11,370</td><td>8,965</td></tr><tr><td></td><td>3,687</td><td>2,886</td><td>13,515</td><td>10,902</td></tr><tr><td colspan="5">Other expenses</td></tr><tr><td>Repairs and maintenance</td><td>316</td><td>331</td><td>1,278</td><td>1,208</td></tr><tr><td>Power and fuel</td><td>49</td><td>46</td><td>199</td><td>176</td></tr><tr><td>Brand and marketing</td><td>285</td><td>265</td><td>1,007</td><td>905</td></tr><tr><td>Rates and taxes</td><td>84</td><td>78</td><td>326</td><td>299</td></tr><tr><td>Consumables</td><td>47</td><td>41</td><td>170</td><td>158</td></tr><tr><td>nsurance</td><td>53</td><td>43</td><td>210</td><td>174</td></tr><tr><td>Provision for post-sales client support and others</td><td>(129)</td><td>(80)</td><td>75</td><td>120</td></tr><tr><td>Commission to non-whole time directors</td><td></td><td></td><td>16</td><td>15</td></tr><tr><td>mpairment loss recognized / (reversed) under expected credit loss model</td><td>(98)</td><td>86</td><td>121</td><td>283</td></tr><tr><td>Contributions towards Corporate Social Responsibility</td><td>182</td><td>151</td><td>533</td><td>471</td></tr><tr><td>Others</td><td>191</td><td>181</td><td>781</td><td>583</td></tr><tr><td></td><td>985</td><td>1,146</td><td>4,716</td><td>4,392</td></tr></tbody></table></br></br><table><thead><tr><th rowspan="2">Particulars</th><th colspan="4">Category of ROU asset</th><th rowspan="2">Total</th></tr><tr><th>Land</th><th>Buildings</th><th>V ehicles</th><th>Computers</th></tr></thead><tbody><tr><td>Balance as of J anuary 1, 2024</td><td>607</td><td>3,527</td><td>18</td><td>2,740</td><td>6,892</td></tr><tr><td>Additions*</td><td></td><td>61</td><td></td><td>376</td><td>439</td></tr><tr><td>Deletions</td><td></td><td>(92)</td><td></td><td>(215)</td><td>(307)</td></tr><tr><td>Impairment#</td><td></td><td>_</td><td></td><td>—</td><td>—</td></tr><tr><td>Depreciation</td><td>()</td><td>(185)</td><td>()</td><td>(234)</td><td>(423)</td></tr><tr><td>Translation difference</td><td></td><td>(13)</td><td>(©)</td><td>(35)</td><td>(49)</td></tr><tr><td>Balance as of March 31, 2024</td><td>605</td><td>3,298</td><td>17</td><td>2,632</td><td>6,552</td></tr></tbody></table></br></br><table><thead><tr><th rowspan="2">Particulars</th><th colspan="4">Category of ROU asset</th><th rowspan="2">(In X crore) Total</th></tr><tr><th>Land</th><th>Buildings</th><th>Vehicles</th><th>Computers</th></tr></thead><tbody><tr><td>Balance as of J anuary 1, 2023</td><td>624</td><td>3,847</td><td>15</td><td>1,994</td><td>6,480</td></tr><tr><td>Additions*</td><td></td><td>228</td><td>2</td><td>651</td><td>881</td></tr><tr><td>Deletions</td><td></td><td>(33)</td><td>\</td><td>(124)</td><td>(157)</td></tr><tr><td>Depreciation</td><td></td><td>(171)</td><td>= w &lt;</td><td>(179)</td><td>(355)</td></tr><tr><td>Translation difference</td><td></td><td>25</td><td>—_</td><td>6</td><td>33</td></tr><tr><td>Balance as of March 31, 2023</td><td>623</td><td>3,896</td><td>15</td><td>2,348</td><td>6,882</td></tr></tbody></table></br></br><table><thead><tr><th rowspan="2">Particulars</th><th colspan="4">Category of ROU asset</th><th rowspan="2">Total</th></tr><tr><th>Land</th><th>Buildings</th><th>V ehicles</th><th>Computers</th></tr></thead><tbody><tr><td>Balance as of April 1, 2023</td><td>623</td><td>3,896</td><td>15</td><td>2,348</td><td>6,882</td></tr><tr><td>Additions*</td><td></td><td>394</td><td>12</td><td>1,872</td><td>2,278</td></tr><tr><td>Deletions</td><td>(10)</td><td>(181)</td><td>1</td><td>(755)</td><td>(947)</td></tr><tr><td>Impairment *</td><td></td><td>(88)</td><td>—</td><td>—</td><td>(88)</td></tr><tr><td>Depreciation</td><td>(6)</td><td>(728)</td><td>(10)</td><td>(851)</td><td>(1,595)</td></tr><tr><td>Translation difference</td><td>2</td><td>5</td><td>1</td><td>18</td><td>22</td></tr><tr><td>Balance as of March 31, 2024</td><td>605</td><td>3,298</td><td>17</td><td>2,632</td><td>6,552</td></tr></tbody></table></br></br><table><thead><tr><th rowspan="2">Particulars</th><th colspan="4">Category of ROU asset</th><th rowspan="2">(In R crore) Total</th></tr><tr><th>Land</th><th>Buildings</th><th>Vehicles</th><th>Computers</th></tr></thead><tbody><tr><td>Balance as of April 1, 2022</td><td>628</td><td>3,711</td><td>16</td><td>468</td><td>4,823</td></tr><tr><td>Additions*</td><td></td><td>847</td><td>8</td><td>2,646</td><td>3,501</td></tr><tr><td>Deletions</td><td></td><td>(45)</td><td>—</td><td>(364)</td><td>(409)</td></tr><tr><td>Depreciation</td><td></td><td>(671)</td><td>(10)</td><td>(499)</td><td>(1,186)</td></tr><tr><td>Translation difference</td><td></td><td>54</td><td>1</td><td>97</td><td>153</td></tr><tr><td>Balance as of March 31, 2023</td><td>623</td><td>3,896</td><td>15</td><td>2,348</td><td>6,882</td></tr></tbody></table></br></br><table><thead><tr><th rowspan="2">Particulars</th><th colspan="2">Asat</th></tr><tr><th>March 31, 2024</th><th>March 31, 2023</th></tr></thead><tbody><tr><td>Current lease liabilities</td><td>1,959</td><td>1,242</td></tr><tr><td>Non-current lease liabilities</td><td>6,400</td><td>7,057</td></tr><tr><td>Total</td><td>8,359</td><td>8,299</td></tr></tbody></table></br></br><table><thead><tr><th rowspan="2">Particulars</th><th>Asat March 31, 2024</th><th>(In % crore) March 31, 2023</th></tr><tr><th></th><th></th></tr></thead><tbody><tr><td colspan="3">Contingent liabilities :</td></tr><tr><td>Claims against the Group, not acknowledged as debts</td><td>3,583</td><td>4,762</td></tr><tr><td>[Amount paid to statutory authorities 28,754 crore (36,539 crore) |</td><td></td><td></td></tr></tbody></table></br></br><table><thead><tr><th>Particulars</th><th>Asat March 2024</th><th>(In % crore) March 2023</th></tr><tr><th></th><th>31,</th><th>31,</th></tr></thead><tbody><tr><td>Estimated amount of contracts remaining to be executed on capital contracts and not provided for (net of advances and deposits)"”</td><td>780</td><td>959</td></tr><tr><td>Other commitments*</td><td>79</td><td>92</td></tr></tbody></table></br></br><table><thead><tr><th rowspan="2">Particulars</th><th colspan="2">Three months ended March 31,</th><th colspan="2">(In X crore) Y ear ended March 31,</th></tr><tr><th>2024</th><th>2023</th><th>2024</th><th>2023</th></tr></thead><tbody><tr><td>Salaries and other short term employee benefits to whole-time directors (B[] and executive officers</td><td>30</td><td>25</td><td>113</td><td>111</td></tr><tr><td>Commission and other benefits to non-executive/independent directors</td><td>5</td><td>4</td><td>17</td><td>16</td></tr><tr><td>Total</td><td>35</td><td>29</td><td>130</td><td>127</td></tr></tbody></table></br></br><table><thead><tr><th rowspan="2">Particulars</th><th rowspan="2">Financial 
 Services *</th><th rowspan="2">Retail ®</th><th rowspan="2">Communic ation (&amp;)</th><th rowspan="2">Energy, 
 Utilities, 
 Resources and Services</th><th></th><th></th><th></th><th></th><th rowspan="2">(In % crore) Total</th></tr><tr><th>Manufacturing</th><th>Hi-Tech</th><th>Life Sciences @</th><th>All other segments ©</th></tr></thead><tbody><tr><td rowspan="2">Revenue from operations</td><td>10,010</td><td>5,429</td><td>4,666</td><td>5,068</td><td>5,589</td><td>3,316</td><td>2,762</td><td>1,083</td><td>37,923</td></tr><tr><td>10,818</td><td>5,537</td><td>4,411</td><td>4,825</td><td>5,078</td><td>2,989</td><td>2,681</td><td>1,102</td><td>37,441</td></tr><tr><td rowspan="2">Identifiable operating expenses</td><td>6,042</td><td>2,591</td><td>3,033</td><td>2,717</td><td>3,656</td><td>1,995</td><td>1,639</td><td>652</td><td>22,325</td></tr><tr><td>6,161</td><td>2,869</td><td>2,613</td><td>2,614</td><td>3,248</td><td>1,734</td><td>1,514</td><td>701</td><td>21,454</td></tr><tr><td rowspan="2">Allocated expenses</td><td>2,027</td><td>974</td><td>823</td><td>920</td><td>852</td><td>518</td><td>491</td><td>209</td><td>6,814</td></tr><tr><td>2,057</td><td>1,034</td><td>840</td><td>909</td><td>928</td><td>505</td><td>462</td><td>254</td><td>6,989</td></tr><tr><td rowspan="2">Segment operating income</td><td>1,941</td><td>1,864</td><td>810</td><td>1,431</td><td>1,081</td><td>803</td><td>632</td><td>222</td><td>8,784</td></tr><tr><td>2,600</td><td>1,634</td><td>958</td><td>1,302</td><td>902</td><td>750</td><td>705</td><td>147</td><td>8,998</td></tr><tr><td rowspan="2">Unallocable expenses</td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td>1,163</td></tr><tr><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td>1,121</td></tr><tr><td rowspan="2">Other income, net (Refer to Note 2.17)</td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td>2,729</td></tr><tr><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td>671</td></tr><tr><td rowspan="2">Finance cost</td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td>110</td></tr><tr><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td>82</td></tr><tr><td rowspan="2">Profit before tax</td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td>10,240</td></tr><tr><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td>8,466</td></tr><tr><td rowspan="2">ncome tax expense</td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td>2,265</td></tr><tr><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td>2,332</td></tr><tr><td rowspan="2">Net Profit</td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td>7,975</td></tr><tr><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td>6,134</td></tr><tr><td rowspan="2">Depreciation and amortization</td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td>1,163</td></tr><tr><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td>1,121</td></tr><tr><td>Non-cash expenses other than depreciation</td><td>and amortization</td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td>—</td></tr></tbody></table></br></br><table><thead><tr><th rowspan="2">Particulars</th><th rowspan="2">Financial 
 * Services</th><th rowspan="2">Retail ®</th><th rowspan="2">Communic (&amp;) ation</th><th rowspan="2">Energy, 
 Utilities, 
 Resources and Services</th><th></th><th></th><th></th><th></th><th>(In % crore)</th></tr><tr><th>Manufacturing</th><th>Hi-Tech</th><th>Life @ Sciences</th><th>All other segments ©</th><th>Total</th></tr></thead><tbody><tr><td rowspan="2">Revenue from operations</td><td>42,158</td><td>22,504</td><td>17,991</td><td>20,035</td><td>22,298</td><td>12,411</td><td>11,515</td><td>4,758</td><td>153,670</td></tr><tr><td>43,763</td><td>21,204</td><td>18,086</td><td>18,539</td><td>19,035</td><td>11,867</td><td>10,085</td><td>4,188</td><td>146,767</td></tr><tr><td rowspan="2">Identifiable operating expenses</td><td>24,782</td><td>11,704</td><td>11,071</td><td>10,838</td><td>14,596</td><td>7,232</td><td>6,716</td><td>2,938</td><td>89,877</td></tr><tr><td>24,990</td><td>10,892</td><td>11,101</td><td>9,923</td><td>12,493</td><td>6,959</td><td>5,834</td><td>2,801</td><td>84,993</td></tr><tr><td rowspan="2">Allocated expenses</td><td>8,052</td><td>3,918</td><td>3,232</td><td>3,674</td><td>3,505</td><td>2,026</td><td>1,901</td><td>1,060</td><td>27,368</td></tr><tr><td>7,930</td><td>3,916</td><td>3,226</td><td>3,461</td><td>3,429</td><td>1,949</td><td>1,685</td><td>1,048</td><td>26,644</td></tr><tr><td rowspan="2">Segment operating income</td><td>9,324</td><td>6,882</td><td>3,688</td><td>5,523</td><td>4,197</td><td>3,153</td><td>2,898</td><td>760</td><td>36,425</td></tr><tr><td>10,843</td><td>6,396</td><td>3,759</td><td>5,155</td><td>3,113</td><td>2,959</td><td>2,566</td><td>339</td><td>35,130</td></tr><tr><td rowspan="2">Unallocable expenses</td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td>4,678</td></tr><tr><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td>4,225</td></tr><tr><td rowspan="2">Other income, net (Refer to Note 2.17)</td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td>4,711</td></tr><tr><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td>2,701</td></tr><tr><td rowspan="2">Finance cost</td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td>470</td></tr><tr><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td>284</td></tr><tr><td rowspan="2">Profit before tax</td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td>35,988</td></tr><tr><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td>33,322</td></tr><tr><td>ncome tax expense</td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td>9,740</td></tr><tr><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td>9,214</td></tr><tr><td rowspan="2">Net Profit</td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td>26,248</td></tr><tr><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td>24,108</td></tr><tr><td rowspan="2">Depreciation and amortization expense</td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td>4,678</td></tr><tr><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td>4,225</td></tr><tr><td rowspan="2">Non-cash expenses other than depreciation</td><td>and amortization</td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td>—</td></tr><tr><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td>—</td></tr></tbody></table></br></br><table><thead><tr><th rowspan="2">Particulars</th><th rowspan="2">Note No.</th><th colspan="2">Three months ended March 31,</th><th colspan="2">Y ear ended March 31,</th></tr><tr><th>2024</th><th>2023</th><th>2024</th><th>2023</th></tr></thead><tbody><tr><td>Revenue from operations</td><td>2.16</td><td>37,923</td><td>37,441</td><td>153,670</td><td>146,767</td></tr><tr><td>Cost of Sales</td><td></td><td>26,748</td><td>26,011</td><td>107,413</td><td>102,353</td></tr><tr><td>Gross profit</td><td></td><td>11,175</td><td>11,430</td><td>46,257</td><td>44,414</td></tr><tr><td colspan="6">Operating expenses</td></tr><tr><td>Selling and marketing expenses</td><td></td><td>1,735</td><td>1,659</td><td>6,973</td><td>6,249</td></tr><tr><td>General and administration expenses</td><td></td><td>1,819</td><td>1,894</td><td>7,537</td><td>7,260</td></tr><tr><td>Total operating expenses</td><td></td><td>3,554</td><td>3,553</td><td>14,510</td><td>13,509</td></tr><tr><td>Operating profit</td><td></td><td>7,621</td><td>7,877</td><td>31,747</td><td>30,905</td></tr><tr><td>Other income, net</td><td>2.17</td><td>2,729</td><td>671</td><td>4,711</td><td>2,701</td></tr><tr><td>Finance cost</td><td></td><td>110</td><td>82</td><td>470</td><td>284</td></tr><tr><td>Profit before tax</td><td></td><td>10,240</td><td>8,466</td><td>35,988</td><td>33,322</td></tr><tr><td colspan="6">Tax expense:</td></tr><tr><td>Current tax</td><td>2.15</td><td>1,173</td><td>2,260</td><td>8,390</td><td>9,287</td></tr><tr><td>Deferred tax</td><td>2.15</td><td>1,092</td><td>72</td><td>1,350</td><td>(73)</td></tr><tr><td>Profit for the period</td><td></td><td>7,975</td><td>6,134</td><td>26,248</td><td>24,108</td></tr><tr><td colspan="6"></td></tr><tr><td colspan="6" rowspan="2">Other comprehensive income Items that will not be reclassified to loss</td></tr><tr><td>subsequently profit or</td><td></td><td></td><td></td><td></td><td></td></tr><tr><td>of the defined benefit</td><td></td><td>26</td><td>25</td><td>120</td><td></td></tr><tr><td>Remeasurement net liability/asset, net</td><td></td><td></td><td></td><td></td><td>8</td></tr><tr><td>Equity instruments through other comprehensive income, net</td><td></td><td>(12)</td><td>(15)</td><td>19</td><td>™</td></tr><tr><td></td><td></td><td>14</td><td>10</td><td>139</td><td>1</td></tr><tr><td colspan="6" rowspan="2">Items that will be reclassified subsequently to profit or loss Fair value derivatives cash flow 28 36 11</td></tr><tr><td>changes on designated as hedge, net</td><td></td><td></td><td></td><td></td><td>(7</td></tr><tr><td>Exchange differences on translation of foreign operations</td><td></td><td>(231)</td><td>61</td><td>226</td><td>776</td></tr><tr><td>Fair value changes on investments, net</td><td></td><td>37</td><td>42</td><td>144</td><td>(256)</td></tr><tr><td></td><td></td><td>(166)</td><td>139</td><td>381</td><td>513</td></tr><tr><td>Total other comprehensive income / (loss), net of tax</td><td></td><td>(152)</td><td>149</td><td>520</td><td>514</td></tr><tr><td>Total comprehensive income for the period</td><td></td><td>7,823</td><td>6,283</td><td>26,768</td><td>24,622</td></tr><tr><td>Profit attributable to:</td><td></td><td></td><td></td><td></td><td></td></tr><tr><td>Owners of the Company</td><td></td><td>7,969</td><td>6,128</td><td>26,233</td><td>24,095</td></tr><tr><td>Non-controlling interests</td><td></td><td>6</td><td>6</td><td>15</td><td>13</td></tr><tr><td></td><td></td><td>7,975</td><td>6,134</td><td>26,248</td><td>24,108</td></tr><tr><td colspan="6">Total comprehensive income attributable to:</td></tr><tr><td>Owners of the Company</td><td></td><td>7,821</td><td>6,276</td><td>26,754</td><td>24,598</td></tr><tr><td rowspan="2">Non-controlling interests</td><td></td><td>2</td><td>7</td><td>14</td><td>24</td></tr><tr><td></td><td>7,823</td><td>6,283</td><td>26,768</td><td>24,622</td></tr></tbody></table></br></br>

##### Creating Embeddings


In [2]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_community.chat_models import ChatOllama

In [39]:
summary_chain = (
    {"doc": lambda x: x}
    | ChatPromptTemplate.from_template("Summarize the following tables or text given below:\n\n{doc}")
    | ChatOllama(model="qwen2.5:3b", temperature=0.1)  # Use Ollama's Qwen model
    | StrOutputParser()
)

C:\Users\Manikpuriya\AppData\Local\Temp\ipykernel_1524\2652247782.py:4: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  | ChatOllama(model="qwen2.5:3b", temperature=0.1)  # Use Ollama's Qwen model


In [40]:
# Table summaries
tables_content = [i.page_content for i in table_elements]
teable_summaries = summary_chain.batch(tables_content, {"max_concurrency": 5})

In [41]:

teable_summaries[:2]

['The provided text appears to be a table containing references to various sections within an Interim Condensed Consolidated Financial Statements document, along with their corresponding page numbers. Here is a summary:\n\n- The table lists different financial statements and notes in the order they appear:\n  - "Condensed Consolidated Balance Sheet" on Page 1\n  - "Condensed Consolidated Statement of Profit and Loss"\n  - "Condensed Consolidated Statement of Changes in Equity" on Page E (E likely stands for End or the end of the document)\n  - "Condensed Consolidated Statement of Cash Flows" on Page 5\n\n- There is also a section titled "Overview and Notes to the Interim Condensed Consolidated Financial Statements":\n  - It includes an Overview, Basis of preparation of financial statements, Basis of consolidation, Use of estimates and judgments, Critical accounting estimates and judgments (pages R7, S, and 7 respectively)\n  \n- The notes section covers various topics such as:\n  - Bus

In [42]:
# Text summaries
text_content = [i.page_content for i in text_elements]
text_summaries = summary_chain.batch(text_content, {"max_concurrency": 5})

##### Multivector Retrieval

In [3]:
import uuid
from langchain.vectorstores import Chroma
from langchain.storage import InMemoryStore
from langchain.schema.document import Document
from langchain.embeddings import OllamaEmbeddings 
from langchain.retrievers.multi_vector import MultiVectorRetriever

In [ ]:
store = InMemoryStore()
id_key = "doc_id"

# The vectorstore to use to index the child chunks
vectorstore = Chroma(
    collection_name="financials",
    embedding_function=OllamaEmbeddings(model="mxbai-embed-large"), 
    persist_directory="./chroma_data",
)

C:\Users\Manikpuriya\AppData\Local\Temp\ipykernel_11784\3991549354.py:7: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embedding_function=OllamaEmbeddings(model="mxbai-embed-large"),  # Use Ollama's mxbai-embed-large model
C:\Users\Manikpuriya\AppData\Local\Temp\ipykernel_11784\3991549354.py:5: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(


In [5]:
# The retriever 
retriever = MultiVectorRetriever(
    vectorstore=vectorstore,
    docstore=store,
    id_key=id_key,
)

### Embedding Storage

##### 1) Table summaries embeddings and storage

In [ ]:

doc_ids = [str(uuid.uuid4()) for _ in table_elements]
summary_tables = [Document(page_content=s, metadata={id_key: doc_ids[i]}) for i, s in enumerate(teable_summaries)]
retriever.vectorstore.add_documents(summary_tables)
retriever.docstore.mset(list(zip(doc_ids, table_elements)))

##### 2) Text summaries embeddings and storage

In [ ]:
# Text summaries embeddings and storage
doc_ids = [str(uuid.uuid4()) for _ in text_elements]
summary_texts = [Document(page_content=s, metadata={id_key: doc_ids[i]}) for i, s in enumerate(text_summaries)]
retriever.vectorstore.add_documents(summary_texts)
retriever.docstore.mset(list(zip(doc_ids, text_elements)))

##### 1st Query and Response

In [8]:
def elements_to_markdown(elements):
    """
    Converts a list of Element objects to Markdown format.

    Args:
        elements: A list of Element objects.

    Returns:
        A string containing the Markdown representation of the elements.
    """
    markdown_text = ""
    for element in elements:
        if element.type == 'text':
            markdown_text += f"{element.page_content}\n\n"
        elif element.type == 'table':
            markdown_text += f"```\n{element.page_content}\n```\n\n" 
        else:
            markdown_text += f"**Unsupported element type:** {element.type}\n\n"
    return markdown_text


In [15]:
retrieved_first_response = retriever.invoke("What is the gross profit for Q3 2024?")

In [14]:
print(retrieved_first_response)

[]


In [10]:
print(elements_to_markdown(retrieved_first_response))

In [ ]:
retriever_first_response_page_content = retrieved_first_response[0].page_content
print(retriever_first_response_page_content)

In [50]:
Markdown(retriever_first_response_page_content)

For the three months and year ended March 31, 2024 and March 31, 2023:

##### 2nd Query and Response

In [51]:
retrieved_second_response = retriever.invoke("How do the net income and operating expenses compare for Q1 2024?")

In [ ]:
print(retrieved_second_response)

In [52]:
retriever_second_response_page_content = retrieved_second_response[0].page_content
print(retriever_second_response_page_content)

For the three months and year ended March 31, 2024 and March 31, 2023:


In [53]:
Markdown(retriever_second_response_page_content)

For the three months and year ended March 31, 2024 and March 31, 2023:

In [ ]:
print(elements_to_markdown(retrieved_second_response))

#### RAG Examples

In [5]:
from langchain.llms import Ollama
from operator import itemgetter
from langchain.schema.runnable import RunnablePassthrough

# Prompt template
template = """Answer the question based only on the following context, which can include text and tables:{context}Question: {question}"""
prompt = ChatPromptTemplate.from_template(template)

# LLM 
model = Ollama(model="qwen2.5:3b", temperature=0) 
#model = Ollama(model="Lllama2:7b", temperature=0) 

# RAG pipeline
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser())

C:\Users\Manikpuriya\AppData\Local\Temp\ipykernel_9192\1878109798.py:12: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  model = Ollama(model="qwen2.5:3b", temperature=0)


In [55]:
response = chain.invoke("How do the net income and operating expenses compare for Q1 2024?")
print(response)

To determine how the net income and operating expenses compare for Q1 2024, we need to look at the relevant sections of both tables provided. The second table contains information about profit before tax, income tax expense, and net profit, which are directly related to these metrics.

From the second table:

- Profit before tax: $8,466 million
- Income tax expense: $2,332 million

We can calculate the net profit as follows:
Net Profit = Profit Before Tax - Income Tax Expense
Net Profit = $8,466 million - $2,332 million = $6,134 million

The first table does not provide specific figures for Q1 2024. However, based on the information provided in the second table, we can see that:

- The net profit for Q1 2024 is $6,134 million.
- Operating expenses are listed under "Unallocable expenses" and "Other income, net (Refer to Note 2.17)" sections.

Unfortunately, without specific operating expense figures from the first table for Q1 2024, we cannot directly compare them with the net profit fi